In [1]:
from bs4 import BeautifulSoup
import requests
# ! pip install wikipedia
import wikipedia
from googlesearch import search 
from urllib.parse import urlparse

In [2]:
import re
output = re.sub(r'\d+', '', '...hm')
print (output)  # 'hello world'

...hm


In [3]:
from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [4]:
is_date('1 January')

True

In [5]:
#initialize Dataframe
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [6]:
#Utility
def tostr(arr):
  res=''
  if res is not None:
    for a in arr:
      res=res+a.strip()+','
    return res[:-1]
  return res

In [7]:
#Getting music director info
# ! pip install wptools
# ! pip install wikipedia
import wptools
import wikipedia
def getmusicdir(movie,year):
  #print(wikipedia.search(movie))
  try:
    #title=wikipedia.page(movie).url[30:]
    #print(title+'|'+movie+" "+year+" tamil")
    so = wptools.page(movie).get_parse()
    infobox = so.data['infobox']
    if infobox is not None:
      if 'music' in infobox.keys():
        #print(infobox['music'])
        alphanumeric = ""
        for char in infobox['music']:
          if char.isalnum() or char is '.':
            alphanumeric += char
          elif char is ' ':
            alphanumeric += char
          else:
            alphanumeric += '-'
            
        music_dir=[]
        for part in alphanumeric.split('-'):
          if part is not '' and part is not ' ':
            music_dir.append(part)
        
        music_list=[]
        for dist in music_dir:
          if dist.lower().strip() not in ['br','\n','editor','nbsp','music director','small','songs','score','original songs','singer','background score','artist','artists','composer'] and len(dist.lower().strip())>1:
            music_list.append(dist.strip())
        print(music_list)
        return music_list
      return []
    return []
  except:
    print("Exception");
    return []

In [8]:
#Get IMDB rating
def getrating(movie,year):
  for j in search(str(movie).strip()+' '+ str(year).strip() +' tamil imdb', tld="co.in", num=5, stop=5, pause=2): 
    parsed_uri = urlparse(j)
    result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    if(result.strip()=='https://www.imdb.com/'):
      print(j)
      webdata=requests.get(j).text
      soup=BeautifulSoup(webdata,'lxml')
      if(len(soup.find_all('span',itemprop='ratingValue'))>0):
        rating=soup.find_all('span',itemprop='ratingValue')[0].text
        return str(rating)
      break
  return ''

In [9]:
xtest=getmusicdir('2.0_(film)','2018')
xtest

en.wikipedia.org (parse) 2.0_(film)
en.wikipedia.org (imageinfo) File:2.0 film poster.jpg


['A. R. Rahman']


2.0 (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:2.0 film ...
  infobox: <dict(20)> name, image, caption, director, producer, wr...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:2...
  pageid: 48116360
  parsetree: <str(95112)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: 2.0 (film)
  wikibase: Q21095433
  wikidata_url: https://www.wikidata.org/wiki/Q21095433
  wikitext: <str(76573)> {{Short description|2018 Indian science f...
}


['A. R. Rahman']

In [10]:
def getAllMusic():
    df['cast_size']=''
    for index,row in df.iterrows():
        mlink=row['cast']
        if mlink is not None:
            arr=str(mlink).split(',')
            df.at[index,'cast_size']=len(arr)

    for index,row in df.iterrows():
        mlink=row['link']
        music_director=row['music']
        if(music_director.strip() is not ''):
            continue
        if mlink is not None:
            print(str(mlink).strip()[6:])
            dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
            strdir=tostr(dirs)
            print(str(index)+' | '+strdir)
            df.at[index,'music']=strdir


In [11]:
def getAllrating():
    for index,row in df.iterrows():
      mname=row['title']
      myear=row['year']
      if mname is not None:
        rating=getrating(mname,myear)
        print(mname+' | '+rating)
        df.at[index,'imdb']=rating.strip()

# 1989

In [31]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1989'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1989=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [32]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [84]:
for table in tables_1989:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1989'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),(all_data[music_index+2].text.strip() if music_member else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),(all_data[music_index+1].text.strip() if music_member else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),(all_data[music_index+2].text.strip() if music_member else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),(all_data[music_index].text.strip() if music_member else 'null')]


NameError: name 'tables_1989' is not defined

In [34]:
df=df.drop(0)
df

year                                title                     director  \
1    1989                      Aararo Aariraro                 K. Bhagyaraj   
2    1989                        Anbu Kattalai                  Yaar Kannan   
3    1989               Andru Peytha Mazhaiyil                  Ashok Kumar   
4    1989                Annakili Sonna Kathai                 S. Devarajan   
5    1989                        Annanukku Jai                Gangai Amaran   
6    1989                Apoorva Sagodharargal      Singeetam Srinivasa Rao   
7    1989                  Athaimadi Methaiadi         K. S. Gopalakrishnan   
8    1989               Chinna Chinna Aasaigal             Muktha S. Sundar   
9    1989                        Chinnappadass              C. V. Rajendran   
10   1989                         Dharma Devan               A. L. N. Mohan   
11   1989                       Dharmam Vellum                  K. Rangaraj   
12   1989                           Dilli Babu              Chitralaya Gopu   
13   1989                             Dravidan            R. Krishnamoorthy   
14   1989                  Ellame En Thangachi                  Bala Ganesh   
15   1989                    En Arumai Manaivi                    Jai Kumar   
16   1989                           En Kanavar                Ashwini Kumar   
17   1989  En Purushanthaan Enakku Mattumthaan                     Manobala   
18   1989                 En Rathathin Rathame  K. VijayanSundar K. Vijayan   
19   1989                           En Thangai               A. Jagannathan   
20   1989                  Enga Annan Varattum            R. Krishnamoorthy   
21   1989                  Enga Ooru Mappillai              T. P. Gajendran   
22   1989                   Enga Veettu Deivam    T. R. Ramesh-Nandha Kumar   
23   1989                     Enne Petha Raasa                       Siraaj   
24   1989                       Idhaya Dheepam              B. Nithyanandam   
25   1989                   Idhu Unga Kudumbam                     S. Umesh   
26   1989                  Kaalathai Vendravan              Mohan Gandhiram   
27   1989                     Kaaval Poonaigal        Kalippuli G. Sekharan   
28   1989              Kadhal Enum Nadhiyinile           M. K. I. Sukumaran   
29   1989               Kai Veesamma Kai Veesu                       Vinodh   
30   1989                           Kakka Kadi          N. S. Ramesh Khanna   
..    ...                                  ...                          ...   
73   1989                       Sandhya Raagam                Balu Mahendra   
74   1989                     Sangu Pushpangal                    Anbukkani   
75   1989                        Sariyana Jodi             Velu Prabhakaran   
76   1989            Sattathin Thirappu Vizhaa                   M. Bhaskar   
77   1989                                 Siva                     Ameerjan   
78   1989                           Sondham 16          T. S. Krishna Kumar   
79   1989                           Solaikuyil                        Rajan   
80   1989                         Sonthakkaran                      L. Raja   
81   1989                          Thaai Naadu                R. Aravindraj   
82   1989                  Thalaippu Seithigal                     K. Madhu   
83   1989              Thalaivanukkor Thalaivi             Vittal T. Gnanam   
84   1989                  Thangamana Purushan               Rama Narayanan   
85   1989                     Thangamana Raasa                V. Azhagappan   
86   1989                 Thangamani Rangamani               Rama Narayanan   
87   1989                   Thangachi Kalyanam                         Prem   
88   1989                      Thaaya Thaarama              T. P. Gajendran   
89   1989                       Thendral Sudum                     Manobala   
90   1989                       Thiruppu Munai        Kalaivanan Kannadasan   
91   1989              Unnai Naan Vazhthugiren         

In [19]:
getAllMusic()

Aararo_Aariraro


en.wikipedia.org (parse) Aararo_Aariraro
en.wikipedia.org (imageinfo) File:Aararo Aariraro.jpg
Aararo Aariraro (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Aararo Aa...
  infobox: <dict(15)> name, image, caption, director, writer, star...
  pageid: 34155452
  parsetree: <str(11188)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Aararo Aariraro
  wikibase: Q12915051
  wikidata_url: https://www.wikidata.org/wiki/Q12915051
  wikitext: <str(7614)> {{Use dmy dates|date=February 2017}}{{Use ...
}
en.wikipedia.org (parse) Anbu_Kattalai


['K. Bhagyaraj']
1 | K. Bhagyaraj
Anbu_Kattalai


en.wikipedia.org (imageinfo) File:AnbuKattalaifilm.jpg
Anbu Kattalai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:AnbuKatta...
  infobox: <dict(14)> name, image, caption, director, writer, stor...
  pageid: 41459781
  parsetree: <str(4066)> <root><template><title>Use dmy dates</tit...
  requests: <list(2)> parse, imageinfo
  title: Anbu Kattalai
  wikibase: Q12973525
  wikidata_url: https://www.wikidata.org/wiki/Q12973525
  wikitext: <str(2361)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Andru_Peytha_Mazhaiyil


['Ilaiyaraaja']
2 | Ilaiyaraaja
Andru_Peytha_Mazhaiyil


...

en.wikipedia.org (imageinfo) File:VettaiyaaduVilaiyaadu89.jpg
Vettaiyaadu Vilaiyaadu (1989 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Vettaiyaa...
  infobox: <dict(8)> name, image, caption, director, starring, mus...
  pageid: 41878167
  parsetree: <str(3224)> <root><template><title>About</title><part...
  requests: <list(2)> parse, imageinfo
  title: Vettaiyaadu Vilaiyaadu (1989 film)
  wikibase: Q16313217
  wikidata_url: https://www.wikidata.org/wiki/Q16313217
  wikitext: <str(1679)> {{About||the 2006 film of same name|Vettai...
}
en.wikipedia.org (parse) ex.php?title=Vizhiyora_Kavidhaigal&actio...


['Chandrabose', 'Chandrabose']
100 | Chandrabose,Chandrabose
ex.php?title=Vizhiyora_Kavidhaigal&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Yogam_Rajayogam&action=edit...


Exception
101 | 
ex.php?title=Yogam_Rajayogam&action=edit&redlink=1
Exception
102 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [20]:
getAllrating()

https://www.imdb.com/title/tt0155468/
Aararo Aariraro | 6.7
https://www.imdb.com/title/tt0318297/
Anbu Kattalai | 
Andru Peytha Mazhaiyil | 
https://www.imdb.com/title/tt0273429/
Annakili Sonna Kathai | 7.0
https://www.imdb.com/title/tt0318305/
Annanukku Jai | 
https://www.imdb.com/title/tt0096827/
Apoorva Sagodharargal | 8.3
https://www.imdb.com/title/tt0215531/
Athaimadi Methaiadi | 
https://www.imdb.com/title/tt10174254/
Chinna Chinna Aasaigal | 
https://www.imdb.com/title/tt0319183/
Chinnappadass | 6.2
https://www.imdb.com/title/tt3422348/
Dharma Devan | 
https://www.imdb.com/title/tt0319289/
Dharmam Vellum | 
https://www.imdb.com/title/tt10189474/
Dilli Babu | 
https://www.imdb.com/title/tt0273181/
Dravidan | 3.9
https://www.imdb.com/title/tt8947968/
Ellame En Thangachi | 
https://www.imdb.com/title/tt2725182/
En Arumai Manaivi | 
https://www.imdb.com/title/tt2373548/
En Kanavar | 4.8
https://www.imdb.com/title/tt0288509/
En Purushanthaan Enakku Mattumthaan | 5.7
https://www.imdb.

In [22]:
df

year                                title                     director  \
1    1989                      Aararo Aariraro                 K. Bhagyaraj   
2    1989                        Anbu Kattalai                  Yaar Kannan   
3    1989               Andru Peytha Mazhaiyil                  Ashok Kumar   
4    1989                Annakili Sonna Kathai                 S. Devarajan   
5    1989                        Annanukku Jai                Gangai Amaran   
6    1989                Apoorva Sagodharargal      Singeetam Srinivasa Rao   
7    1989                  Athaimadi Methaiadi         K. S. Gopalakrishnan   
8    1989               Chinna Chinna Aasaigal             Muktha S. Sundar   
9    1989                        Chinnappadass              C. V. Rajendran   
10   1989                         Dharma Devan               A. L. N. Mohan   
11   1989                       Dharmam Vellum                  K. Rangaraj   
12   1989                           Dilli Babu              Chitralaya Gopu   
13   1989                             Dravidan            R. Krishnamoorthy   
14   1989                  Ellame En Thangachi                  Bala Ganesh   
15   1989                    En Arumai Manaivi                    Jai Kumar   
16   1989                           En Kanavar                Ashwini Kumar   
17   1989  En Purushanthaan Enakku Mattumthaan                     Manobala   
18   1989                 En Rathathin Rathame  K. VijayanSundar K. Vijayan   
19   1989                           En Thangai               A. Jagannathan   
20   1989                  Enga Annan Varattum            R. Krishnamoorthy   
21   1989                  Enga Ooru Mappillai              T. P. Gajendran   
22   1989                   Enga Veettu Deivam    T. R. Ramesh-Nandha Kumar   
23   1989                     Enne Petha Raasa                       Siraaj   
24   1989                       Idhaya Dheepam              B. Nithyanandam   
25   1989                   Idhu Unga Kudumbam                     S. Umesh   
26   1989                  Kaalathai Vendravan              Mohan Gandhiram   
27   1989                     Kaaval Poonaigal        Kalippuli G. Sekharan   
28   1989              Kadhal Enum Nadhiyinile           M. K. I. Sukumaran   
29   1989               Kai Veesamma Kai Veesu                       Vinodh   
30   1989                           Kakka Kadi          N. S. Ramesh Khanna   
..    ...                                  ...                          ...   
73   1989                       Sandhya Raagam                Balu Mahendra   
74   1989                     Sangu Pushpangal                    Anbukkani   
75   1989                        Sariyana Jodi             Velu Prabhakaran   
76   1989            Sattathin Thirappu Vizhaa                   M. Bhaskar   
77   1989                                 Siva                     Ameerjan   
78   1989                           Sondham 16          T. S. Krishna Kumar   
79   1989                           Solaikuyil                        Rajan   
80   1989                         Sonthakkaran                      L. Raja   
81   1989                          Thaai Naadu                R. Aravindraj   
82   1989                  Thalaippu Seithigal                     K. Madhu   
83   1989              Thalaivanukkor Thalaivi             Vittal T. Gnanam   
84   1989                  Thangamana Purushan               Rama Narayanan   
85   1989                     Thangamana Raasa                V. Azhagappan   
86   1989                 Thangamani Rangamani               Rama Narayanan   
87   1989                   Thangachi Kalyanam                         Prem   
88   1989                      Thaaya Thaarama              T. P. Gajendran   
89   1989                       Thendral Sudum                     Manobala   
90   1989                       Thiruppu Munai        Kalaivanan Kannadasan   
91   1989              Unnai Naan Vazhthugiren         

In [23]:
df.to_csv('1989_movies.csv')

In [2]:
'is'is not'is'

False

# 1988

In [14]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1988'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1988=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [15]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [16]:
for table in tables_1988:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1988'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),(all_data[music_index+2].text.strip() if music_member else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),(all_data[music_index+1].text.strip() if music_member else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),(all_data[music_index+2].text.strip() if music_member else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),(all_data[music_index].text.strip() if music_member else 'null')]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Adhu Antha Kaalam|Valampuri John|Sarath Babu, Lakshmi, Charan Raj, Rajeev|null|Bhanu Revathi Combines|/wiki/Adhu_Antha_Kaalam
2!Adimai Vilangu|||null||/wiki/Adimai_Vilangu
3!Agni Natchathiram|Mani Ratnam|Prabhu, Karthik, Amala, Nirosha, Vijayakumar, Jayachitra, Sumithra, V. K. Ramasamy, Janagaraj, Thara|null|Sujatha Productions|/wiki/Agni_Natchathiram
4!Anbe En Anbe|K. Murali Raj|Anand, Madhuri|null|Yogalaya Productions|/w/index.php?title=Anbe_En_Anbe_(film)&action=edit&redlink=1
5!Annanagar Mudhal Theru|Balu Anand|Sathyaraj, Ambika, Radha, Anand, Janagaraj, Manorama|null|Vivekanandha Film Circuit|/wiki/Annanagar_Mudhal_Theru
6!Aval Mella Sirithal|M. N. Jai Sundar|Murali, Seetha, Madhuri, S. S. Chandran, Chinni Jayanth, Charle|null|Sri Kavitha Cine Arts|/wiki/Aval_Mella_Sirithal
7!Chutti Poonai||Baby Shalini|null|Lakshmi Raja Films|/w/index.php?title=Chutti_Poonai&action=edit&redlink=1
8!Dharmathin Thalaivan|S. P. Muthuraman|Rajinika

59!Paarthal Pasu|K. S. Gopalakrishnan|Ramarajan, Chandrasekhar, Pallavi, Sri Bharathi, Dharani, S. S. Chandran, Senthil|null|C. R. Production|/wiki/Paarthal_Pasu
60!Paasa Paravaigal|V. M. C. Haneefa|Sivakumar, Mohan, Lakshmi, Raadhika, S. S. Chandran|null|Poompuhar Productions|/wiki/Paasa_Paravaigal
61!Paatti Sollai Thattathe|Rajasekhar|Pandiarajan, Urvashi, Manorama, S. S. Chandran, Silk Smitha, Disco Shanti|null|AVM Productions|/wiki/Paatti_Sollai_Thattathe
62!Paimara Kappal|K. Radha|S. S. R. Rajendrakumar, Pallavi, Janagaraj, Kalaiselvi, Chandrasekhar|null|Nirmala Aajaa Creations|/w/index.php?title=Paimara_Kappal&action=edit&redlink=1
63!Palaivanathil Pattamoppchi|Durai|Chandrasekhar, Senthil|null|Sunitha Cine Arts|/w/index.php?title=Palaivanathil_Pattamoppchi&action=edit&redlink=1
64!Paravaigal Palavitham|Robert-Rajasekaran|Ramki, Nirosha, Janagaraj, Thara|null|Esskay Film Combines|/wiki/Paravaigal_Palavitham
65!Paris Paris|Rajan|Senthil, V. K. Ramasamy|null|Cowjoy Enterprises|/w/i

In [17]:
df=df.drop(0)
df

year                            title             director  \
1    1988                Adhu Antha Kaalam       Valampuri John   
2    1988                   Adimai Vilangu                        
3    1988                Agni Natchathiram          Mani Ratnam   
4    1988                     Anbe En Anbe        K. Murali Raj   
5    1988           Annanagar Mudhal Theru           Balu Anand   
6    1988              Aval Mella Sirithal     M. N. Jai Sundar   
7    1988                    Chutti Poonai                        
8    1988             Dharmathin Thalaivan     S. P. Muthuraman   
9    1988                      Dhayam Onnu           Selvakumar   
10   1988          En Bommukutty Ammavukku                Fazil   
11   1988                En Jeevan Paduthu       R. Sundarrajan   
12   1988             En Thamizh En Makkal    Santhana Bharathi   
13   1988           En Thangachi Padichava              P. Vasu   
14   1988               En Thangai Kalyani          T. Rajendar   
15   1988                 En Uyir Kannamma         Sivachandran   
16   1988             En Vazhi Thani Vazhi        V. Azhagappan   
17   1988             Enga Ooru Kavalkaran      T. P. Gajendran   
18   1988             Ennai Vittu Pogaathe           T. K. Bose   
19   1988           Ganam Courtar Avargale           Manivannan   
20   1988                     Guru Sishyan     S. P. Muthuraman   
21   1988                  Idhu Namma Aalu          Balakumaran   
22   1988                            Illam           I. V. Sasi   
23   1988                    Irandil Ondru        V. Azhagappan   
24   1988                Ithu Engal Neethi  S. A. Chandrasekhar   
25   1988                 Ithuthan Arambam           Raj Barath   
26   1988                 Jadikketha Moodi      Umesh-Prabhakar   
27   1988                            Jeeva     Pratap K. Pothen   
28   1988  Kaalaiyum Neeye Maalaiyum Neeye       R. Sundarrajan   
29   1988               Kadarkarai Thaagam               Ilango   
30   1988         Kai Koduppal Karpagambal        S. Jagadeesan   
..    ...                              ...                  ...   
73   1988                   Puthiya Vaanam     R. V. Udayakumar   
74   1988              Raasave Unnai Nambi           K. J. Bose   
75   1988                   Raththa Dhanam         Sivachandran   
76   1988              Rayilukku Neramachu       Bharathi Mohan   
77   1988               Rendum Rendum Anju         Jayabharathi   
78   1988              Sahadevan Mahadevan       Rama Narayanan   
79   1988                 Sakkarai Panthal        Gangai Amaran   
80   1988                           Sathya       Suresh Krissna   
81   1988                  Senthoora Poove        P. R. Devaraj   
82   1988          Shenbagamae Shenbagamae        Gangai Amaran   
83   1988                    Sigappu Thali        R.Thyagarajan   
84   1988          Solla Thudikuthu Manasu             B. Lenin   
85   1988                  Soora Samhaaram   Chithra Lakshmanan   
86   1988    Sudhanthira Naattin Adimaigal  S. A. Chandrasekhar   
87   1988                     Thaai Paasam    R. Krishnamoorthy   
88   1988                    Thaimel Aanai              L. Raja   
89   1988              Thambi Thanga Kambi           K. Shankar   
90   1988                   Thanga Kalasam         London Gopal   
91   1988                   Thappu Kanakku          Jeeva Balan   
92   1988              Thenpandi Seemayile      C. P. Kollappan   
93   1988                 Therkathi Kallan         P. Kalaimani   
94   1988             Ullathil Nalla Ullam           Manivannan   
95   1988             Unnal Mudiyum Thambi       K. Balachander   
96   1988                   Urimai Geetham     R. V. Udayakumar   
97   1988           Uzhaithu Vaazha Vendum             Ameerjan   
98   1988             Vacha Kuri Thappathu   M. R. Vijaychander   
99   1988                       Valaikaapu         A. Jayaraman   
100  1988                         Vasanthi      Chit

In [21]:
getAllMusic()

Adimai_Vilangu


en.wikipedia.org (parse) Adimai_Vilangu
Adimai Vilangu (en) data
{
  infobox: <dict(8)> name, director, producer, starring, music, re...
  pageid: 42566935
  parsetree: <str(2777)> <root><template><title>Orphan</title><par...
  requests: <list(1)> parse
  title: Adimai Vilangu
  wikibase: Q16825454
  wikidata_url: https://www.wikidata.org/wiki/Q16825454
  wikitext: <str(1363)> {{Orphan|date=May 2018}}{{Use dmy dates|da...
}
en.wikipedia.org (parse) ex.php?title=Kalloori_Kanavugal&action=e...


['ISAIVAANAR V.R.SAMBATHSELVAM.']
2 | ISAIVAANAR V.R.SAMBATHSELVAM.
ex.php?title=Kalloori_Kanavugal&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Oomai_Thurai&action=edit&re...


Exception
34 | 
ex.php?title=Oomai_Thurai&action=edit&redlink=1
Exception
55 | 
ex.php?title=Palaivanathil_Pattamoppchi&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Palaivanathil_Pattamoppchi&...


Exception
63 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [22]:
getAllrating()

https://www.imdb.com/search/title/?title_type=feature&year=1988-01-01,1988-12-31&languages=ta
Adhu Antha Kaalam | 
https://www.imdb.com/title/tt7208618/
Adimai Vilangu | 6.5
https://www.imdb.com/title/tt0094621/
Agni Natchathiram | 7.7
https://www.imdb.com/title/tt0319361/
Anbe En Anbe | 
https://www.imdb.com/title/tt8914562/
Annanagar Mudhal Theru | 5.4
https://www.imdb.com/title/tt8919500/
Aval Mella Sirithal | 
https://www.imdb.com/title/tt7208618/
Chutti Poonai | 6.5
https://www.imdb.com/title/tt0319290/
Dharmathin Thalaivan | 7.0
https://www.imdb.com/title/tt0319287/
Dhayam Onnu | 
https://www.imdb.com/title/tt0214666/
En Bommukutty Ammavukku | 7.1
https://www.imdb.com/title/tt0319348/
En Jeevan Paduthu | 7.0
https://www.imdb.com/title/tt1440163/
En Thamizh En Makkal | 
https://www.imdb.com/title/tt10226498/
En Thangachi Padichava | 
https://www.imdb.com/title/tt0259293/
En Thangai Kalyani | 5.3
https://www.imdb.com/title/tt0319349/
En Uyir Kannamma | 5.9
https://www.imdb.com/titl

In [23]:
df

year                            title             director  \
1    1988                Adhu Antha Kaalam       Valampuri John   
2    1988                   Adimai Vilangu                        
3    1988                Agni Natchathiram          Mani Ratnam   
4    1988                     Anbe En Anbe        K. Murali Raj   
5    1988           Annanagar Mudhal Theru           Balu Anand   
6    1988              Aval Mella Sirithal     M. N. Jai Sundar   
7    1988                    Chutti Poonai                        
8    1988             Dharmathin Thalaivan     S. P. Muthuraman   
9    1988                      Dhayam Onnu           Selvakumar   
10   1988          En Bommukutty Ammavukku                Fazil   
11   1988                En Jeevan Paduthu       R. Sundarrajan   
12   1988             En Thamizh En Makkal    Santhana Bharathi   
13   1988           En Thangachi Padichava              P. Vasu   
14   1988               En Thangai Kalyani          T. Rajendar   
15   1988                 En Uyir Kannamma         Sivachandran   
16   1988             En Vazhi Thani Vazhi        V. Azhagappan   
17   1988             Enga Ooru Kavalkaran      T. P. Gajendran   
18   1988             Ennai Vittu Pogaathe           T. K. Bose   
19   1988           Ganam Courtar Avargale           Manivannan   
20   1988                     Guru Sishyan     S. P. Muthuraman   
21   1988                  Idhu Namma Aalu          Balakumaran   
22   1988                            Illam           I. V. Sasi   
23   1988                    Irandil Ondru        V. Azhagappan   
24   1988                Ithu Engal Neethi  S. A. Chandrasekhar   
25   1988                 Ithuthan Arambam           Raj Barath   
26   1988                 Jadikketha Moodi      Umesh-Prabhakar   
27   1988                            Jeeva     Pratap K. Pothen   
28   1988  Kaalaiyum Neeye Maalaiyum Neeye       R. Sundarrajan   
29   1988               Kadarkarai Thaagam               Ilango   
30   1988         Kai Koduppal Karpagambal        S. Jagadeesan   
..    ...                              ...                  ...   
73   1988                   Puthiya Vaanam     R. V. Udayakumar   
74   1988              Raasave Unnai Nambi           K. J. Bose   
75   1988                   Raththa Dhanam         Sivachandran   
76   1988              Rayilukku Neramachu       Bharathi Mohan   
77   1988               Rendum Rendum Anju         Jayabharathi   
78   1988              Sahadevan Mahadevan       Rama Narayanan   
79   1988                 Sakkarai Panthal        Gangai Amaran   
80   1988                           Sathya       Suresh Krissna   
81   1988                  Senthoora Poove        P. R. Devaraj   
82   1988          Shenbagamae Shenbagamae        Gangai Amaran   
83   1988                    Sigappu Thali        R.Thyagarajan   
84   1988          Solla Thudikuthu Manasu             B. Lenin   
85   1988                  Soora Samhaaram   Chithra Lakshmanan   
86   1988    Sudhanthira Naattin Adimaigal  S. A. Chandrasekhar   
87   1988                     Thaai Paasam    R. Krishnamoorthy   
88   1988                    Thaimel Aanai              L. Raja   
89   1988              Thambi Thanga Kambi           K. Shankar   
90   1988                   Thanga Kalasam         London Gopal   
91   1988                   Thappu Kanakku          Jeeva Balan   
92   1988              Thenpandi Seemayile      C. P. Kollappan   
93   1988                 Therkathi Kallan         P. Kalaimani   
94   1988             Ullathil Nalla Ullam           Manivannan   
95   1988             Unnal Mudiyum Thambi       K. Balachander   
96   1988                   Urimai Geetham     R. V. Udayakumar   
97   1988           Uzhaithu Vaazha Vendum             Ameerjan   
98   1988             Vacha Kuri Thappathu   M. R. Vijaychander   
99   1988                       Valaikaapu         A. Jayaraman   
100  1988                         Vasanthi      Chit

In [24]:
df.to_csv('1988_movies.csv')

# 1987

In [40]:
import re
output = re.sub(r'\d+', '', '...hm')
print (output)  # 'hello world'

...hm


In [35]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1987'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1987=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [36]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [37]:
for table in tables_1987:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1987'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Aalappirandhavan|A. S. Pragasam|Sathyaraj, Ambika, Silk Smitha, Master Tinku|null|Pragaas Productions|/wiki/Aalappirandhavan
2!Aankalai Nambathey|K. Alex Pandian|Pandiyan, Rekha, Ramya Krishnan, Senthil, V. K. Ramasamy, S. S. Chandran|null|Alamu Movies|/wiki/Aankalai_Nambathey
3!Aayusu Nooru|Ponmani Rajan|Pandiyan, Pandiarajan, Ranjini, Senthil, Kovai Sarala|null|Sampath Creations|/wiki/Aayusu_Nooru
4!Anand|C. V. Rajendran|Prabhu, Radha, Jayashree, Chinni Jayanth|null|Sivaji Productions|/wiki/Anand_(1987_film)
5!Ananda Aradhanai|Dinesh Baboo|Mohan, Suhasini, Lizy, Srividya, Senthil, Kovai Sarala|null|Jai Krishna Combines|/wiki/Ananda_Aradhanai
6!Anbulla Appa|A. C. Tirulokchandar|Sivaji Ganesan, Nadhiya, Rahman, V. K. Ramasamy, Y. G. Mahendra, Manorama, Sangeetha, Baby Sujitha|null|AVM Productions|/wiki/Anbulla_Appa
7!Anjatha Singam|M. R. Vijaychander|Prabhu, Sathyaraj, Nalini, Jaishankar, Silk Smitha, Anuradha, Senthil, Vanitha|null|

90!Uzhavan Magan|R. Aravindraj|Vijayakanth, Raadhika, Radha, Senthil, S. S. Chandran, Kovai Sarala|null|Ravuther Films|/wiki/Uzhavan_Magan
91!Vairakkiyam|K. Vijayan|Prabhu, Radha, Sowcar Janaki, Jaishankar, Manorama, Janagaraj|null|Sujatha Cine Arts|/wiki/Vairakkiyam
92!Valayal Satham|Jeevabalan|Murali, Bhagya, Raja, Madhuri, Chandrasekhar, M. N. Nambiar, Manorama, Janagaraj, Senthil|null|Balan Pictures|/wiki/Valayal_Satham
93!Vanna Kanavugal|Ameerjan|Karthik, Murali, Jayashree, Charle|null|Kavithalayaa Productions|/wiki/Vanna_Kanavugal
94!Vaazhga Valarga|Vijaya Krishnaraj|Pandiarajan, Radha Ravi, Saritha, Deepa, Senthil, Kovai Sarala|null|Niruma Creation|/w/index.php?title=Vaazhga_Valarga&action=edit&redlink=1
95!Vedham Pudhithu|P. Bharathiraja|Sathyaraj, Amala, Saritha, Raja, Janagaraj|null|Janani Art Creations|/wiki/Vedham_Pudhithu
96!Veerapandiyan|Karthick Raghunath|Sivaji Ganesan, Vijayakanth, Raadhika, Jaishankar, Sumithra, V. K. Ramasamy, Anuradha|null|Prakash Pictures|/wiki/Vee

In [38]:
df=df.drop(0)
df

year                        title              director  \
1    1987             Aalappirandhavan        A. S. Pragasam   
2    1987           Aankalai Nambathey       K. Alex Pandian   
3    1987                 Aayusu Nooru         Ponmani Rajan   
4    1987                        Anand       C. V. Rajendran   
5    1987             Ananda Aradhanai          Dinesh Baboo   
6    1987                 Anbulla Appa  A. C. Tirulokchandar   
7    1987               Anjatha Singam    M. R. Vijaychander   
8    1987         Arul Tharum Ayyappan            Dasarathan   
9    1987          Aval Mella Sirithal      M. N. Jai Sundar   
10   1987                  Chellakutti             K. Natraj   
11   1987        Chinna Kuyil Paaduthu           P. Madhavan   
12   1987      Chinna Poove Mella Pesu   Robert - Rajasekhar   
13   1987  Chinna Thambi Periya Thambi            Manivannan   
14   1987                 Cooliekkaran            Rajasekhar   
15   1987             Dhoorathu Pachai              Manobala   
16   1987                 Elangeswaran         T. R. Ramanna   
17   1987                    Ellaikodu      K. Jothipandiyan   
18   1987             Enga Chinna Rasa          K. Bhagyaraj   
19   1987         Enga Ooru Pattukaran         Gangai Amaran   
20   1987        Enga Veettu Ramayanan       K. Chozha Rajan   
21   1987                 Etikku Potti       R. Govindarajan   
22   1987           Evargal Indiyargal         S. Jagadeesan   
23   1987             Gramatthu Minnal           K. Rangaraj   
24   1987       Idhu Oru Thodar Kathai             Anu Mohan   
25   1987         Ini Oru Sudhanthiram            Manivannan   
26   1987        Iniya Uravu Poothathu         C. V. Sridhar   
27   1987            Ival Oru Pournami              T. P. K.   
28   1987   Ivargal Varungala Thoongal                Venkat   
29   1987                   Jallikattu            Manivannan   
30   1987                  Kaani Nilam          S. Arunmozhi   
..    ...                          ...                   ...   
72   1987             Poove Ilam Poove        Sirumugai Ravi   
73   1987            Poovizhi Vasalile                 Fazil   
74   1987           Puyal Paadum Pattu            Manivannan   
75   1987              Raja Mariyadhai    Karthick Raghunath   
76   1987                        Rekha             N. Raghav   
77   1987           Rettai Vaal Kuruvi         Balu Mahendra   
78   1987        Sattam Oru Vilayaattu   S. A. Chandrasekhar   
79   1987                 Shankar Guru               L. Raja   
80   1987                Sirai Paravai              Manobala   
81   1987           Solvathellam Unmai               Nethaji   
82   1987                 Thalidhaanam          R. C. Sakthi   
83   1987                  Thambathyam            K. Vijayan   
84   1987                    Thangachi      R. Krishnamurthy   
85   1987           Thaye Neeye Thunai            P. R. Somu   
86   1987         Theertha Karaiyinile            Manivannan   
87   1987     Thirumathi Oru Vegumathi                  Visu   
88   1987                      Thulasi              Ameerjan   
89   1987       Ullam Kavarndha Kalvan           Ashok Kumar   
90   1987                Uzhavan Magan         R. Aravindraj   
91   1987                  Vairakkiyam            K. Vijayan   
92   1987               Valayal Satham            Jeevabalan   
93   1987              Vanna Kanavugal              Ameerjan   
94   1987              Vaazhga Valarga     Vijaya Krishnaraj   
95   1987              Vedham Pudhithu       P. Bharathiraja   
96   1987                Veerapandiyan    Karthick Raghunath   
97   1987            Veeran Veluthambi        Rama Narayanan   
98   1987                   Velaikaran      S. P. Muthuraman   
99   1987                     Velicham     Sundar K. Vijayan   
100  1987          Velundu Vinaiyillai            K. Shankar   
101  1987                      Vilangu           P. Jayadevi   

                                    

In [39]:
getAllMusic()

ex.php?title=Rekha_(1987_film)&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Rekha_(1987_film)&action=ed...


Exception
76 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [41]:
getAllrating()

https://www.imdb.com/title/tt0275174/
Aalappirandhavan | 
https://www.imdb.com/title/tt10229830/
Aankalai Nambathey | 
https://www.imdb.com/title/tt8929186/
Aayusu Nooru | 
https://www.imdb.com/title/tt0092556/
Anand | 5.0
https://www.imdb.com/title/tt10229838/
Ananda Aradhanai | 
https://www.imdb.com/title/tt1435455/
Anbulla Appa | 6.2
https://www.imdb.com/title/tt8914540/
Anjatha Singam | 
https://www.imdb.com/title/tt8919118/
Arul Tharum Ayyappan | 
https://www.imdb.com/title/tt8919500/
Aval Mella Sirithal | 
https://www.imdb.com/title/tt7987840/
Chellakutti | 
https://www.imdb.com/title/tt0319181/
Chinna Kuyil Paaduthu | 4.6
https://www.imdb.com/title/tt2148420/
Chinna Poove Mella Pesu | 
https://www.imdb.com/title/tt0308113/
Chinna Thambi Periya Thambi | 6.0
Cooliekkaran | 
https://www.imdb.com/title/tt0319291/
Dhoorathu Pachai | 6.2
https://www.imdb.com/title/tt0320718/
Elangeswaran | 6.7
https://www.imdb.com/search/title/?languages=ta&sort=num_votes&title_type=feature&year=1987,

In [42]:
df

year                        title              director  \
1    1987             Aalappirandhavan        A. S. Pragasam   
2    1987           Aankalai Nambathey       K. Alex Pandian   
3    1987                 Aayusu Nooru         Ponmani Rajan   
4    1987                        Anand       C. V. Rajendran   
5    1987             Ananda Aradhanai          Dinesh Baboo   
6    1987                 Anbulla Appa  A. C. Tirulokchandar   
7    1987               Anjatha Singam    M. R. Vijaychander   
8    1987         Arul Tharum Ayyappan            Dasarathan   
9    1987          Aval Mella Sirithal      M. N. Jai Sundar   
10   1987                  Chellakutti             K. Natraj   
11   1987        Chinna Kuyil Paaduthu           P. Madhavan   
12   1987      Chinna Poove Mella Pesu   Robert - Rajasekhar   
13   1987  Chinna Thambi Periya Thambi            Manivannan   
14   1987                 Cooliekkaran            Rajasekhar   
15   1987             Dhoorathu Pachai              Manobala   
16   1987                 Elangeswaran         T. R. Ramanna   
17   1987                    Ellaikodu      K. Jothipandiyan   
18   1987             Enga Chinna Rasa          K. Bhagyaraj   
19   1987         Enga Ooru Pattukaran         Gangai Amaran   
20   1987        Enga Veettu Ramayanan       K. Chozha Rajan   
21   1987                 Etikku Potti       R. Govindarajan   
22   1987           Evargal Indiyargal         S. Jagadeesan   
23   1987             Gramatthu Minnal           K. Rangaraj   
24   1987       Idhu Oru Thodar Kathai             Anu Mohan   
25   1987         Ini Oru Sudhanthiram            Manivannan   
26   1987        Iniya Uravu Poothathu         C. V. Sridhar   
27   1987            Ival Oru Pournami              T. P. K.   
28   1987   Ivargal Varungala Thoongal                Venkat   
29   1987                   Jallikattu            Manivannan   
30   1987                  Kaani Nilam          S. Arunmozhi   
..    ...                          ...                   ...   
72   1987             Poove Ilam Poove        Sirumugai Ravi   
73   1987            Poovizhi Vasalile                 Fazil   
74   1987           Puyal Paadum Pattu            Manivannan   
75   1987              Raja Mariyadhai    Karthick Raghunath   
76   1987                        Rekha             N. Raghav   
77   1987           Rettai Vaal Kuruvi         Balu Mahendra   
78   1987        Sattam Oru Vilayaattu   S. A. Chandrasekhar   
79   1987                 Shankar Guru               L. Raja   
80   1987                Sirai Paravai              Manobala   
81   1987           Solvathellam Unmai               Nethaji   
82   1987                 Thalidhaanam          R. C. Sakthi   
83   1987                  Thambathyam            K. Vijayan   
84   1987                    Thangachi      R. Krishnamurthy   
85   1987           Thaye Neeye Thunai            P. R. Somu   
86   1987         Theertha Karaiyinile            Manivannan   
87   1987     Thirumathi Oru Vegumathi                  Visu   
88   1987                      Thulasi              Ameerjan   
89   1987       Ullam Kavarndha Kalvan           Ashok Kumar   
90   1987                Uzhavan Magan         R. Aravindraj   
91   1987                  Vairakkiyam            K. Vijayan   
92   1987               Valayal Satham            Jeevabalan   
93   1987              Vanna Kanavugal              Ameerjan   
94   1987              Vaazhga Valarga     Vijaya Krishnaraj   
95   1987              Vedham Pudhithu       P. Bharathiraja   
96   1987                Veerapandiyan    Karthick Raghunath   
97   1987            Veeran Veluthambi        Rama Narayanan   
98   1987                   Velaikaran      S. P. Muthuraman   
99   1987                     Velicham     Sundar K. Vijayan   
100  1987          Velundu Vinaiyillai            K. Shankar   
101  1987                      Vilangu           P. Jayadevi   

                                    

In [44]:
df.to_csv('1987_movies.csv')

# 1986

In [45]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1986'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1986=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [46]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [47]:
for table in tables_1986:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1986'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Aayiram Kannudayaal|K. Shankar|Padmini, Jeevitha, Prabha, Baby Shalini, Rajeev, Senthil|null|Sasivarnam Films|/wiki/Aayiram_Kannudayaal
2!Aayiram Pookkal Malarattum|E. Ramadoss|Mohan, Seetha, Ranjini, Goundamani, Senthil, Janagaraj|null|Motherland Pictures|/wiki/Aayiram_Pookkal_Malarattum
3!Adutha Veedu|Rama Narayanan|S. Ve. Shekher, Chandrasekhar, Ilavarasi, Madhuri|null|Visakam Art Productions|/w/index.php?title=Adutha_Veedu&action=edit&redlink=1
4!Africavil Appu|Vijay Anand|Nalini, Suresh|null|Sri Rajeswari Creations|/w/index.php?title=Africavil_Appu&action=edit&redlink=1
5!Amman Kovil Kizhakale|R. Sundarrajan|Vijayakanth, Radha, Ravichandran, Srividya, Senthil|null|V. N. S. Productions|/wiki/Amman_Kovil_Kizhakale
6!Anandha Kanneer|K. Vijayan|Sivaji Ganesan, Lakshmi, Visu, Rajeev, Nizhalgal Ravi, Jayashree, Rajyalakshmi, Janagaraj|null|Sivaji Productions|/wiki/Anandha_Kanneer
7!Annai En Dheivam|R. Krishnamoorthy|Vijayakanth, Jaish

86!Raja Nee Vaazhga|C. V. Rajendran|Prabhu, Ambika, Sathyaraj, Nizhalgal Ravi, Anuradha|null|Garuda Films|/w/index.php?title=Raja_Nee_Vaazhga&action=edit&redlink=1
87!Rasigan Oru Rasigai|Balu Anand|Sathyaraj, Ambika, Jaishankar, Goundamani, Senthil|null|Vasan Productions|/w/index.php?title=Rasigan_Oru_Rasigai&action=edit&redlink=1
88!Revathy|Sundar K. Vijayan|Revathi, Suresh|null|J. V. Movies|/w/index.php?title=Revathy_(film)&action=edit&redlink=1
89!Saadhanai|A. S. Pragasam|Sivaji Ganesan, K. R. Vijaya, Prabhu, Nalini, Y. G. Mahendra|null|Pragaas Productions|/wiki/Saadhanai
90!Samsaram Adhu Minsaram|Visu|Visu, Lakshmi, Raghuvaran, Chandrasekhar, Kismu, Manorama, Madhuri, Ilavarasi, Kamala Kamesh|null|AVM Productions|/wiki/Samsaram_Adhu_Minsaram
91!Sarvam Sakthimayam|P. R. Somasundar|Rajesh, Sudha Chandran, S. Ve. Shekher, Ramya Krishnan, Manorama, Radha Ravi|null|Preethi Indar Combines|/wiki/Sarvam_Sakthimayam
92!Selvakku|M. A. Kaja|Chandrasekhar, Radha Ravi, Sulakshana, Soorya, Sri S

In [48]:
df=df.drop(0)
df

year                       title             director  \
1    1986         Aayiram Kannudayaal           K. Shankar   
2    1986  Aayiram Pookkal Malarattum          E. Ramadoss   
3    1986                Adutha Veedu       Rama Narayanan   
4    1986              Africavil Appu          Vijay Anand   
5    1986       Amman Kovil Kizhakale       R. Sundarrajan   
6    1986             Anandha Kanneer           K. Vijayan   
7    1986            Annai En Dheivam    R. Krishnamoorthy   
8    1986               Aruvadai Naal          G. M. Kumar   
9    1986   Avalai Solli Kuttramillai          A. A. Soman   
10   1986    Bathil Solval Bhadrakali        S. Jagadeesan   
11   1986                       Choru       Rama Narayanan   
12   1986               Cinema Cinema      T. P. Gajendran   
13   1986             December Pookal          R. Boopathy   
14   1986             Dharma Devathai     S. P. Muthuraman   
15   1986              Dharma Pathini             Ameerjan   
16   1986                     Dharmam       R. Thyagarajan   
17   1986                 En Sabatham  S. A. Chandrasekhar   
18   1986      Enakku Nane Needipathi  S. A. Chandrasekhar   
19   1986         Endravathu Oru Naal           Balu Anand   
20   1986     Engal Thaikulame Varuge     V. N. Sambantham   
21   1986               Iravu Pookkal        Sridhar Rajan   
22   1986        Isai Paadum Thendral         S. Devarajan   
23   1986                  Jeevanathi       Sanjay-Bhaskar   
24   1986             Jigu Jigu Rayil           Manivannan   
25   1986                 Jothi Malar       Rama Narayanan   
26   1986            Kadaikan Paarvai         Raaj Sridhar   
27   1986        Kadalora Kavithaigal      P. Bharathiraja   
28   1986                Kagidha Odam       Rama Narayanan   
29   1986           Kaithiyin Theerpu      Vijaya Bharathy   
30   1986       Kalamellam Un Madiyil           Rajasekhar   
..    ...                         ...                  ...   
79   1986     Paru Paru Pattanam Paru             Manobala   
80   1986      Piranthaen Valarnthaen         Vijayasingam   
81   1986                Poi Mugangal      C. V. Rajendran   
82   1986     Pookalai Parikatheergal        V. Azhagappan   
83   1986            Pudhiya Poovithu                Surya   
84   1986             Punnagai Mannan       K. Balachander   
85   1986                      Puthir     S. Siddalingaiah   
86   1986            Raja Nee Vaazhga      C. V. Rajendran   
87   1986         Rasigan Oru Rasigai           Balu Anand   
88   1986                     Revathy    Sundar K. Vijayan   
89   1986                   Saadhanai       A. S. Pragasam   
90   1986      Samsaram Adhu Minsaram                 Visu   
91   1986          Sarvam Sakthimayam     P. R. Somasundar   
92   1986                    Selvakku           M. A. Kaja   
93   1986            Sigappu Malargal  S. A. Chandrasekhar   
94   1986            Solai Pushpangal            Ramarajan   
95   1986      Thaaiku Oru Thaalaattu    Balachandra Menon   
96   1986       Thalaiyatti Bommaigal            K. Natraj   
97   1986          Thazhuvatha Kaigal       R. Sundarrajan   
98   1986              Thodarum Uravu                        
99   1986      Unakkaagave Vaazhgiren          K. Rangaraj   
100  1986          Unnai Ondru Ketpen        Sakthi Kannan   
101  1986             Unnidathil Naan       Arun Veerappan   
102  1986              Uyire Unakkaga          K. Rangaraj   
103  1986             Vasantha Raagam  S. A. Chandrasekhar   
104  1986                      Vettai         Senthinathan   
105  1986            Vidinja Kalyanam           Manivannan   
106  1986                  Viduthalai           K. Vijayan   
107  1986                      Vikram           Rajasekhar   
108  1986    Yaaro Ezhuthiya Kavithai        C. V. Sridhar   

                                                  cast genre  \
1    Padmini, Jeevitha, Prabha, Baby Shalini, Rajee...  null   
2    Mohan, Seetha, Ranjini, Gou

In [49]:
getAllMusic()

In [50]:
getAllrating()

https://www.imdb.com/title/tt0271346/
Aayiram Kannudayaal | 7.1
https://www.imdb.com/title/tt10251580/
Aayiram Pookkal Malarattum | 
https://www.imdb.com/title/tt8908026/
Adutha Veedu | 
Africavil Appu | 
https://www.imdb.com/title/tt0318983/
Amman Kovil Kizhakale | 6.8
https://www.imdb.com/title/tt0321215/
Anandha Kanneer | 7.9
https://www.imdb.com/title/tt3343598/
Annai En Dheivam | 
https://www.imdb.com/title/tt0319008/
Aruvadai Naal | 6.0
https://www.imdb.com/title/tt0320316/
Avalai Solli Kuttramillai | 
https://www.imdb.com/title/tt3319440/
Bathil Solval Bhadrakali | 
https://www.imdb.com/title/tt0091670/
Choru | 8.1
https://www.imdb.com/title/tt3319448/
Cinema Cinema | 
https://www.imdb.com/title/tt0319272/
December Pookal | 5.7
https://www.imdb.com/title/tt10128062/
Dharma Devathai | 
https://www.imdb.com/title/tt0319288/
Dharma Pathini | 
https://www.imdb.com/title/tt8719346/
Dharmam | 
https://www.imdb.com/title/tt1385882/
En Sabatham | 
https://www.imdb.com/title/tt1385884/
E

In [51]:
df

year                       title             director  \
1    1986         Aayiram Kannudayaal           K. Shankar   
2    1986  Aayiram Pookkal Malarattum          E. Ramadoss   
3    1986                Adutha Veedu       Rama Narayanan   
4    1986              Africavil Appu          Vijay Anand   
5    1986       Amman Kovil Kizhakale       R. Sundarrajan   
6    1986             Anandha Kanneer           K. Vijayan   
7    1986            Annai En Dheivam    R. Krishnamoorthy   
8    1986               Aruvadai Naal          G. M. Kumar   
9    1986   Avalai Solli Kuttramillai          A. A. Soman   
10   1986    Bathil Solval Bhadrakali        S. Jagadeesan   
11   1986                       Choru       Rama Narayanan   
12   1986               Cinema Cinema      T. P. Gajendran   
13   1986             December Pookal          R. Boopathy   
14   1986             Dharma Devathai     S. P. Muthuraman   
15   1986              Dharma Pathini             Ameerjan   
16   1986                     Dharmam       R. Thyagarajan   
17   1986                 En Sabatham  S. A. Chandrasekhar   
18   1986      Enakku Nane Needipathi  S. A. Chandrasekhar   
19   1986         Endravathu Oru Naal           Balu Anand   
20   1986     Engal Thaikulame Varuge     V. N. Sambantham   
21   1986               Iravu Pookkal        Sridhar Rajan   
22   1986        Isai Paadum Thendral         S. Devarajan   
23   1986                  Jeevanathi       Sanjay-Bhaskar   
24   1986             Jigu Jigu Rayil           Manivannan   
25   1986                 Jothi Malar       Rama Narayanan   
26   1986            Kadaikan Paarvai         Raaj Sridhar   
27   1986        Kadalora Kavithaigal      P. Bharathiraja   
28   1986                Kagidha Odam       Rama Narayanan   
29   1986           Kaithiyin Theerpu      Vijaya Bharathy   
30   1986       Kalamellam Un Madiyil           Rajasekhar   
..    ...                         ...                  ...   
79   1986     Paru Paru Pattanam Paru             Manobala   
80   1986      Piranthaen Valarnthaen         Vijayasingam   
81   1986                Poi Mugangal      C. V. Rajendran   
82   1986     Pookalai Parikatheergal        V. Azhagappan   
83   1986            Pudhiya Poovithu                Surya   
84   1986             Punnagai Mannan       K. Balachander   
85   1986                      Puthir     S. Siddalingaiah   
86   1986            Raja Nee Vaazhga      C. V. Rajendran   
87   1986         Rasigan Oru Rasigai           Balu Anand   
88   1986                     Revathy    Sundar K. Vijayan   
89   1986                   Saadhanai       A. S. Pragasam   
90   1986      Samsaram Adhu Minsaram                 Visu   
91   1986          Sarvam Sakthimayam     P. R. Somasundar   
92   1986                    Selvakku           M. A. Kaja   
93   1986            Sigappu Malargal  S. A. Chandrasekhar   
94   1986            Solai Pushpangal            Ramarajan   
95   1986      Thaaiku Oru Thaalaattu    Balachandra Menon   
96   1986       Thalaiyatti Bommaigal            K. Natraj   
97   1986          Thazhuvatha Kaigal       R. Sundarrajan   
98   1986              Thodarum Uravu                        
99   1986      Unakkaagave Vaazhgiren          K. Rangaraj   
100  1986          Unnai Ondru Ketpen        Sakthi Kannan   
101  1986             Unnidathil Naan       Arun Veerappan   
102  1986              Uyire Unakkaga          K. Rangaraj   
103  1986             Vasantha Raagam  S. A. Chandrasekhar   
104  1986                      Vettai         Senthinathan   
105  1986            Vidinja Kalyanam           Manivannan   
106  1986                  Viduthalai           K. Vijayan   
107  1986                      Vikram           Rajasekhar   
108  1986    Yaaro Ezhuthiya Kavithai        C. V. Sridhar   

                                                  cast genre  \
1    Padmini, Jeevitha, Prabha, Baby Shalini, Rajee...  null   
2    Mohan, Seetha, Ranjini, Gou

In [52]:
df.to_csv('1986_movies.csv')

# 1985

In [16]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1985'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1985=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [17]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [18]:
for table in tables_1985:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1985'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Music', 'Producer', 'Cast']
1!Aagaya Thamaraigal|V. Azhagappan|Suresh, Revathi, Sathyaraj, Goundamani, Manorama, Senthil, S. S. Chandran|null|Yuvaraj Productions|/wiki/Aagaya_Thamaraigal
2!Aan Paavam|Pandiarajan|Pandiyan, Pandiarajan, Revathi, Seetha, V. K. Ramasamy, Janagaraj|null|Alamu Movies|/wiki/Aan_Paavam
3!Aandavan Sothu|M. Karnan|Jaishankar, Madhavi|null|Kalaithottam|/w/index.php?title=Aandavan_Sothu&action=edit&redlink=1
4!Aasha|V. T. Thyagarajan|Suresh, Nalini|null|V. T. T. Creation|/w/index.php?title=Aasha_(1985_film)&action=edit&redlink=1
5!Aduthathu Albert|G. N. Rangarajan|Prabhu, Urvashi, Poornima Bhagyaraj, Y. G. Mahendra|null|Kumaravel Films|/wiki/Aduthathu_Albert
6!Alai Osai|Sirumugai Ravi|Vijayakanth, Nalini, Radha Ravi, Goundamani, Senthil|null|Thirumal Cine Films|/wiki/Alai_Osai
7!Ammavum Neeye Appavum Neeye|S. P. Muthuraman|Rajesh, Saritha, Y. G. Mahendra|null|Vasan Brothers|/w/index.php?title=Ammavum_Neeye_Appavum_Neeye&action=edit&redlink=1

91!Puthiya Sagaptham|Visu|Vijayakanth, Ambika, Visu, Kismu|null|Sri Bhagavathi Creations|/wiki/Puthiya_Sagaptham
92!Puthiya Theerpu|C. V. Rajendran|Vijayakanth, Ambika, Cho, Janagaraj|null|Ram Arts|/w/index.php?title=Puthiya_Theerpu&action=edit&redlink=1
93!Rahasiyam|Rama Narayanan|Chandrasekhar, Nalini, Radha Ravi|null|Meena Production|/w/index.php?title=Rahasiyam&action=edit&redlink=1
94!Raja Gopuram|P. G. Pandian|Rajesh, Sulakshana, Pandiyan, Nalini, Goundamani|null|Sri Jayanthi Cine Arts|/w/index.php?title=Raja_Gopuram&action=edit&redlink=1
95!Raja Rishi|K. Shankar|Sivaji Ganesan, Prabhu, K. R. Vijaya, Lakshmi, Raadhika, Nalini, Vijayakumar, Y. G. Mahendra, Vanitha|null|Bhairavi Films|/wiki/Raja_Rishi
96!Raja Yuvaraja|Somasundaram|Thiagarajan, Urvashi, Deepa, Goundamani, Senthil|null|Be Yess Art Films|/w/index.php?title=Raja_Yuvaraja&action=edit&redlink=1
97!Rajathi Rojakili|Rama Narayanan|Rajesh, Suresh, Nalini, Sulakshana, Goundamani, Manorama, Anuradha, Senthil|null|Niruma Creat

In [19]:
df=df.drop(0)
df

year                        title            director  \
1    1985           Aagaya Thamaraigal       V. Azhagappan   
2    1985                   Aan Paavam         Pandiarajan   
3    1985               Aandavan Sothu           M. Karnan   
4    1985                        Aasha   V. T. Thyagarajan   
5    1985             Aduthathu Albert    G. N. Rangarajan   
6    1985                    Alai Osai      Sirumugai Ravi   
7    1985  Ammavum Neeye Appavum Neeye    S. P. Muthuraman   
8    1985                Amutha Gaanam         K. Rangaraj   
9    1985               Anbin Mugavari          Manivannan   
10   1985            Andha Oru Nimidam   Major Sundarrajan   
11   1985                 Annai Bhoomi      R. Thyagarajan   
12   1985                         Anni  Karthick Raghunath   
13   1985                    Anthasthu      R. Thyagarajan   
14   1985          Arthamulla Aasaigal       Babu Maharaja   
15   1985           Aval Sumangalithan                Visu   
16   1985                         Avan        Chozha Rajan   
17   1985                      Bandham          K. Vijayan   
18   1985                Chain Jayapal      Rama Narayanan   
19   1985         Chidambara Rahasiyam                Visu   
20   1985                 Chinna Veedu        K. Bhagyaraj   
21   1985        Chithirame Chithirame       Vittal Gnanam   
22   1985                 Deiva Piravi   R. Krishnamoorthy   
23   1985                        Eetti          Rajasekhar   
24   1985                  Elan Kandru            Ameerjan   
25   1985                   En Selvame    S. P. Muthuraman   
26   1985                  Engal Kural      Rama Narayanan   
27   1985         Engirunthalum Vazhga         R. Selvaraj   
28   1985                     Erimalai         Gopikrishna   
29   1985                  Geethanjali         K. Rangaraj   
30   1985         Hello Yaar Pesurathu           Ramarajan   
..    ...                          ...                 ...   
102  1985          Santhosha Kanavukal        R. C. Sakthi   
103  1985                        Selvi           K. Natraj   
104  1985              Sindhu Bhairavi      K. Balachander   
105  1985                 Sivappu Nila      Rama Narayanan   
106  1985         Sonna Nambamatteenga      S. Jayachandar   
107  1985             Sri Raghavendrar    S. P. Muthuraman   
108  1985           Sugamana Raagangal      R. Sundarrajan   
109  1985                    Thandanai      Rama Narayanan   
110  1985               Thanga Mama 3D            K. Simon   
111  1985                       Thavam        R. C. Sakthi   
112  1985      Thendral Thodatha Malar         G. P. Balan   
113  1985        Thendrale Ennai Thodu       C. V. Sridhar   
114  1985                     Thiramai    Vijai Krishnaraj   
115  1985                Udaya Geetham         K. Rangaraj   
116  1985     Un Kannil Neer Vazhindal       Balu Mahendra   
117  1985            Unakkaga Oru Roja     C. V. Rajendran   
118  1985          Unnai Thedi Varuven       C. V. Sridhar   
119  1985             Unnai Vidamatten             Nethaji   
120  1985                       Urimai      Rama Narayanan   
121  1985                       Uthami      M. Vellaichami   
122  1985               Uyarndha Ullam    S. P. Muthuraman   
123  1985                         Veli               Durai   
124  1985                Vellai Manasu                Gopu   
125  1985                       Vesham      Rama Narayanan   
126  1985                    Vetrikani  M. R. Vijaychander   
127  1985                 Vilangu Meen            Jayadevi   
128  1985      Viswanathan Velai Venum            Raghuram   
129  1985                        Yaar?       Sakthi Kannan   
130  1985           Yaro Azhaikirargal         Rajan Sarma   
131  1985      Yemaatrathe Yemaaraathe    V. C. Guhanathan   

                                                  cast genre  \
1    Suresh, Revathi, Sathyaraj, Goundamani, Manora...  null   
2    Pandiyan, Pandiarajan, Reva

In [20]:
getAllMusic()

ex.php?title=Samudhaya_Chanthaiyile&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Samudhaya_Chanthaiyile&acti...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Sonna_Nambamatteenga&action...


Exception
101 | 
ex.php?title=Sonna_Nambamatteenga&action=edit&redlink=1
Exception
106 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [21]:
getAllrating()

https://www.imdb.com/title/tt8715206/
Aagaya Thamaraigal | 
https://www.imdb.com/title/tt0275175/
Aan Paavam | 7.7
https://www.imdb.com/title/tt8719276/
Aandavan Sothu | 4.8
https://www.imdb.com/title/tt0050090/
Aasha | 6.4
https://www.imdb.com/title/tt0318967/
Aduthathu Albert | 5.6
https://www.imdb.com/title/tt0318973/
Alai Osai | 6.8
https://www.imdb.com/title/tt2710254/
Ammavum Neeye Appavum Neeye | 
https://www.imdb.com/title/tt0318986/
Amutha Gaanam | 
https://www.imdb.com/title/tt0318990/
Anbin Mugavari | 5.2
https://www.imdb.com/title/tt0318993/
Andha Oru Nimidam | 7.0
https://www.imdb.com/title/tt0317083/
Annai Bhoomi | 8.2
https://www.imdb.com/title/tt8715118/
Anni | 
https://www.imdb.com/title/tt8918930/
Anthasthu | 
https://www.imdb.com/title/tt0372773/
Arthamulla Aasaigal | 
https://www.imdb.com/title/tt2710344/
Aval Sumangalithan | 5.0
https://www.imdb.com/title/tt1781771/
Avan | 4.0
https://www.imdb.com/title/tt1435465/
Bandham | 6.2
https://www.imdb.com/title/tt8929818/

In [22]:
df

year                        title            director  \
1    1985           Aagaya Thamaraigal       V. Azhagappan   
2    1985                   Aan Paavam         Pandiarajan   
3    1985               Aandavan Sothu           M. Karnan   
4    1985                        Aasha   V. T. Thyagarajan   
5    1985             Aduthathu Albert    G. N. Rangarajan   
6    1985                    Alai Osai      Sirumugai Ravi   
7    1985  Ammavum Neeye Appavum Neeye    S. P. Muthuraman   
8    1985                Amutha Gaanam         K. Rangaraj   
9    1985               Anbin Mugavari          Manivannan   
10   1985            Andha Oru Nimidam   Major Sundarrajan   
11   1985                 Annai Bhoomi      R. Thyagarajan   
12   1985                         Anni  Karthick Raghunath   
13   1985                    Anthasthu      R. Thyagarajan   
14   1985          Arthamulla Aasaigal       Babu Maharaja   
15   1985           Aval Sumangalithan                Visu   
16   1985                         Avan        Chozha Rajan   
17   1985                      Bandham          K. Vijayan   
18   1985                Chain Jayapal      Rama Narayanan   
19   1985         Chidambara Rahasiyam                Visu   
20   1985                 Chinna Veedu        K. Bhagyaraj   
21   1985        Chithirame Chithirame       Vittal Gnanam   
22   1985                 Deiva Piravi   R. Krishnamoorthy   
23   1985                        Eetti          Rajasekhar   
24   1985                  Elan Kandru            Ameerjan   
25   1985                   En Selvame    S. P. Muthuraman   
26   1985                  Engal Kural      Rama Narayanan   
27   1985         Engirunthalum Vazhga         R. Selvaraj   
28   1985                     Erimalai         Gopikrishna   
29   1985                  Geethanjali         K. Rangaraj   
30   1985         Hello Yaar Pesurathu           Ramarajan   
..    ...                          ...                 ...   
102  1985          Santhosha Kanavukal        R. C. Sakthi   
103  1985                        Selvi           K. Natraj   
104  1985              Sindhu Bhairavi      K. Balachander   
105  1985                 Sivappu Nila      Rama Narayanan   
106  1985         Sonna Nambamatteenga      S. Jayachandar   
107  1985             Sri Raghavendrar    S. P. Muthuraman   
108  1985           Sugamana Raagangal      R. Sundarrajan   
109  1985                    Thandanai      Rama Narayanan   
110  1985               Thanga Mama 3D            K. Simon   
111  1985                       Thavam        R. C. Sakthi   
112  1985      Thendral Thodatha Malar         G. P. Balan   
113  1985        Thendrale Ennai Thodu       C. V. Sridhar   
114  1985                     Thiramai    Vijai Krishnaraj   
115  1985                Udaya Geetham         K. Rangaraj   
116  1985     Un Kannil Neer Vazhindal       Balu Mahendra   
117  1985            Unakkaga Oru Roja     C. V. Rajendran   
118  1985          Unnai Thedi Varuven       C. V. Sridhar   
119  1985             Unnai Vidamatten             Nethaji   
120  1985                       Urimai      Rama Narayanan   
121  1985                       Uthami      M. Vellaichami   
122  1985               Uyarndha Ullam    S. P. Muthuraman   
123  1985                         Veli               Durai   
124  1985                Vellai Manasu                Gopu   
125  1985                       Vesham      Rama Narayanan   
126  1985                    Vetrikani  M. R. Vijaychander   
127  1985                 Vilangu Meen            Jayadevi   
128  1985      Viswanathan Velai Venum            Raghuram   
129  1985                        Yaar?       Sakthi Kannan   
130  1985           Yaro Azhaikirargal         Rajan Sarma   
131  1985      Yemaatrathe Yemaaraathe    V. C. Guhanathan   

                                                  cast genre  \
1    Suresh, Revathi, Sathyaraj, Goundamani, Manora...  null   
2    Pandiyan, Pandiarajan, Reva

In [23]:
df.to_csv('1985_movies.csv')

# 1984

In [27]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1984'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1984=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [28]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [29]:
for table in tables_1984:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1984'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!24 Mani Neram|Manivannan|Mohan, Jaishankar, Nalini, Sathyaraj, Swapna, Ilavarasi|null|Thirupathiswamy Pictures|/wiki/24_Mani_Neram
2!Aathora Aatha|Mangadu Ramachandran|Chakravarthy, Raghuvaran, Anuradha, Y. Vijaya|null|Sri Jothi Arts|/w/index.php?title=Aathora_Aatha&action=edit&redlink=1
3!Aayiram Kaigal|A. V. Babu|Ramakrishnan, Samundeeswari|null|Kailash Combines|/w/index.php?title=Aayiram_Kaigal&action=edit&redlink=1
4!Achamillai Achamillai|K. Balachander|Rajesh, Saritha, Delhi Ganesh, Pavithra, Ahalaya|null|Kavithalayaa Productions|/wiki/Achamillai_Achamillai
5!Alaya Deepam|C. V. Sridhar|Sujatha, Jaishankar, Suresh, Rajesh, Ilavarasi, Vanitha, Y. G. Mahendra, Thengai Srinivasan, Manorama|null|K. R. G. Film Circuit|/wiki/Alaya_Deepam
6!Ambigai Neril Vanthaal|Manivannan|Mohan, Radha, Y. G. Mahendra, Janagaraj|null|Ram Arts|/wiki/Ambigai_Neril_Vanthaal
7!Amma Irukka|Major Sundarrajan|Sivakumar, Jaishankar, Sulakshana, V. K. Ramasamy,

72!Pillaiyar|V. T. Arasu|Arunkumar, Sathyakala, Y. G. Mahendra|null|Shasti Films|/w/index.php?title=Pillaiyar_(film)&action=edit&redlink=1
73!Ponnu Pudichirukku|K. Rangaraj|Pandiyan, Revathi, Goundamani|null|Sri Amman Creations|
74!Poovilangu|Ameerjan|Murali, Kuyili, Poovilangu Mohan, Y. Vijaya|null|Kavithalayaa Productions|/wiki/Poovilangu
75!Pozhuthu Vidinchachu|Gangai Amaran|Prabhu, Sulakshana|null|Sooraj Enterprises|/w/index.php?title=Pozhuthu_Vidinchachu&action=edit&redlink=1
76!Priyamudan Prabhu|P. Gangaikondan|Prabhu, Rajeev, Brindha[disambiguation needed], Nagesh, Goundamani|null|Mohan Productions|/w/index.php?title=Priyamudan_Prabhu&action=edit&redlink=1
77!Pudhiavan|Ameerjan|Murali, Anitha, Raveendran|null|Kavithalayaa Productions|/wiki/Pudhiavan
78!Pudhumai Penn|P. Bharathiraja|Pandiyan, Revathi, Pratap K. Pothen, Rajasekhar|null|AVM Productions|/wiki/Pudhumai_Penn_(1984_film)
79!Puthiya Sangamam|Charuhasan|Prabhu, Suhasini, Charuhasan|null|Radhakrishna Combines|/w/index.php

In [30]:
df=df.drop(0)
df

year                   title                      director  \
1    1984           24 Mani Neram                    Manivannan   
2    1984           Aathora Aatha          Mangadu Ramachandran   
3    1984          Aayiram Kaigal                    A. V. Babu   
4    1984   Achamillai Achamillai                K. Balachander   
5    1984            Alaya Deepam                 C. V. Sridhar   
6    1984  Ambigai Neril Vanthaal                    Manivannan   
7    1984             Amma Irukka             Major Sundarrajan   
8    1984            Anbe Odi Vaa              R. Ranjith Kumar   
9    1984          Anbulla Malare             B. R. Ravishankar   
10   1984     Anbulla Rajinikanth                     K. Natraj   
11   1984   Antha June 16 Am Naal                  A. Shanmugam   
12   1984   Antha Uravukku Satchi              R. Pattabhiraman   
13   1984                  Azhagu                    K. Vijayan   
14   1984             Chiranjeevi                    K. Shankar   
15   1984           Devi Sri Devi                 Gangai Amaran   
16   1984            Dharmakartha                      K. V. R.   
17   1984       Dhavani Kanavugal                  K. Bhagyaraj   
18   1984          Enakkul Oruvan              S. P. Muthuraman   
19   1984              Etho Mogam                  T. K. Manian   
20   1984    Ezhuthatha Sattangal                    K. Shankar   
21   1984         Idhu Enga Boomi                     M. Karnan   
22   1984      Ingeyum Oru Gangai                    Manivannan   
23   1984           Iru Medhaigal                 V. Srinivasan   
24   1984           Irumbu Kaigal                    G. S. Mani   
25   1984               January 1                    Manivannan   
26   1984                 Kadamai                Rama Narayanan   
27   1984        Kai Kodukkum Kai                  J. Mahendran   
28   1984           Kairasikkaran               S. S. K. Sankar   
29   1984       Kalyana Kanavugal                  K. Sivarajan   
30   1984             Kathula Poo                         G. K.   
..    ...                     ...                           ...   
91   1984          Simma Soppanam  S. S. Karuppasamy-R. Krishna   
92   1984                   Sirai                  R. C. Sakthi   
93   1984             Sukra Desai                   T. K. Mohan   
94   1984         Sumangali Kolam                Vijaya Nirmala   
95   1984   Thalaiyanai Manthiram                  N. Venkatesh   
96   1984    Thambikku Entha Ooru                    Rajasekhar   
97   1984           Thanga Koppai                  R. C. Sakthi   
98   1984      Thangamadi Thangam                Rama Narayanan   
99   1984                Tharaasu                Raja Ganapathi   
100  1984       Theerpu En Kaiyil                  V. P. Sunder   
101  1984              Then Koodu              V. Satyanarayana   
102  1984          Then Sittukkal                 N. Damodharan   
103  1984               Thiruppam             R. Krishnamoorthy   
104  1984       Thiruttu Rajakkal                Rama Narayanan   
105  1984         Ullam Urugudadi             Erode N. Murugesh   
106  1984       Unga Veetu Pillai               C. V. Rajendran   
107  1984  Unnai Naan Santhithaen                   K. Rangaraj   
108  1984      Uravai Kaatha Kili                   T. Rajendar   
109  1984     Urimai Thedum Uravu                    Sakthi Das   
110  1984                Vaazhkai               C. V. Rajendran   
111  1984              Vai Pandal                Rama Narayanan   
112  1984    Vai Sollil Veeranadi                          Visu   
113  1984    Vaidehi Kathirunthal                R. Sundarrajan   
114  1984           Vamsa Vilakku             R. Krishnamoorthy   
115  1984    Vanga Mappille Vanga                   Sivashankar   
116  1984     Veetuku Oru Kannagi           S. A. Chandrasekhar   
117  1984       Vellai Pura Ondru                 Gangai Amaran   
118  1984      Vengaiyin Mainthan                Ram

In [31]:
getAllMusic()

ex.php?title=Matran_Thottathu_Malligai&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Matran_Thottathu_Malligai&a...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Then_Sittukkal&action=edit&...


Exception
43 | 
ex.php?title=Then_Sittukkal&action=edit&redlink=1
Exception
102 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [32]:
getAllrating()

https://www.imdb.com/title/tt0319576/
24 Mani Neram | 5.8
https://www.imdb.com/title/tt0255448/
Aathora Aatha | 6.3
https://www.imdb.com/title/tt3323828/
Aayiram Kaigal | 
https://www.imdb.com/title/tt0086848/
Achamillai Achamillai | 6.9
https://www.imdb.com/title/tt6626606/
Alaya Deepam | 
https://www.imdb.com/title/tt0318979/
Ambigai Neril Vanthaal | 5.6
https://www.imdb.com/name/nm1107894/
Amma Irukka | 
https://www.imdb.com/title/tt0318989/
Anbe Odi Vaa | 
https://www.imdb.com/title/tt0318991/
Anbulla Malare | 
https://www.imdb.com/title/tt0318992/
Anbulla Rajinikanth | 6.2
https://www.imdb.com/title/tt0309925/
Antha June 16 Am Naal | 7.1
https://www.imdb.com/title/tt8914134/
Antha Uravukku Satchi | 
https://www.imdb.com/name/nm7598802/
Azhagu | 
https://www.imdb.com/title/tt1437592/
Chiranjeevi | 
https://www.imdb.com/title/tt0186958/
Devi Sri Devi | 5.4
https://www.imdb.com/title/tt3323836/
Dharmakartha | 
https://www.imdb.com/title/tt0155664/
Dhavani Kanavugal | 6.9
https://www.

In [33]:
df

year                   title                      director  \
1    1984           24 Mani Neram                    Manivannan   
2    1984           Aathora Aatha          Mangadu Ramachandran   
3    1984          Aayiram Kaigal                    A. V. Babu   
4    1984   Achamillai Achamillai                K. Balachander   
5    1984            Alaya Deepam                 C. V. Sridhar   
6    1984  Ambigai Neril Vanthaal                    Manivannan   
7    1984             Amma Irukka             Major Sundarrajan   
8    1984            Anbe Odi Vaa              R. Ranjith Kumar   
9    1984          Anbulla Malare             B. R. Ravishankar   
10   1984     Anbulla Rajinikanth                     K. Natraj   
11   1984   Antha June 16 Am Naal                  A. Shanmugam   
12   1984   Antha Uravukku Satchi              R. Pattabhiraman   
13   1984                  Azhagu                    K. Vijayan   
14   1984             Chiranjeevi                    K. Shankar   
15   1984           Devi Sri Devi                 Gangai Amaran   
16   1984            Dharmakartha                      K. V. R.   
17   1984       Dhavani Kanavugal                  K. Bhagyaraj   
18   1984          Enakkul Oruvan              S. P. Muthuraman   
19   1984              Etho Mogam                  T. K. Manian   
20   1984    Ezhuthatha Sattangal                    K. Shankar   
21   1984         Idhu Enga Boomi                     M. Karnan   
22   1984      Ingeyum Oru Gangai                    Manivannan   
23   1984           Iru Medhaigal                 V. Srinivasan   
24   1984           Irumbu Kaigal                    G. S. Mani   
25   1984               January 1                    Manivannan   
26   1984                 Kadamai                Rama Narayanan   
27   1984        Kai Kodukkum Kai                  J. Mahendran   
28   1984           Kairasikkaran               S. S. K. Sankar   
29   1984       Kalyana Kanavugal                  K. Sivarajan   
30   1984             Kathula Poo                         G. K.   
..    ...                     ...                           ...   
91   1984          Simma Soppanam  S. S. Karuppasamy-R. Krishna   
92   1984                   Sirai                  R. C. Sakthi   
93   1984             Sukra Desai                   T. K. Mohan   
94   1984         Sumangali Kolam                Vijaya Nirmala   
95   1984   Thalaiyanai Manthiram                  N. Venkatesh   
96   1984    Thambikku Entha Ooru                    Rajasekhar   
97   1984           Thanga Koppai                  R. C. Sakthi   
98   1984      Thangamadi Thangam                Rama Narayanan   
99   1984                Tharaasu                Raja Ganapathi   
100  1984       Theerpu En Kaiyil                  V. P. Sunder   
101  1984              Then Koodu              V. Satyanarayana   
102  1984          Then Sittukkal                 N. Damodharan   
103  1984               Thiruppam             R. Krishnamoorthy   
104  1984       Thiruttu Rajakkal                Rama Narayanan   
105  1984         Ullam Urugudadi             Erode N. Murugesh   
106  1984       Unga Veetu Pillai               C. V. Rajendran   
107  1984  Unnai Naan Santhithaen                   K. Rangaraj   
108  1984      Uravai Kaatha Kili                   T. Rajendar   
109  1984     Urimai Thedum Uravu                    Sakthi Das   
110  1984                Vaazhkai               C. V. Rajendran   
111  1984              Vai Pandal                Rama Narayanan   
112  1984    Vai Sollil Veeranadi                          Visu   
113  1984    Vaidehi Kathirunthal                R. Sundarrajan   
114  1984           Vamsa Vilakku             R. Krishnamoorthy   
115  1984    Vanga Mappille Vanga                   Sivashankar   
116  1984     Veetuku Oru Kannagi           S. A. Chandrasekhar   
117  1984       Vellai Pura Ondru                 Gangai Amaran   
118  1984      Vengaiyin Mainthan                Ram

In [34]:
df.to_csv('1984_movies.csv')

# 1983

In [38]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1983'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1983=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [39]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [40]:
for table in tables_1983:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1983'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Aanandha Kummi|Gokula Krishnan|Bhanuchander, Ashwini, Goundamani|null|Ilaiyaraaja Films|/wiki/Aanandha_Kummi
2!Aayiram Nilave Vaa|A. S. Pragasam|Karthik, Sulakshana, Silk Smitha, Nagesh|null|New Wave Pictures|/wiki/Aayiram_Nilave_Vaa
3!Adutha Varisu|S. P. Muthuraman|Rajinikanth, Sridevi, Silk Smitha, S. Varalakshmi, Cho, Manorama|null|Dwarkish Chitra|/wiki/Adutha_Varisu
4!Alai Payum Nenjangal|H. Ramesh|Suresh, Sujatha, Rajesh, Ilavarasi|null|D & PG Productions|/wiki/Alai_Payum_Nenjangal
5!Anal Kaatru|Komal Swaminathan|Rajesh, Delhi Ganesh, Vanitha|null|Cine Friends|/wiki/Anal_Kaatru
6!Anney Anney|Moulee|Moulee, Sumithra, Viji|null|Kalaivani Productions|/wiki/Anney_Anney
7!Antha Sila Naatkal|Venkat|Mohan, Poornima Jayaram, Manorama, Nithya, Silk Smitha, Dhilip|null|Meenakshi Finance|/wiki/Antha_Sila_Naatkal
8!Apoorva Sahodarigal|R. Thyagarajan|Karthik, Suhasini, Radha, Suresh, Urvashi, Jaishankar, K. R. Vijaya|null|Devar Films|/wiki/A

69!Seerum Singangal|Rama Narayanan|Suresh, Nalini, S. Ve. Shekher|null|Sri Thenandal Films|/w/index.php?title=Seerum_Singangal&action=edit&redlink=1
70!Shasti Viratham|R. Thyagarajan|Sivakumar, Poornima Jayaram|null|Devar Films|/w/index.php?title=Shasti_Viratham&action=edit&redlink=1
71!Sivappu Sooriyan|V. Srinivasan|Rajinikanth, Saritha, Radha, Y. G. Mahendra, Silk Smitha|null|Muktha Films|/wiki/Sivappu_Sooriyan
72!Silk Silk Silk|Y. V. Gopikrishnan|Silk Smitha, Bhanu Chander, Raghuvaran|null|Sri Ganesh Kalamandir|/wiki/Silk_Silk_Silk
73!Soorakottai Singakutti|Rama Narayanan|Prabhu, Silk Smitha, Gemini Ganesan, Prameela|null|AVM Productions|/wiki/Soorakottai_Singakutti
74!Soorapuli|Venkat|Prabhu, Ambika, Y. G. Mahendra|null|Jai Geetha Movies|/w/index.php?title=Soorapuli&action=edit&redlink=1
75!Subha Muhurtham|Raghu|S. Ve. Shekher, Sulakshana|null|Indralayam Productions|/w/index.php?title=Subha_Muhurtham&action=edit&redlink=1
76!Sumangali|D. Yoganand|Sivaji Ganesan, Sujatha, Prabhu, Y.

In [41]:
df=df.drop(0)
df

year                            title              director  \
1   1983                   Aanandha Kummi       Gokula Krishnan   
2   1983               Aayiram Nilave Vaa        A. S. Pragasam   
3   1983                    Adutha Varisu      S. P. Muthuraman   
4   1983             Alai Payum Nenjangal             H. Ramesh   
5   1983                      Anal Kaatru     Komal Swaminathan   
6   1983                      Anney Anney                Moulee   
7   1983               Antha Sila Naatkal                Venkat   
8   1983              Apoorva Sahodarigal        R. Thyagarajan   
9   1983                 Aval Oru Kaviyam                 Durai   
10  1983     Bhagavathipuram Railway Gate           R. Selvaraj   
11  1983                    Bramacharigal         Swarnamoorthy   
12  1983            Dhooram Adhighamillai    Robert-Rajasekaran   
13  1983                   Dowry Kalyanam                  Visu   
14  1983              En Aasai Unnoduthan          K. Narayanan   
15  1983                       En Priyame             Sivakumar   
16  1983       Ennai Paar En Azhagai Paar          N. S. Maniam   
17  1983                 Gramathu Kiligal       Sanjeevi Ganesh   
18  1983                  Idhu Enga Naadu        Rama Narayanan   
19  1983                 Ilamai Kaalangal            Manivannan   
20  1983                   Ilayapiravigal        A. Irudaya Raj   
21  1983                          Imaigal      R. Krishnamurthy   
22  1983             Indru Nee Nalai Naan     Major Sundarrajan   
23  1983                 Inimai Idho Idho         R. Ramalingam   
24  1983                        Jodi Pura        V. K. Ramasamy   
25  1983                            Jothi            Manivannan   
26  1983                      Kai Varisai            Prem Kumar   
27  1983                Kalvadiyum Pookal        V. Karthigeyan   
28  1983                   Kaman Pandigai            H. S. Venu   
29  1983    Kann Sivanthaal Mann Sivakkum        Sreedhar Rajan   
30  1983                 Kasappum Inippum         C. R. Devaraj   
..   ...                              ...                   ...   
68  1983                           Sattam            K. Vijayan   
69  1983                 Seerum Singangal        Rama Narayanan   
70  1983                  Shasti Viratham        R. Thyagarajan   
71  1983                 Sivappu Sooriyan         V. Srinivasan   
72  1983                   Silk Silk Silk    Y. V. Gopikrishnan   
73  1983           Soorakottai Singakutti        Rama Narayanan   
74  1983                        Soorapuli                Venkat   
75  1983                  Subha Muhurtham                 Raghu   
76  1983                        Sumangali           D. Yoganand   
77  1983                       Thai Veedu        R. Thyagarajan   
78  1983                      Thalaimagan   M. R. Vijayachander   
79  1983                     Thambathigal       N. S. Rajendran   
80  1983        Thandikkappatta Nyayangal            M. Bhaskar   
81  1983                     Thanga Magan        A. Jagannathan   
82  1983              Thangaikkor Geetham           T. Rajendar   
83  1983                      Thodi Ragam                 Ramki   
84  1983        Thoongadha Kannendru Onru        R. Sundarrajan   
85  1983   Thoongathey Thambi Thoongathey      S. P. Muthuraman   
86  1983               Thudikkum Karangal         C. V. Sridhar   
87  1983                         Unmaigal          R. C. Sakthi   
88  1983             Urangatha Ninaivugal          R. Bhaskaran   
89  1983                Uruvangal Maralam                 Ramji   
90  1983               Uyirullavarai Usha           T. Rajendar   
91  1983                     Valarthakada    Kandasami-Singaram   
92  1983  Veetula Raman Veliyila Krishnan            Manivannan   
93  1983                      Vellai Roja        A. Jagannathan   
94  1983                 Villiyanur Matha         K. Thangappan   
95  1983                Yamirukka Bayamen          

In [42]:
getAllMusic()

ex.php?title=Manaivi_Illatha_Neram&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Manaivi_Illatha_Neram&actio...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Onnu_Theriyadha_Pappa&actio...


Exception
36 | 
ex.php?title=Onnu_Theriyadha_Pappa&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Penmayin_Unmai&action=edit&...


Exception
50 | 
ex.php?title=Penmayin_Unmai&action=edit&redlink=1
Exception
57 | 


In [43]:
getAllrating()

https://www.imdb.com/title/tt8903530/
Aanandha Kummi | 
https://www.imdb.com/title/tt0315139/
Aayiram Nilave Vaa | 5.8
https://www.imdb.com/title/tt0186801/
Adutha Varisu | 5.5
https://www.imdb.com/name/nm0006137/
Alai Payum Nenjangal | 
https://www.imdb.com/title/tt3323842/
Anal Kaatru | 
https://www.imdb.com/title/tt0315198/
Anney Anney | 5.9
https://www.imdb.com/title/tt0315189/
Antha Sila Naatkal | 
https://www.imdb.com/title/tt0362416/
Apoorva Sahodarigal | 
https://www.imdb.com/title/tt0214505/
Aval Oru Kaviyam | 
https://www.imdb.com/title/tt0315283/
Bhagavathipuram Railway Gate | 6.0
https://www.imdb.com/title/tt3323854/
Bramacharigal | 
https://www.imdb.com/title/tt6750994/
Dhooram Adhighamillai | 
https://www.imdb.com/title/tt3323862/
Dowry Kalyanam | 7.2
En Aasai Unnoduthan | 
https://www.imdb.com/title/tt0322983/
En Priyame | 
https://www.imdb.com/title/tt0315549/
Ennai Paar En Azhagai Paar | 
Gramathu Kiligal | 
https://www.imdb.com/title/tt2725262/
Idhu Enga Naadu | 
http

In [44]:
df

year                            title              director  \
1   1983                   Aanandha Kummi       Gokula Krishnan   
2   1983               Aayiram Nilave Vaa        A. S. Pragasam   
3   1983                    Adutha Varisu      S. P. Muthuraman   
4   1983             Alai Payum Nenjangal             H. Ramesh   
5   1983                      Anal Kaatru     Komal Swaminathan   
6   1983                      Anney Anney                Moulee   
7   1983               Antha Sila Naatkal                Venkat   
8   1983              Apoorva Sahodarigal        R. Thyagarajan   
9   1983                 Aval Oru Kaviyam                 Durai   
10  1983     Bhagavathipuram Railway Gate           R. Selvaraj   
11  1983                    Bramacharigal         Swarnamoorthy   
12  1983            Dhooram Adhighamillai    Robert-Rajasekaran   
13  1983                   Dowry Kalyanam                  Visu   
14  1983              En Aasai Unnoduthan          K. Narayanan   
15  1983                       En Priyame             Sivakumar   
16  1983       Ennai Paar En Azhagai Paar          N. S. Maniam   
17  1983                 Gramathu Kiligal       Sanjeevi Ganesh   
18  1983                  Idhu Enga Naadu        Rama Narayanan   
19  1983                 Ilamai Kaalangal            Manivannan   
20  1983                   Ilayapiravigal        A. Irudaya Raj   
21  1983                          Imaigal      R. Krishnamurthy   
22  1983             Indru Nee Nalai Naan     Major Sundarrajan   
23  1983                 Inimai Idho Idho         R. Ramalingam   
24  1983                        Jodi Pura        V. K. Ramasamy   
25  1983                            Jothi            Manivannan   
26  1983                      Kai Varisai            Prem Kumar   
27  1983                Kalvadiyum Pookal        V. Karthigeyan   
28  1983                   Kaman Pandigai            H. S. Venu   
29  1983    Kann Sivanthaal Mann Sivakkum        Sreedhar Rajan   
30  1983                 Kasappum Inippum         C. R. Devaraj   
..   ...                              ...                   ...   
68  1983                           Sattam            K. Vijayan   
69  1983                 Seerum Singangal        Rama Narayanan   
70  1983                  Shasti Viratham        R. Thyagarajan   
71  1983                 Sivappu Sooriyan         V. Srinivasan   
72  1983                   Silk Silk Silk    Y. V. Gopikrishnan   
73  1983           Soorakottai Singakutti        Rama Narayanan   
74  1983                        Soorapuli                Venkat   
75  1983                  Subha Muhurtham                 Raghu   
76  1983                        Sumangali           D. Yoganand   
77  1983                       Thai Veedu        R. Thyagarajan   
78  1983                      Thalaimagan   M. R. Vijayachander   
79  1983                     Thambathigal       N. S. Rajendran   
80  1983        Thandikkappatta Nyayangal            M. Bhaskar   
81  1983                     Thanga Magan        A. Jagannathan   
82  1983              Thangaikkor Geetham           T. Rajendar   
83  1983                      Thodi Ragam                 Ramki   
84  1983        Thoongadha Kannendru Onru        R. Sundarrajan   
85  1983   Thoongathey Thambi Thoongathey      S. P. Muthuraman   
86  1983               Thudikkum Karangal         C. V. Sridhar   
87  1983                         Unmaigal          R. C. Sakthi   
88  1983             Urangatha Ninaivugal          R. Bhaskaran   
89  1983                Uruvangal Maralam                 Ramji   
90  1983               Uyirullavarai Usha           T. Rajendar   
91  1983                     Valarthakada    Kandasami-Singaram   
92  1983  Veetula Raman Veliyila Krishnan            Manivannan   
93  1983                      Vellai Roja        A. Jagannathan   
94  1983                 Villiyanur Matha         K. Thangappan   
95  1983                Yamirukka Bayamen          

In [45]:
df.to_csv('1983_movies.csv')

# 1982

In [46]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1982'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1982=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [47]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [48]:
for table in tables_1982:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1982'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Adhisayappiravigal|R. Thyagarajan|Karthik, Jaishankar, Prabhu, Radha, Rajyalakshmi, Y. G. Mahendra, Vanitha, S. Ve. Shekher, Silk Smitha|null|Devar Films|/wiki/Adhisayappiravigal
2!Agaya Gangai|Manobala|Karthik, Suhasini, Goundamani|null|Sreeni Enterprises|/wiki/Agaya_Gangai_(film)
3!Agni Sakshi|K. Balachander|Sivakumar, Saritha, Swapna|null|Kavithalayaa Productions|/wiki/Agni_Sakshi_(1982_film)
4!Amma|Rajasekhar|Pratap K. Pothen, Saritha|null|AVM Productions|/w/index.php?title=Amma_(1982_film)&action=edit&redlink=1
5!Anandha Ragam|Bharani|Sivakumar, Radha, Aruna, Goundamani|null|Panchu Arts|/wiki/Anandha_Ragam
6!Antha Rathirikku Satchi Illai|R. Sundarrajan|Kapildev, Sivachandran, Sulakshana, Vanitha, Usha Rajendar|null|K. R. G. Art Productions|/wiki/Antha_Rathirikku_Satchi_Illai
7!Anu|P. N. Menon|Jaya Devi, Raja, S. S. R. Rajendrakumar|null|T. R. M. Creations|/w/index.php?title=Anu_(1982_film)&action=edit&redlink=1
8!Archanai Pookal

65!Nandri, Meendum Varuga|Moulee|Pratap K. Pothen, Suhasini, Jaishankar|null|Ponmalar International|/wiki/Nandri,_Meendum_Varuga
66!Nayakkirin Magal|K. S. Gopalakrishnan|Vijayakumar, Jayachitra, Jai Ganesh, Sathyaraj|null|Karpagam Studios|/w/index.php?title=Nayakkirin_Magal&action=edit&redlink=1
67!Neethi Devan Mayakkam|Bapu|Bhanu Chander, Madhavi|null|Sathya Chithra Combines|/wiki/Neethi_Devan_Mayakkam
68!Nenjangal|Major Sundarrajan|Sivaji Ganesan, Lakshmi, Vijayakumar, Silk Smitha|null|Vanitha Film Production|/wiki/Nenjangal
69!Nenjil Oru Ragam|T. Rajendar|Saritha, Rajeev|null|K. R. G. Art Productions|/w/index.php?title=Nenjil_Oru_Ragam&action=edit&redlink=1
70!Neram Vandhachu|M. Vellaisamy|Karthik, Radha, Thiagarajan, Usharani, Silk Smitha|null|Karthik Arts|/wiki/Neram_Vandhachu
71!Nijangal|K. S. Sethumadhavan|Sivakumar, Menaka|null|Ramya Chithra Productions|/w/index.php?title=Nijangal_(1982_film)&action=edit&redlink=1
72!Ninaivellam Nithya|C. V. Sridhar|Karthik, Jigi, Nizhalgal Rav

In [49]:
df=df.drop(0)
df

year                          title               director  \
1    1982             Adhisayappiravigal         R. Thyagarajan   
2    1982                   Agaya Gangai               Manobala   
3    1982                    Agni Sakshi         K. Balachander   
4    1982                           Amma             Rajasekhar   
5    1982                  Anandha Ragam                Bharani   
6    1982  Antha Rathirikku Satchi Illai         R. Sundarrajan   
7    1982                            Anu            P. N. Menon   
8    1982                Archanai Pookal        Gokula Krishnan   
9    1982                     Asthivaram     T. Thirunavukarasu   
10   1982                      Auto Raja             K. Vijayan   
11   1982             Aval Etriya Deepam          P. Bhanumathi   
12   1982            Avanuku Nigar Avane              M. Karnan   
13   1982               Ayiram Muthangal           S. Devarajan   
14   1982                 Azhagiya Kanne           J. Mahendran   
15   1982                 Boomboom Maadu            Adaikalavan   
16   1982             Chinnan Chirusugal         Rama Narayanan   
17   1982      Darling, Darling, Darling           K. Bhagyaraj   
18   1982        Deviyin Thiruvilaiyadal   K. S. Gopalakrishnan   
19   1982               Echchil Iravugal         A. S. Pragasam   
20   1982         Eera Vizhi Kaaviyangal      B. R. Ravishankar   
21   1982             Enkeyo Ketta Kural       S. P. Muthuraman   
22   1982               Erattai Manithan             K. Shankar   
23   1982             Ezhavathu Manithan           K. Hariharan   
24   1982               Garuda Saukiyama      K. S. Prakash Rao   
25   1982        Gopurangal Saivathillai             Manivannan   
26   1982                 Hitler Umanath            P. Madhavan   
27   1982            Idhayam Pesugirathu    S. A. Chandrasekhar   
28   1982                 Idho Varugiren  K. C. Krishna Moorthy   
29   1982                    Ilanjodigal         Rama Narayanan   
30   1982                  Iniyavale Vaa     N. C. Chakravarthy   
..    ...                            ...                    ...   
91   1982                Raga Bandhangal        P. Ramachandren   
92   1982                          Ranga         R. Thyagarajan   
93   1982                     Rani Theni       G. N. Rangarajan   
94   1982            Sakalakala Vallavan       S. P. Muthuraman   
95   1982                        Sangili        C. V. Rajendran   
96   1982            Sattam Sirikkiradhu          T. R. Ramanna   
97   1982                  Simla Special          V. Srinivasan   
98   1982                Sivantha Kangal         Rama Narayanan   
99   1982                       Sparisam           R. C. Sakthi   
100  1982             Thaai Mookaambikai             K. Shankar   
101  1982     Thambathyam Oru Sangeetham                 Venkat   
102  1982                Thanikattu Raja       V. C. Guhanathan   
103  1982              Thaniyatha Dhagam          E. M. Ibrahim   
104  1982    Theeradha Vilaiyattu Pillai         Rama Narayanan   
105  1982                        Theerpu       R. Krishnamurthy   
106  1982   Theerpugal Thiruththapadalam             M. Bhaskar   
107  1982                   Thooku Medai               Amirtham   
108  1982          Thooral Ninnu Pochchu           K. Bhagyaraj   
109  1982                  Thottal Sudum      N. S. Raj Bharath   
110  1982                         Thunai                  Durai   
111  1982                       Thunaivi   Valampuri Somanathan   
112  1982                         Thyagi        C. V. Rajendran   
113  1982                  Vaa Kanna Vaa            D. Yoganand   
114  1982                     Vadaimalai          Maruthi Valli   
115  1982                     Vadivangal       Madhukoor Kannan   
116  1982              Valibamey Vaa Vaa        P. Bharathiraja   
117  1982           Vasandhathil Or Naal   A. C. Tirulokchandar   
118  1982                 Vazhvey Maayam       R. Kr

In [50]:
getAllMusic()

ex.php?title=Aval_Etriya_Deepam&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Aval_Etriya_Deepam&action=e...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Kaduvulluku_Oru_Kadidham&ac...


Exception
11 | 
ex.php?title=Kaduvulluku_Oru_Kadidham&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Madhulai_Muthukkal&action=e...


Exception
33 | 
ex.php?title=Madhulai_Muthukkal&action=edit&redlink=1
Exception
48 | 


In [51]:
getAllrating()

https://www.imdb.com/title/tt8907852/
Adhisayappiravigal | 
https://www.imdb.com/title/tt0275172/
Agaya Gangai | 5.7
https://www.imdb.com/title/tt0154120/
Agni Sakshi | 6.9
https://www.imdb.com/title/tt1394157/
Amma | 
https://www.imdb.com/title/tt0280390/
Anandha Ragam | 6.1
https://www.imdb.com/title/tt8914134/
Antha Rathirikku Satchi Illai | 
https://www.imdb.com/title/tt0240344/
Anu | 
https://www.imdb.com/title/tt8919066/
Archanai Pookal | 
https://www.imdb.com/title/tt3325316/
Asthivaram | 
https://www.imdb.com/title/tt8919300/
Auto Raja | 
https://www.imdb.com/title/tt0275205/
Aval Etriya Deepam | 7.5
https://www.imdb.com/title/tt3338460/
Avanuku Nigar Avane | 
https://www.imdb.com/title/tt10189644/
Ayiram Muthangal | 
https://www.imdb.com/title/tt0235203/
Azhagiya Kanne | 5.8
https://www.imdb.com/title/tt0321501/
Boomboom Maadu | 
https://www.imdb.com/title/tt0323112/
Chinnan Chirusugal | 
https://www.imdb.com/title/tt0155646/
Darling, Darling, Darling | 6.7
https://www.imdb.co

In [52]:
df

year                          title               director  \
1    1982             Adhisayappiravigal         R. Thyagarajan   
2    1982                   Agaya Gangai               Manobala   
3    1982                    Agni Sakshi         K. Balachander   
4    1982                           Amma             Rajasekhar   
5    1982                  Anandha Ragam                Bharani   
6    1982  Antha Rathirikku Satchi Illai         R. Sundarrajan   
7    1982                            Anu            P. N. Menon   
8    1982                Archanai Pookal        Gokula Krishnan   
9    1982                     Asthivaram     T. Thirunavukarasu   
10   1982                      Auto Raja             K. Vijayan   
11   1982             Aval Etriya Deepam          P. Bhanumathi   
12   1982            Avanuku Nigar Avane              M. Karnan   
13   1982               Ayiram Muthangal           S. Devarajan   
14   1982                 Azhagiya Kanne           J. Mahendran   
15   1982                 Boomboom Maadu            Adaikalavan   
16   1982             Chinnan Chirusugal         Rama Narayanan   
17   1982      Darling, Darling, Darling           K. Bhagyaraj   
18   1982        Deviyin Thiruvilaiyadal   K. S. Gopalakrishnan   
19   1982               Echchil Iravugal         A. S. Pragasam   
20   1982         Eera Vizhi Kaaviyangal      B. R. Ravishankar   
21   1982             Enkeyo Ketta Kural       S. P. Muthuraman   
22   1982               Erattai Manithan             K. Shankar   
23   1982             Ezhavathu Manithan           K. Hariharan   
24   1982               Garuda Saukiyama      K. S. Prakash Rao   
25   1982        Gopurangal Saivathillai             Manivannan   
26   1982                 Hitler Umanath            P. Madhavan   
27   1982            Idhayam Pesugirathu    S. A. Chandrasekhar   
28   1982                 Idho Varugiren  K. C. Krishna Moorthy   
29   1982                    Ilanjodigal         Rama Narayanan   
30   1982                  Iniyavale Vaa     N. C. Chakravarthy   
..    ...                            ...                    ...   
91   1982                Raga Bandhangal        P. Ramachandren   
92   1982                          Ranga         R. Thyagarajan   
93   1982                     Rani Theni       G. N. Rangarajan   
94   1982            Sakalakala Vallavan       S. P. Muthuraman   
95   1982                        Sangili        C. V. Rajendran   
96   1982            Sattam Sirikkiradhu          T. R. Ramanna   
97   1982                  Simla Special          V. Srinivasan   
98   1982                Sivantha Kangal         Rama Narayanan   
99   1982                       Sparisam           R. C. Sakthi   
100  1982             Thaai Mookaambikai             K. Shankar   
101  1982     Thambathyam Oru Sangeetham                 Venkat   
102  1982                Thanikattu Raja       V. C. Guhanathan   
103  1982              Thaniyatha Dhagam          E. M. Ibrahim   
104  1982    Theeradha Vilaiyattu Pillai         Rama Narayanan   
105  1982                        Theerpu       R. Krishnamurthy   
106  1982   Theerpugal Thiruththapadalam             M. Bhaskar   
107  1982                   Thooku Medai               Amirtham   
108  1982          Thooral Ninnu Pochchu           K. Bhagyaraj   
109  1982                  Thottal Sudum      N. S. Raj Bharath   
110  1982                         Thunai                  Durai   
111  1982                       Thunaivi   Valampuri Somanathan   
112  1982                         Thyagi        C. V. Rajendran   
113  1982                  Vaa Kanna Vaa            D. Yoganand   
114  1982                     Vadaimalai          Maruthi Valli   
115  1982                     Vadivangal       Madhukoor Kannan   
116  1982              Valibamey Vaa Vaa        P. Bharathiraja   
117  1982           Vasandhathil Or Naal   A. C. Tirulokchandar   
118  1982                 Vazhvey Maayam       R. Kr

In [53]:
df.to_csv('1982_movies.csv')

# 1981

In [54]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1981'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1981=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [55]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [56]:
for table in tables_1981:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1981'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!47 Natkal|K. Balachander|Chiranjeevi, Jaya Prada, Sarath Babu, Rama Prabha|null|Premalaya Productions|/wiki/47_Natkal
2!Aani Ver|K. Vijayan|Sivakumar, Saritha, Sathyaraj|null|Vivekanandha Pictures|/wiki/Aani_Ver
3!Sanditthanam|J.soban|Soban, Rukku|null|Dimple Creations|/w/index.php?title=Sanditthanam&action=edit&redlink=1
4!Adugal Nanaigindrana|P. Madhavan|Prasad, Roopa|null|Shanti Chithra Productions|/w/index.php?title=Adugal_Nanaigindrana&action=edit&redlink=1
5!Alaigal Oivathillai|P. Bharathiraja|Karthik, Radha, Thiagarajan, Silk Smitha|null|Pavalar Creation|/wiki/Alaigal_Oivathillai
6!Amara Kaaviyam|Amirtham|Sivaji Ganesan, Sripriya, Madhavi, Jai Ganesh, Nagesh|null|Viswanathan Combines|/wiki/Amara_Kaaviyam_(1981_film)
7!Anbulla Atthan|Kanmani Subbhu|Shoba, Kalaivanan|null|Metha Movies|/w/index.php?title=Anbulla_Atthan&action=edit&redlink=1
8!Andhi Mayakkam|Bhanudasan|Vanitha, Manoj, Ganthimathi, Vinod, Suveetha|null|Yuvaraja Pic

78!Rail Payanangalil|T. Rajendar|Sreenath, Rajeev, Jyothi, Sivaranjani|null|G. R. P. Arts|/wiki/Rail_Payanangalil
79!Raja Paarvai|Singeetam Srinivasa Rao|Kamal Haasan, Madhavi, L. V. Prasad, Y. G. Mahendra|null|Haasan Brothers|/wiki/Raja_Paarvai
80!Rajangam|R. C. Sakthi|Chandrasekhar, Vijayashanti, Charuhasan, Delhi Ganesh, Manorama, P. R. Varalakshmi|null|R. V. Creations|/w/index.php?title=Rajangam_(film)&action=edit&redlink=1
81!Ram Lakshman|R. Thyagarajan|Kamal Haasan, Sripriya, Raveendran|null|Devar Films|/wiki/Ram_Lakshman
82!Rani|Bharathan|Pratap K. Pothen, Nithya|null|Supriya Creations|/w/index.php?title=Rani_(film)&action=edit&redlink=1
83!Ranuva Veeran|S. P. Muthuraman|Rajinikanth, Sridevi, Chiranjeevi|null|Sathya Movies|/wiki/Ranuva_Veeran
84!Rathathin Rathame|Joe Devanand||null|Brighton Films|/w/index.php?title=Rathathin_Rathame&action=edit&redlink=1
85!Sankarlal|T. N. Balu|Kamal Haasan, Sridevi, Seema|null|Balu Cine Arts|/wiki/Sankarlal
86!Sathya Sundharam|K. S. Prakash Rao

In [57]:
df=df.drop(0)
df

year                     title                                  director  \
1    1981                 47 Natkal                            K. Balachander   
2    1981                  Aani Ver                                K. Vijayan   
3    1981              Sanditthanam                                   J.soban   
4    1981      Adugal Nanaigindrana                               P. Madhavan   
5    1981       Alaigal Oivathillai                           P. Bharathiraja   
6    1981            Amara Kaaviyam                                  Amirtham   
7    1981            Anbulla Atthan                            Kanmani Subbhu   
8    1981            Andhi Mayakkam                                Bhanudasan   
9    1981  Andru Muthal Indru Varai                               R. Sundaram   
10   1981              Anicha Malar                           Sri Vijayakumar   
11   1981        Antha Ezhu Naatkal                              K. Bhagyaraj   
12   1981         Anjatha Nenjangal                            R. Thyagarajan   
13   1981                 Aradhanai                                    Prasad   
14   1981          Arthangal Ayiram                                Raj Kannan   
15   1981                 Arumbugal                       Jupiter Chinnadurai   
16   1981         Avalum Thaayaanal                              Mohammed Ali   
17   1981               Avasarakari                           K. S. Mathangan   
18   1981              Bala Nagamma                                K. Shankar   
19   1981      Chinna Mul Peria Mul                         N. S. Raj Bharath   
20   1981       Deiva Thirumanangal  P. NeelakantanK. ShankarK. Kameswara Rao   
21   1981           Devi Dharisanam                                K. Shankar   
22   1981         Ellam Inba Mayyam                          G. N. Rangarajan   
23   1981         Enakkaga Kaathiru                               P. S. Nivas   
24   1981         Enga Ooru Kannagi                            K. Balachander   
25   1981          Engamma Magarani                                M. A. Kaja   
26   1981                  Garjanai                           C. V. Rajendran   
27   1981      Indru Poi Naalai Vaa                              K. Bhagyaraj   
28   1981          Jadhikkoru Needi                       S. A. Chandrasekhar   
29   1981                    Kaalam                          R. S. Somanathan   
30   1981             Kadal Meengal                          G. N. Rangarajan   
..    ...                       ...                                       ...   
75   1981            Pennin Vazhkai                                K. Vijayan   
76   1981                Ponnazhagi                                  O. Muthu   
77   1981           Purandharadasar                             R. Ramamurthy   
78   1981         Rail Payanangalil                               T. Rajendar   
79   1981              Raja Paarvai                   Singeetam Srinivasa Rao   
80   1981                  Rajangam                              R. C. Sakthi   
81   1981              Ram Lakshman                            R. Thyagarajan   
82   1981                      Rani                                 Bharathan   
83   1981             Ranuva Veeran                          S. P. Muthuraman   
84   1981         Rathathin Rathame                              Joe Devanand   
85   1981                 Sankarlal                                T. N. Balu   
86   1981          Sathya Sundharam                         K. S. Prakash Rao   
87   1981      Sattam Oru Iruttarai                       S. A. Chandrasekhar   
88   1981                    Savaal                          R. Krishnamurthy   
89   1981             Sivappu Malli                            Rama Narayanan   
90   1981    Sollathe Yarum Kettaal                         N. S. Raj Bharath   
91   1981                 Sooravali                               Nimai Ghosh   
92   1981  Sorgathin Thirappu Vizha 

In [58]:
getAllMusic()

In [59]:
getAllrating()

https://www.imdb.com/title/tt0154101/
47 Natkal | 6.8
https://www.imdb.com/title/tt8702632/
Aani Ver | 
https://www.imdb.com/title/tt0371933/
Sanditthanam | 
https://www.imdb.com/title/tt10183950/
Adugal Nanaigindrana | 
https://www.imdb.com/title/tt0155481/
Alaigal Oivathillai | 6.6
https://www.imdb.com/title/tt1435448/
Amara Kaaviyam | 
https://www.imdb.com/title/tt2710272/
Anbulla Atthan | 
Andhi Mayakkam | 
https://www.imdb.com/title/tt8914650/
Andru Muthal Indru Varai | 
Anicha Malar | 
https://www.imdb.com/title/tt0155500/
Antha Ezhu Naatkal | 7.3
Anjatha Nenjangal | 
https://www.imdb.com/title/tt0278892/
Aradhanai | 
https://www.imdb.com/title/tt10137060/
Arthangal Ayiram | 
https://www.imdb.com/title/tt0186830/
Arumbugal | 3.8
Avalum Thaayaanal | 
https://www.imdb.com/title/tt8929104/
Avasarakari | 
https://www.imdb.com/title/tt0186849/
Bala Nagamma | 5.0
https://www.imdb.com/title/tt8938380/
Chinna Mul Peria Mul | 
https://www.imdb.com/title/tt0186940/
Deiva Thirumanangal | 5.

In [60]:
df

year                     title                                  director  \
1    1981                 47 Natkal                            K. Balachander   
2    1981                  Aani Ver                                K. Vijayan   
3    1981              Sanditthanam                                   J.soban   
4    1981      Adugal Nanaigindrana                               P. Madhavan   
5    1981       Alaigal Oivathillai                           P. Bharathiraja   
6    1981            Amara Kaaviyam                                  Amirtham   
7    1981            Anbulla Atthan                            Kanmani Subbhu   
8    1981            Andhi Mayakkam                                Bhanudasan   
9    1981  Andru Muthal Indru Varai                               R. Sundaram   
10   1981              Anicha Malar                           Sri Vijayakumar   
11   1981        Antha Ezhu Naatkal                              K. Bhagyaraj   
12   1981         Anjatha Nenjangal                            R. Thyagarajan   
13   1981                 Aradhanai                                    Prasad   
14   1981          Arthangal Ayiram                                Raj Kannan   
15   1981                 Arumbugal                       Jupiter Chinnadurai   
16   1981         Avalum Thaayaanal                              Mohammed Ali   
17   1981               Avasarakari                           K. S. Mathangan   
18   1981              Bala Nagamma                                K. Shankar   
19   1981      Chinna Mul Peria Mul                         N. S. Raj Bharath   
20   1981       Deiva Thirumanangal  P. NeelakantanK. ShankarK. Kameswara Rao   
21   1981           Devi Dharisanam                                K. Shankar   
22   1981         Ellam Inba Mayyam                          G. N. Rangarajan   
23   1981         Enakkaga Kaathiru                               P. S. Nivas   
24   1981         Enga Ooru Kannagi                            K. Balachander   
25   1981          Engamma Magarani                                M. A. Kaja   
26   1981                  Garjanai                           C. V. Rajendran   
27   1981      Indru Poi Naalai Vaa                              K. Bhagyaraj   
28   1981          Jadhikkoru Needi                       S. A. Chandrasekhar   
29   1981                    Kaalam                          R. S. Somanathan   
30   1981             Kadal Meengal                          G. N. Rangarajan   
..    ...                       ...                                       ...   
75   1981            Pennin Vazhkai                                K. Vijayan   
76   1981                Ponnazhagi                                  O. Muthu   
77   1981           Purandharadasar                             R. Ramamurthy   
78   1981         Rail Payanangalil                               T. Rajendar   
79   1981              Raja Paarvai                   Singeetam Srinivasa Rao   
80   1981                  Rajangam                              R. C. Sakthi   
81   1981              Ram Lakshman                            R. Thyagarajan   
82   1981                      Rani                                 Bharathan   
83   1981             Ranuva Veeran                          S. P. Muthuraman   
84   1981         Rathathin Rathame                              Joe Devanand   
85   1981                 Sankarlal                                T. N. Balu   
86   1981          Sathya Sundharam                         K. S. Prakash Rao   
87   1981      Sattam Oru Iruttarai                       S. A. Chandrasekhar   
88   1981                    Savaal                          R. Krishnamurthy   
89   1981             Sivappu Malli                            Rama Narayanan   
90   1981    Sollathe Yarum Kettaal                         N. S. Raj Bharath   
91   1981                 Sooravali                               Nimai Ghosh   
92   1981  Sorgathin Thirappu Vizha 

In [61]:
df.to_csv('1981_movies.csv')

# 1980

In [116]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1980'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1980=soup.find_all('table')[2:3]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [117]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [118]:
for table in tables_1980:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=1
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    minus_factor=0
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1980'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

   
    i=0
    for datarow in rows:
      if 'Genre' in column_names:
        genre_member=True
        genre_index=column_names.index('Genre')-minus_factor

      if 'Director' in column_names:
        director_member=True
        director_index=column_names.index('Director')-minus_factor

      if 'Cast' in column_names:
        cast_member=True
        cast_index=column_names.index('Cast')-minus_factor

      if 'Studio' in column_names:
        producer_index=column_names.index('Studio')-minus_factor
        producer_member=True

      if 'Producer' in column_names:
        producer_index=column_names.index('Producer')-minus_factor
        producer_member=True

      if 'Production' in column_names:
        producer_index=column_names.index('Production')-minus_factor
        producer_member=True

      if 'Production Studio' in column_names:
        producer_index=column_names.index('Production Studio')-minus_factor
        producer_member=True
      if 'Music' in column_names:
        music_index=column_names.index('Music')-minus_factor
        music_member=True
      if i==0:
        i=i+1
        continue
        
      all_data=datarow.find_all('td')


      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]




['Date', 'Title', 'Director', 'Producer', 'Music', 'Cast']
1!Aayiram Vaasal Idhayam|A. Jagannathan|Sudhakar, Raadhika, Roja Ramani, Thengai Srinivasan|null|Selvam Arts|/wiki/Aayiram_Vaasal_Idhayam
2!Rishi Moolam|S. P. Muthuraman|Sivaji Ganesan, K. R. Vijaya|null|S. S. K. Films|/wiki/Rishi_Moolam
3!Billa|R. Krishnamurthy|Rajinikanth, Sripriya, Helen, K. Balaji|null|Suresh Arts|/wiki/Billa_(1980_film)
4!Deiveega Raagangal|A. Veerappan|Srikanth, Vadivukkarasi, Roja Ramani|null|Aachi Cine Arts|/wiki/Deiveega_Raagangal
5!Avan Aval Adhu|V. Srinivasan|Sivakumar, Lakshmi, Sripriya, Thengai Srinivasan, Manorama|null|Maya Arts|/wiki/Avan_Aval_Adhu
6!Yamanukku Yaman|D. Yoganand|Sivaji Ganesan, Sripriya|null|Lakshmi Narasimha Pictures|/wiki/Yamanukku_Yaman
7!Andharangam Oomaiyanathu|G. Premkumar|Sarath Babu, Jose, M. G. Soman, Roopa, Rajani Sharma, Jayamalini, Jalaja, Suruli Rajan|null|Lankal Films|/w/index.php?title=Andharangam_Oomaiyanathu&action=edit&redlink=1
8!Anna Paravai|R. Pattabhiraman|La

84!Samandhipoo|K. S. Mathangan|Sivakumar, Shoba|null|Anand Creation|/w/index.php?title=Samandhipoo&action=edit&redlink=1
85!Saranam Ayyappa|Dasarathan|Jayabharathi, Vijayan, Radha Ravi|null|Amudeshwari Films|/wiki/Saranam_Ayyappa
86!Savitri|Bharathan|Vinod, Menaka|null|Lena Productions|/w/index.php?title=Savitri_(1980_film)&action=edit&redlink=1
87!Soolam|M. Bhaskar|Rajkumar, Ponni|null|Oscar Movies|/wiki/Soolam
88!Soundaryame Varuga Varuga|C. V. Sridhar|Sivachandran, Sripriya|null|Sri Bharani Chithra Enterprises|/wiki/Soundaryame_Varuga_Varuga
89!Sridevi|Balaji||null|U. M. Productions|/w/index.php?title=Sridevi_(Tamil_film)&action=edit&redlink=1
90!Sujatha|Mohan|Vijayan, Saritha, Shankar, Rajyalakshmi, Raveendran|null|Sujatha Cine Arts|/wiki/Sujatha_(1980_film)
91!Thai Pongal|M. G. Vallabhan|Vijayan, Chakravarthy, Raadhika, Saritha|null|Anjaneya Combines|/w/index.php?title=Thai_Pongal_(film)&action=edit&redlink=1
92!Thanimaram|Durai|Rajesh, Nalini, Suruli Rajan|null|Jayadevi Movies|/w

In [119]:
df=df.drop(0)
df

year                        title              director  \
1    1980       Aayiram Vaasal Idhayam        A. Jagannathan   
2    1980                 Rishi Moolam      S. P. Muthuraman   
3    1980                        Billa      R. Krishnamurthy   
4    1980           Deiveega Raagangal          A. Veerappan   
5    1980               Avan Aval Adhu         V. Srinivasan   
6    1980              Yamanukku Yaman           D. Yoganand   
7    1980     Andharangam Oomaiyanathu          G. Premkumar   
8    1980                 Anna Paravai      R. Pattabhiraman   
9    1980          Anbukku Naan Adimai        R. Thyagarajan   
10   1980            Azhaithal Varuven            P. R. Somu   
11   1980                Bhama Rukmani          R. Bhaskaran   
12   1980              Bombay Mail 109        T. P. Sundaram   
13   1980    Chinna Chinna Veedu Katti              Yuvaraja   
14   1980            Chinnajiru Kiliye       K. Chandra Bose   
15   1980                  Dharma Raja      M. A. Thirumugam   
16   1980          Edhir Veettu Jannal           Kalaignanam   
17   1980             Ellam Un Kairasi    M. A. Thirumurugan   
18   1980            Enga Ooru Rasathi       N. S. Rajendran   
19   1980               Engal Vadhiyar                 Durai   
20   1980       Geetha Oru Shenbagapoo          S. A. Kannan   
21   1980           Gramathu Athiyayam          C. Rudhraiya   
22   1980                         Guru            I. V. Sasi   
23   1980        Idhayaththil Ore Idam              Prashath   
24   1980                 Ilamai Kolam          N. Venkatesh   
25   1980         Inaindha Dhuruvangal  K. S. Gopalakrishnan   
26   1980  Ivargal Vidhiyasamanavargal                Moulee   
27   1980                       Jamboo             M. Karnan   
28   1980                       Johnny          J. Mahendran   
29   1980                        Kaadu                 Durai   
30   1980         Kaalam Badhil Sollum              Amirtham   
..    ...                          ...                   ...   
76   1980                   Porkkaalam                 Durai   
77   1980        Poottaatha Poottukkal          J. Mahendran   
78   1980             Pournami Nilavil            M. A. Kaja   
79   1980          Pudhiya Thoranangal             M. Karnan   
80   1980            Raman Parasuraman        M. S. Gopinath   
81   1980   Ramayi Vayasukku Vandhutta          V. Alagappan   
82   1980                 Ratha Paasam            K. Vijayan   
83   1980            Rusi Kanda Poonai        G.N.Rangarajan   
84   1980                  Samandhipoo       K. S. Mathangan   
85   1980              Saranam Ayyappa            Dasarathan   
86   1980                      Savitri             Bharathan   
87   1980                       Soolam            M. Bhaskar   
88   1980    Soundaryame Varuga Varuga         C. V. Sridhar   
89   1980                      Sridevi                Balaji   
90   1980                      Sujatha                 Mohan   
91   1980                  Thai Pongal       M. G. Vallabhan   
92   1980                   Thanimaram                 Durai   
93   1980        Tharayil Pootha Malar             Jai Durai   
94   1980                Theru Vilakku            M. A. Kaja   
95   1980              Thunive Thozhan           R. Sundaram   
96   1980                   Uchakattam     N. S. Raj Bharath   
97   1980            Ullasa Paravaigal       C. V. Rajendran   
98   1980               Vandichakkaram            K. Vijayan   
99   1980      Varumayin Niram Sivappu        K. Balachander   
100  1980                 Vishwaroopam  A. C. Tirulokchandar   
101  1980       Doorathu Idi Muzhakkam            K. Vijayan   
102  1980                 Yaaga Saalai     K. V. Manisekaran   
103  1980         Vasantha Azhaippugal           T. Rajendar   
104  1980        Veli Thandiya Velladu            M. A. Kaja   
105  1980                  Valli Mayil            M. A. Kaja   

                                    

In [120]:
getAllMusic()

ex.php?title=Madhavi_Vandhal&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Madhavi_Vandhal&action=edit...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Manmatha_Ragangal&action=ed...


Exception
39 | 
ex.php?title=Manmatha_Ragangal&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Sridevi_(Tamil_film)&action...


Exception
43 | 
ex.php?title=Sridevi_(Tamil_film)&action=edit&redlink=1
Exception
89 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [121]:
getAllrating()

https://www.imdb.com/title/tt0274351/
Aayiram Vaasal Idhayam | 
https://www.imdb.com/title/tt0279370/
Rishi Moolam | 4.5
https://www.imdb.com/title/tt0364236/
Billa | 7.4
https://www.imdb.com/title/tt2720480/
Deiveega Raagangal | 
https://www.imdb.com/title/tt2716504/
Avan Aval Adhu | 4.4
https://www.imdb.com/title/tt0372035/
Yamanukku Yaman | 6.2
https://www.imdb.com/title/tt0362413/
Andharangam Oomaiyanathu | 5.8
https://www.imdb.com/title/tt3338508/
Anna Paravai | 
https://www.imdb.com/title/tt0274336/
Anbukku Naan Adimai | 6.1
https://www.imdb.com/title/tt2727220/
Azhaithal Varuven | 
https://www.imdb.com/title/tt0155546/
Bhama Rukmani | 6.0
https://www.imdb.com/title/tt2720338/
Bombay Mail 109 | 
https://www.imdb.com/title/tt8938340/
Chinna Chinna Veedu Katti | 
https://www.imdb.com/title/tt3338512/
Chinnajiru Kiliye | 
https://www.imdb.com/title/tt1437599/
Dharma Raja | 
https://www.imdb.com/search/title/?title_type=feature&year=1980-01-01,1989-12-31&languages=ta
Edhir Veettu Jan

In [122]:
df

year                        title              director  \
1    1980       Aayiram Vaasal Idhayam        A. Jagannathan   
2    1980                 Rishi Moolam      S. P. Muthuraman   
3    1980                        Billa      R. Krishnamurthy   
4    1980           Deiveega Raagangal          A. Veerappan   
5    1980               Avan Aval Adhu         V. Srinivasan   
6    1980              Yamanukku Yaman           D. Yoganand   
7    1980     Andharangam Oomaiyanathu          G. Premkumar   
8    1980                 Anna Paravai      R. Pattabhiraman   
9    1980          Anbukku Naan Adimai        R. Thyagarajan   
10   1980            Azhaithal Varuven            P. R. Somu   
11   1980                Bhama Rukmani          R. Bhaskaran   
12   1980              Bombay Mail 109        T. P. Sundaram   
13   1980    Chinna Chinna Veedu Katti              Yuvaraja   
14   1980            Chinnajiru Kiliye       K. Chandra Bose   
15   1980                  Dharma Raja      M. A. Thirumugam   
16   1980          Edhir Veettu Jannal           Kalaignanam   
17   1980             Ellam Un Kairasi    M. A. Thirumurugan   
18   1980            Enga Ooru Rasathi       N. S. Rajendran   
19   1980               Engal Vadhiyar                 Durai   
20   1980       Geetha Oru Shenbagapoo          S. A. Kannan   
21   1980           Gramathu Athiyayam          C. Rudhraiya   
22   1980                         Guru            I. V. Sasi   
23   1980        Idhayaththil Ore Idam              Prashath   
24   1980                 Ilamai Kolam          N. Venkatesh   
25   1980         Inaindha Dhuruvangal  K. S. Gopalakrishnan   
26   1980  Ivargal Vidhiyasamanavargal                Moulee   
27   1980                       Jamboo             M. Karnan   
28   1980                       Johnny          J. Mahendran   
29   1980                        Kaadu                 Durai   
30   1980         Kaalam Badhil Sollum              Amirtham   
..    ...                          ...                   ...   
76   1980                   Porkkaalam                 Durai   
77   1980        Poottaatha Poottukkal          J. Mahendran   
78   1980             Pournami Nilavil            M. A. Kaja   
79   1980          Pudhiya Thoranangal             M. Karnan   
80   1980            Raman Parasuraman        M. S. Gopinath   
81   1980   Ramayi Vayasukku Vandhutta          V. Alagappan   
82   1980                 Ratha Paasam            K. Vijayan   
83   1980            Rusi Kanda Poonai        G.N.Rangarajan   
84   1980                  Samandhipoo       K. S. Mathangan   
85   1980              Saranam Ayyappa            Dasarathan   
86   1980                      Savitri             Bharathan   
87   1980                       Soolam            M. Bhaskar   
88   1980    Soundaryame Varuga Varuga         C. V. Sridhar   
89   1980                      Sridevi                Balaji   
90   1980                      Sujatha                 Mohan   
91   1980                  Thai Pongal       M. G. Vallabhan   
92   1980                   Thanimaram                 Durai   
93   1980        Tharayil Pootha Malar             Jai Durai   
94   1980                Theru Vilakku            M. A. Kaja   
95   1980              Thunive Thozhan           R. Sundaram   
96   1980                   Uchakattam     N. S. Raj Bharath   
97   1980            Ullasa Paravaigal       C. V. Rajendran   
98   1980               Vandichakkaram            K. Vijayan   
99   1980      Varumayin Niram Sivappu        K. Balachander   
100  1980                 Vishwaroopam  A. C. Tirulokchandar   
101  1980       Doorathu Idi Muzhakkam            K. Vijayan   
102  1980                 Yaaga Saalai     K. V. Manisekaran   
103  1980         Vasantha Azhaippugal           T. Rajendar   
104  1980        Veli Thandiya Velladu            M. A. Kaja   
105  1980                  Valli Mayil            M. A. Kaja   

                                    

In [123]:
df.to_csv('1980_movies.csv')

# 1979

In [20]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1979'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1979=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [21]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [22]:
for table in tables_1979:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1979'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Aadu Pambe|Amirtham|Jaishankar, Sangeeta, Sumithra|null|Poompuhar Productions|/w/index.php?title=Aadu_Pambe&action=edit&redlink=1
2!Aarilirunthu Arubathu Varai|S. P. Muthuraman|Rajinikanth, Fatafat Jayalaxmi, Jaya Guhanathan, Cho, Sangeetha|null|P. A. Art Productions|/wiki/Aarilirunthu_Arubathu_Varai
3!Aasaikku Vayasillai|Gopu|R. Muthuraman, K. R. Vijaya|null|Shanmugam Cine Enterprises|/w/index.php?title=Aasaikku_Vayasillai&action=edit&redlink=1
4!Adhisaya Raagam|Mohanraj-Suresh|Delhi Ganesh, Jamila|null|Film Talents Acedamy|/w/index.php?title=Adhisaya_Raagam&action=edit&redlink=1
5!Adukku Malli|K. S. Gopalakrishnan|Vijayakumar, Sujatha, Vanitha, Thengai Srinivasan|null|Meena Art Pictures|/w/index.php?title=Adukku_Malli&action=edit&redlink=1
6!Agal Vilakku|R. Selvaraj|Vijayakanth, Shoba|null|Sunil Pictures|/wiki/Agal_Vilakku
7!Allaudinaum Arputha Vilakkum|I. V. Sasi|Kamal Haasan, Gemini Ganesan, Rajinikanth, Jayabharathi, Sripriya,

81!Sigappukkal Mookkuthi|Valampuri Somanathan|Kamal Haasan, Sridevi, Vijayakumar, Roja Ramani|null|Swarnambika Films|/wiki/Sigappukkal_Mookkuthi
82!Siri Siri Mama|Karunanidi|Vijayakumar, Padmapriya|null|Manju Films|/w/index.php?title=Siri_Siri_Mama&action=edit&redlink=1
83!Sri Rama Jayam|V. Srinivasan|Thengai Srinivasan, Sujatha, Abarna|null|Karthigai Films|/w/index.php?title=Sri_Rama_Jayam&action=edit&redlink=1
84!Suprabadham|K. Shankar|R. Muthuraman, K. R. Vijaya, Jai Ganesh, Sripriya|null|P. V. T. Productions|/w/index.php?title=Suprabadham&action=edit&redlink=1
85!Suvarilladha Chiththirangal|K. Bhagyaraj|Sudhakar, Sumathi, K. Bhagyaraj|null|Bagavathy Creations|/wiki/Suvarilladha_Chiththirangal
86!Thaayillamal Naan Illai|R. Thyagarajan|Kamal Haasan, Sridevi, Jai Ganesh|null|Devar Films|/wiki/Thaayillamal_Naan_Illai
87!Thevaigal|N. S. Ravishankar|Jai Ganesh, Sumithra|null|Amirtha Jothi Combines|/w/index.php?title=Thevaigal&action=edit&redlink=1
88!Thirisoolam|K. Vijayan|Sivaji Ganesan

In [23]:
df=df.drop(0)
df

year                         title               director  \
1   1979                    Aadu Pambe               Amirtham   
2   1979   Aarilirunthu Arubathu Varai       S. P. Muthuraman   
3   1979           Aasaikku Vayasillai                   Gopu   
4   1979               Adhisaya Raagam        Mohanraj-Suresh   
5   1979                  Adukku Malli   K. S. Gopalakrishnan   
6   1979                  Agal Vilakku            R. Selvaraj   
7   1979  Allaudinaum Arputha Vilakkum             I. V. Sasi   
8   1979                      Alankari                   Gopu   
9   1979                Anbe Sangeetha           K. Narayanan   
10  1979                 Anbin Alaigal              M. Mastan   
11  1979             Annai Oru Aalayam         R. Thyagarajan   
12  1979       Appothe Sonnene Kettiya            V. T. Arasu   
13  1979    Azhage Unnai Aarathikkiren          C. V. Sridhar   
14  1979            Azhiyatha Kolangal          Balu Mahendra   
15  1979                    Chellakili     K. M. Balakrishnan   
16  1979            Chithirai Sevvanam     N. C. Chakraborthy   
17  1979                      Devathai            P. N. Menon   
18  1979               Dhairya Lakshmi                   Gopu   
19  1979                Dharma Yuddham           R. C. Sakthi   
20  1979                   Enippadigal            P. Madhavan   
21  1979              Ennadi Meenakshi           K. Narayanan   
22  1979              Gnana Kuzhandhai      K. Kameshwara Rao   
23  1979                        Imayam          V. Srinivasan   
24  1979                Inikkum Ilamai             M. A. Kaja   
25  1979           Jaya Nee Jayuchutte         A. Jagannathan   
26  1979                Kadamai Nenjam                  Durai   
27  1979         Kadavul Amaitha Medai       S. P. Muthuraman   
28  1979                  Kalyanaraman       G. N. Rangarajan   
29  1979                 Kamasasthiram            Balamurugan   
30  1979             Kandhar Alangaram            K. Sundaram   
..   ...                           ...                    ...   
68  1979             Pagalil Oru Iravu             I. V. Sasi   
69  1979                Pancha Boodham                Sathyam   
70  1979                Pancha Kalyani           N. Sambandam   
71  1979                      Pappathi       R. K. Shanmugham   
72  1979                          Pasi                  Durai   
73  1979         Pattakkathi Bhairavan  V. B. Rajendra Prasad   
74  1979         Ponnu Oorukku Pudhusu            R. Selvaraj   
75  1979                    Poonthalir          Devaraj–Mohan   
76  1979              Portar Ponnusamy                 Adiyar   
77  1979             Puthiya Vaarpugal        P. Bharathiraja   
78  1979                Raja Rajeswari           N. Sambandam   
79  1979          Rosappu Ravikkaikari          Devaraj–Mohan   
80  1979                   Chakkalathi          Devaraj Mohan   
81  1979         Sigappukkal Mookkuthi   Valampuri Somanathan   
82  1979                Siri Siri Mama             Karunanidi   
83  1979                Sri Rama Jayam          V. Srinivasan   
84  1979                   Suprabadham             K. Shankar   
85  1979   Suvarilladha Chiththirangal           K. Bhagyaraj   
86  1979       Thaayillamal Naan Illai         R. Thyagarajan   
87  1979                     Thevaigal      N. S. Ravishankar   
88  1979                   Thirisoolam             K. Vijayan   
89  1979     Thisai Maariya Paravaigal          S. Jagadeesan   
90  1979                Udhiri Pookkal           J. Mahendran   
91  1979           Urangadha Kanavugal            A. K. Velan   
92  1979            Vallavan Varugiran            R. Sundaram   
93  1979      Veettukku Veedu Vasapadi            P. Madhavan   
94  1979                  Velli Ratham         Krishnan–Panju   
95  1979          Velum Mayilum Thunai           Ra. Sankaran   
96  1979               Vetrikku Oruvan       S. P. Muthuraman   
97  1979            Yarukku Yar Kaaval 

In [24]:
getAllMusic()

ex.php?title=Adhisaya_Raagam&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Adhisaya_Raagam&action=edit...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Oorukku_Oru_Raja&action=edi...


Exception
4 | 
ex.php?title=Oorukku_Oru_Raja&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Siri_Siri_Mama&action=edit&...


Exception
63 | 
ex.php?title=Siri_Siri_Mama&action=edit&redlink=1
Exception
82 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [25]:
getAllrating()

https://www.imdb.com/title/tt2710208/
Aadu Pambe | 
https://www.imdb.com/title/tt0274314/
Aarilirunthu Arubathu Varai | 8.3
https://www.imdb.com/title/tt2710222/
Aasaikku Vayasillai | 
https://www.imdb.com/title/tt0154159/
Adhisaya Raagam | 7.5
https://www.imdb.com/title/tt0215469/
Adukku Malli | 
https://www.imdb.com/title/tt0277557/
Agal Vilakku | 6.4
https://www.imdb.com/title/tt0324977/
Allaudinaum Arputha Vilakkum | 6.6
https://www.imdb.com/title/tt0186794/
Alankari | 6.5
https://www.imdb.com/title/tt0277572/
Anbe Sangeetha | 
https://www.imdb.com/title/tt0318990/
Anbin Alaigal | 5.2
https://www.imdb.com/title/tt0277580/
Annai Oru Aalayam | 7.0
https://www.imdb.com/title/tt0277946/
Appothe Sonnene Kettiya | 3.8
https://www.imdb.com/title/tt0277589/
Azhage Unnai Aarathikkiren | 5.8
https://www.imdb.com/title/tt0235204/
Azhiyatha Kolangal | 6.8
https://www.imdb.com/title/tt0277627/
Chellakili | 
https://www.imdb.com/title/tt8718580/
Chithirai Sevvanam | 9.2
https://www.imdb.com/titl

In [26]:
df

year                         title               director  \
1   1979                    Aadu Pambe               Amirtham   
2   1979   Aarilirunthu Arubathu Varai       S. P. Muthuraman   
3   1979           Aasaikku Vayasillai                   Gopu   
4   1979               Adhisaya Raagam        Mohanraj-Suresh   
5   1979                  Adukku Malli   K. S. Gopalakrishnan   
6   1979                  Agal Vilakku            R. Selvaraj   
7   1979  Allaudinaum Arputha Vilakkum             I. V. Sasi   
8   1979                      Alankari                   Gopu   
9   1979                Anbe Sangeetha           K. Narayanan   
10  1979                 Anbin Alaigal              M. Mastan   
11  1979             Annai Oru Aalayam         R. Thyagarajan   
12  1979       Appothe Sonnene Kettiya            V. T. Arasu   
13  1979    Azhage Unnai Aarathikkiren          C. V. Sridhar   
14  1979            Azhiyatha Kolangal          Balu Mahendra   
15  1979                    Chellakili     K. M. Balakrishnan   
16  1979            Chithirai Sevvanam     N. C. Chakraborthy   
17  1979                      Devathai            P. N. Menon   
18  1979               Dhairya Lakshmi                   Gopu   
19  1979                Dharma Yuddham           R. C. Sakthi   
20  1979                   Enippadigal            P. Madhavan   
21  1979              Ennadi Meenakshi           K. Narayanan   
22  1979              Gnana Kuzhandhai      K. Kameshwara Rao   
23  1979                        Imayam          V. Srinivasan   
24  1979                Inikkum Ilamai             M. A. Kaja   
25  1979           Jaya Nee Jayuchutte         A. Jagannathan   
26  1979                Kadamai Nenjam                  Durai   
27  1979         Kadavul Amaitha Medai       S. P. Muthuraman   
28  1979                  Kalyanaraman       G. N. Rangarajan   
29  1979                 Kamasasthiram            Balamurugan   
30  1979             Kandhar Alangaram            K. Sundaram   
..   ...                           ...                    ...   
68  1979             Pagalil Oru Iravu             I. V. Sasi   
69  1979                Pancha Boodham                Sathyam   
70  1979                Pancha Kalyani           N. Sambandam   
71  1979                      Pappathi       R. K. Shanmugham   
72  1979                          Pasi                  Durai   
73  1979         Pattakkathi Bhairavan  V. B. Rajendra Prasad   
74  1979         Ponnu Oorukku Pudhusu            R. Selvaraj   
75  1979                    Poonthalir          Devaraj–Mohan   
76  1979              Portar Ponnusamy                 Adiyar   
77  1979             Puthiya Vaarpugal        P. Bharathiraja   
78  1979                Raja Rajeswari           N. Sambandam   
79  1979          Rosappu Ravikkaikari          Devaraj–Mohan   
80  1979                   Chakkalathi          Devaraj Mohan   
81  1979         Sigappukkal Mookkuthi   Valampuri Somanathan   
82  1979                Siri Siri Mama             Karunanidi   
83  1979                Sri Rama Jayam          V. Srinivasan   
84  1979                   Suprabadham             K. Shankar   
85  1979   Suvarilladha Chiththirangal           K. Bhagyaraj   
86  1979       Thaayillamal Naan Illai         R. Thyagarajan   
87  1979                     Thevaigal      N. S. Ravishankar   
88  1979                   Thirisoolam             K. Vijayan   
89  1979     Thisai Maariya Paravaigal          S. Jagadeesan   
90  1979                Udhiri Pookkal           J. Mahendran   
91  1979           Urangadha Kanavugal            A. K. Velan   
92  1979            Vallavan Varugiran            R. Sundaram   
93  1979      Veettukku Veedu Vasapadi            P. Madhavan   
94  1979                  Velli Ratham         Krishnan–Panju   
95  1979          Velum Mayilum Thunai           Ra. Sankaran   
96  1979               Vetrikku Oruvan       S. P. Muthuraman   
97  1979            Yarukku Yar Kaaval 

In [27]:
df.to_csv('1979_movies.csv')

# 1978

In [28]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1978'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1978=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [29]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [30]:
for table in tables_1978:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1978'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Achchani|Devaraj–Mohan|R. Muthuraman, Lakshmi, Shoba|null|Mahalakshmi Combines|/wiki/Achchani
2!Adhirstakaran|Venunath|Nagesh, Fatafat Jayalaxmi, V. K. Ramasamy, A. Sakunthala, Sukumari|null|Soundhara Combines|/w/index.php?title=Adhirstakaran&action=edit&redlink=1
3!Agni Pravesam|K. Narayanan|Srikanth, Padmapriya|null|United Combines|/w/index.php?title=Agni_Pravesam&action=edit&redlink=1
4!Alli Darbar|K. M. Balakrishnan|Vijayakumar, Manjula, Cho|null|Aiyaa Creations|/wiki/Alli_Darbar
5!Anantha Bairavi|Mohan Gandhiram|Ravichandran, K. R. Vijaya, Jayadevi|null|Sreevidhya Productions|/w/index.php?title=Anantha_Bairavi&action=edit&redlink=1
6!Andaman Kadhali|V. Srinivasan|Sivaji Ganesan, Sujatha, Chandra Mohan, Kavitha|null|Muktha Films|/wiki/Andaman_Kadhali
7!Annapoorani|Krishnan–Panju|R. Muthuraman, K. R. Vijaya|null|Vijayambika Films|/w/index.php?title=Annapoorani&action=edit&redlink=1
8!Athaivida Ragasiyam|Kiruba Shankar|Sivakumar,

In [31]:
df=df.drop(0)
df

year                       title              director  \
1   1978                    Achchani         Devaraj–Mohan   
2   1978               Adhirstakaran              Venunath   
3   1978               Agni Pravesam          K. Narayanan   
4   1978                 Alli Darbar    K. M. Balakrishnan   
5   1978             Anantha Bairavi       Mohan Gandhiram   
6   1978             Andaman Kadhali         V. Srinivasan   
7   1978                 Annapoorani        Krishnan–Panju   
8   1978         Athaivida Ragasiyam        Kiruba Shankar   
9   1978             Aval Appadithan          C. Rudhraiya   
10  1978          Aval Oru Adhisayam      B. V. Srinivasan   
11  1978  Aval Oru Pachai Kuzhandhai          S. C. Sekhar   
12  1978          Aval Thandha Uravu                 Durai   
13  1978           Aayiram Jenmangal                 Durai   
14  1978                     Bairavi            M. Bhaskar   
15  1978               Chittu Kuruvi         Devaraj–Mohan   
16  1978     En Kelvikku Enna Bathil           P. Madhavan   
17  1978            Ennai Pol Oruvan         T. R. Ramanna   
18  1978         Ganga Yamuna Kaveri             K. Sornam   
19  1978        General Chakravarthi           D. Yoganand   
20  1978   Ilamai Oonjal Aadukirathu         C. V. Sridhar   
21  1978      Ilaya Rani Rajalakshmi    Madurai Thirumaran   
22  1978       Iraivan Kodutha Varam          A. Bhimsingh   
23  1978               Iravu 12 Mani          S. Rajendran   
24  1978          Ithu Eppadi Irukku       R. Pattabiraman   
25  1978            Ival Oru Seethai        A. Jagannathan   
26  1978            Justice Gopinath           D. Yoganand   
27  1978    Kaatrinile Varum Geetham      S. P. Muthuraman   
28  1978               Kaipidithaval         A. Bhim Singh   
29  1978         Kamakshiyin Karunai          K. Narayanan   
30  1978                 Kannamoochi       R. Pattabiraman   
..   ...                         ...                   ...   
56  1978               Punniya Boomi            K. Vijayan   
57  1978         Radhai Ketra Kannan      R. Krishnamurthy   
58  1978            Rajavuketha Rani               L. Balu   
59  1978           Rudhra Thaandavam            K. Vijayan   
60  1978                 Sadhurangam                 Durai   
61  1978        Sakka Podu Podu Raja      S. P. Muthuraman   
62  1978            Sattam En Kaiyil            T. N. Balu   
63  1978         Shankar Salim Simon           P. Madhavan   
64  1978            Sigappu Rojakkal       P. Bharathiraja   
65  1978         Sonnadhu Nee Thanaa           C. N. Muthu   
66  1978        Shri Kanchi Kamakshi  K. S. Gopalakrishnan   
67  1978                 Taxi Driver          N. S. Maniam   
68  1978        Thai Meethu Sathiyam        R. Thyagarajan   
69  1978               Thanga Rangan  S. R. Dakshinamurthy   
70  1978            Thappu Thalangal        K. Balachander   
71  1978              Thirukkalyanam        K. Chandrabose   
72  1978             Tripura Sundari        K. Chandrabose   
73  1978                     Thyagam            K. Vijayan   
74  1978     Ullathil Kuzhanthaiyadi  K. S. Gopalakrishnan   
75  1978       Unakkum Vaazhvu Varum         G. Srinivasan   
76  1978     Uravugal Endrum Vaazhga            H. S. Venu   
77  1978            Vaazhkai Alaigal          G. C. Sekhar   
78  1978   Vanakkatukuriya Kathaliye  A. C. Tirulokchandar   
79  1978            Vandikaran Magan              Amirtham   
80  1978           Varuvan Vadivelan            K. Shankar   
81  1978       Vatathukkul Chadhuram      S. P. Muthuraman   
82  1978                Vayasu Ponnu            K. Shankar   
83  1978    Vazha Ninaithal Vazhalam         Devaraj–Mohan   
84  1978                 Vazhthungal       C. V. Rajendran   
85  1978            Vetri Thirumagal            H. V. Babu   

                                                 cast genre  \
1                       R. Muthuraman, Lakshmi, Shoba  null   
2   Nagesh, Fatafat Jayalaxmi, V.

In [32]:
getAllMusic()

In [33]:
getAllrating()

https://www.imdb.com/title/tt0273408/
Achchani | 
https://www.imdb.com/title/tt0275205/
Adhirstakaran | 7.5
https://www.imdb.com/title/tt2710230/
Agni Pravesam | 
https://www.imdb.com/title/tt9635540/
Alli Darbar | 6.2
https://www.imdb.com/title/tt0276823/
Anantha Bairavi | 6.8
https://www.imdb.com/title/tt0370280/
Andaman Kadhali | 6.5
https://www.imdb.com/title/tt0233224/
Annapoorani | 
https://www.imdb.com/title/tt0274961/
Athaivida Ragasiyam | 5.2
https://www.imdb.com/title/tt0275205/
Aval Appadithan | 7.5
https://www.imdb.com/title/tt0275205/
Aval Oru Adhisayam | 7.5
https://www.imdb.com/title/tt0276788/
Aval Oru Pachai Kuzhandhai | 
https://www.imdb.com/title/tt0214506/
Aval Thandha Uravu | 
https://www.imdb.com/title/tt0186844/
Aayiram Jenmangal | 5.5
https://www.imdb.com/title/tt0276823/
Bairavi | 6.8
https://www.imdb.com/title/tt0276859/
Chittu Kuruvi | 6.7
https://www.imdb.com/title/tt0364326/
En Kelvikku Enna Bathil | 5.5
https://www.imdb.com/title/tt0333410/
Ennai Pol Oruva

In [34]:
df

year                       title              director  \
1   1978                    Achchani         Devaraj–Mohan   
2   1978               Adhirstakaran              Venunath   
3   1978               Agni Pravesam          K. Narayanan   
4   1978                 Alli Darbar    K. M. Balakrishnan   
5   1978             Anantha Bairavi       Mohan Gandhiram   
6   1978             Andaman Kadhali         V. Srinivasan   
7   1978                 Annapoorani        Krishnan–Panju   
8   1978         Athaivida Ragasiyam        Kiruba Shankar   
9   1978             Aval Appadithan          C. Rudhraiya   
10  1978          Aval Oru Adhisayam      B. V. Srinivasan   
11  1978  Aval Oru Pachai Kuzhandhai          S. C. Sekhar   
12  1978          Aval Thandha Uravu                 Durai   
13  1978           Aayiram Jenmangal                 Durai   
14  1978                     Bairavi            M. Bhaskar   
15  1978               Chittu Kuruvi         Devaraj–Mohan   
16  1978     En Kelvikku Enna Bathil           P. Madhavan   
17  1978            Ennai Pol Oruvan         T. R. Ramanna   
18  1978         Ganga Yamuna Kaveri             K. Sornam   
19  1978        General Chakravarthi           D. Yoganand   
20  1978   Ilamai Oonjal Aadukirathu         C. V. Sridhar   
21  1978      Ilaya Rani Rajalakshmi    Madurai Thirumaran   
22  1978       Iraivan Kodutha Varam          A. Bhimsingh   
23  1978               Iravu 12 Mani          S. Rajendran   
24  1978          Ithu Eppadi Irukku       R. Pattabiraman   
25  1978            Ival Oru Seethai        A. Jagannathan   
26  1978            Justice Gopinath           D. Yoganand   
27  1978    Kaatrinile Varum Geetham      S. P. Muthuraman   
28  1978               Kaipidithaval         A. Bhim Singh   
29  1978         Kamakshiyin Karunai          K. Narayanan   
30  1978                 Kannamoochi       R. Pattabiraman   
..   ...                         ...                   ...   
56  1978               Punniya Boomi            K. Vijayan   
57  1978         Radhai Ketra Kannan      R. Krishnamurthy   
58  1978            Rajavuketha Rani               L. Balu   
59  1978           Rudhra Thaandavam            K. Vijayan   
60  1978                 Sadhurangam                 Durai   
61  1978        Sakka Podu Podu Raja      S. P. Muthuraman   
62  1978            Sattam En Kaiyil            T. N. Balu   
63  1978         Shankar Salim Simon           P. Madhavan   
64  1978            Sigappu Rojakkal       P. Bharathiraja   
65  1978         Sonnadhu Nee Thanaa           C. N. Muthu   
66  1978        Shri Kanchi Kamakshi  K. S. Gopalakrishnan   
67  1978                 Taxi Driver          N. S. Maniam   
68  1978        Thai Meethu Sathiyam        R. Thyagarajan   
69  1978               Thanga Rangan  S. R. Dakshinamurthy   
70  1978            Thappu Thalangal        K. Balachander   
71  1978              Thirukkalyanam        K. Chandrabose   
72  1978             Tripura Sundari        K. Chandrabose   
73  1978                     Thyagam            K. Vijayan   
74  1978     Ullathil Kuzhanthaiyadi  K. S. Gopalakrishnan   
75  1978       Unakkum Vaazhvu Varum         G. Srinivasan   
76  1978     Uravugal Endrum Vaazhga            H. S. Venu   
77  1978            Vaazhkai Alaigal          G. C. Sekhar   
78  1978   Vanakkatukuriya Kathaliye  A. C. Tirulokchandar   
79  1978            Vandikaran Magan              Amirtham   
80  1978           Varuvan Vadivelan            K. Shankar   
81  1978       Vatathukkul Chadhuram      S. P. Muthuraman   
82  1978                Vayasu Ponnu            K. Shankar   
83  1978    Vazha Ninaithal Vazhalam         Devaraj–Mohan   
84  1978                 Vazhthungal       C. V. Rajendran   
85  1978            Vetri Thirumagal            H. V. Babu   

                                                 cast genre  \
1                       R. Muthuraman, Lakshmi, Shoba  null   
2   Nagesh, Fatafat Jayalaxmi, V.

In [35]:
df.to_csv('1978_movies.csv')

# 1977

In [39]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1977'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1977=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [40]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [41]:
for table in tables_1977:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1977'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!16 Vayathinile|P. Bharathiraja|Kamal Haasan, Sridevi, Rajinikanth|null|Sri Amman Creations|/wiki/16_Vayathinile
2!Aadu Puli Attam|S. P. Muthuraman|Kamal Haasan, Sripriya, Rajinikanth, Sangeetha, Thengai Srinivasan|null|Swarnambika Productions|/wiki/Aadu_Puli_Attam_(1977_film)
3!Aalukkoru Aasai|S. P. Muthuraman|R. Muthuraman, Jayachitra, Thengai Srinivasan|null|Cine Enterprises|/w/index.php?title=Aalukkoru_Aasai_(1977_film)&action=edit&redlink=1
4!Aarupushpangal|K. M. Balakrishnan|Vijayakumar, Srividya, Rajinikanth|null|Ashtalakshmi Pictures|/wiki/Aarupushpangal
5!Aasai Manaivi|K. Sornam|Jaishankar, Sujatha, Bhavani|null|Suryalaya Productions|/wiki/Aasai_Manaivi
6!Aattukara Alamelu|R. Thyagarajan|Sivakumar, Sripriya, Jai Ganesh|null|Dhandayudhapani Films|/wiki/Aattukara_Alamelu
7!Agraharathil Kazhutai|John Abraham|M. B. Sreenivasan, Swathi, Srilalitha|null|Nirmiti Films|/wiki/Agraharathil_Kazhutai
8!Andru Sinthiya Ratham|R. Sundaram

62!Unnai Suttrum Ulagam|G. Subramaniam|Jayalalithaa, Savitri, Vijayakumar, Vidhubala, Prameela|null|Sri Navaneetha Films|/wiki/Unnai_Suttrum_Ulagam
63!Uyarndhavargal|T. N. Balu|Kamal Haasan, Sujatha, Thengai Srinivasan|null|Rasaleela Pictures|/wiki/Uyarndhavargal


In [42]:
df=df.drop(0)
df

year                        title                    director  \
1   1977               16 Vayathinile             P. Bharathiraja   
2   1977              Aadu Puli Attam            S. P. Muthuraman   
3   1977              Aalukkoru Aasai            S. P. Muthuraman   
4   1977               Aarupushpangal          K. M. Balakrishnan   
5   1977                Aasai Manaivi                   K. Sornam   
6   1977            Aattukara Alamelu              R. Thyagarajan   
7   1977        Agraharathil Kazhutai                John Abraham   
8   1977        Andru Sinthiya Ratham                 R. Sundaram   
9   1977              Annan Oru Koyil                  K. Vijayan   
10  1977          Avan Oru Sarithiram           K. S. Prakash Rao   
11  1977          Avar Enakke Sontham                       Pattu   
12  1977                      Avargal              K. Balachander   
13  1977                 Balaparichai               V. Srinivasan   
14  1977       Bhuvana Oru Kelvi Kuri            S. P. Muthuraman   
15  1977                 Chakravarthy              Krishnan–Panju   
16  1977          Chollu Kanna Chollu  T. N. Thirumalaisamy Nadar   
17  1977           Deviyin Thirumanam                 P. Madhavan   
18  1977                      Dheepam                  K. Vijayan   
19  1977                   Durga Devi                Ra. Sankaran   
20  1977                  Ellam Avale                    Amirtham   
21  1977          Enna Thavam Seithen              Krishnan–Panju   
22  1977       Etharkkum Thuninthavan                   M. Karnan   
23  1977            Gaslight Mangamma                N. S. Maniam   
24  1977                    Gaayathri             R. Pattabiraman   
25  1977            Ilaya Thalaimurai              Krishnan–Panju   
26  1977     Indru Pol Endrum Vaazhga                  K. Shankar   
27  1977               Kalamadi Kalam                        Gopu   
28  1977                   Kavikkuyil               Devaraj–Mohan   
29  1977               Madhurageetham            V. C. Guhanathan   
30  1977               Maamiyar Veedu            N. Krishnamurthy   
..   ...                          ...                         ...   
34  1977               Murugan Adimai              R. Thyagarajan   
35  1977           Naam Pirandha Mann                  A. Vincent   
36  1977       Nallathukku Kalamillai                  T. N. Balu   
37  1977               Nandha En Nila              A. Jagannathan   
38  1977                 Navarathinam             A. P. Nagarajan   
39  1977             Nee Vazha Vendum                A. Bhimsingh   
40  1977          Odi Vilayadu Thatha                  T. N. Balu   
41  1977      Olimayamana Ethirkaalam            K. Krishnamurthy   
42  1977            Oruvanukku Oruthi                Ra. Sankaran   
43  1977                Palabishegham        K. S. Gopalakrishnan   
44  1977             Pattina Pravesam              K. Balachander   
45  1977                   Pen Jenmam        A. C. Tirulokchandar   
46  1977     Pennai Solli Kutramillai            S. P. Muthuraman   
47  1977            Perumaikkuriyaval                Ra. Sankaran   
48  1977           Punidha Anthoniyar                Nanjil Durai   
49  1977           Punniyam Seithaval        K. S. Gopalakrishnan   
50  1977  Raghupathi Raghavan Rajaram                       Durai   
51  1977              Rasi Nalla Rasi                        Gopu   
52  1977               Rowdy Rakkamma        K. S. Gopalakrishnan   
53  1977       Sainthadamma Sainthadu               Devaraj–Mohan   
54  1977        Sonthamadi Nee Enakku            S. P. Muthuraman   
55  1977             Sonnathai Seiven              Krishnan–Panju   
56  1977               Sorgam Naragam              R. Thyagarajan   
57  1977            Sri Krishna Leela             A. P. Nagarajan   
58  1977           Thaliya Salangaiya               T. R. Ramanna   
59  1977              Thanikudithanam                S. A. Kannan   
60

In [43]:
getAllMusic()

ex.php?title=Munnooru_Naal&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Munnooru_Naal&action=edit&r...


Exception
33 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [44]:
getAllrating()

https://www.imdb.com/title/tt0140448/
16 Vayathinile | 8.0
https://www.imdb.com/title/tt0402738/
Aadu Puli Attam | 6.6
https://www.imdb.com/title/tt0273423/
Aalukkoru Aasai | 5.8
https://www.imdb.com/title/tt0353150/
Aarupushpangal | 5.3
https://www.imdb.com/title/tt0273423/
Aasai Manaivi | 5.8
https://www.imdb.com/title/tt8907712/
Aattukara Alamelu | 5.2
https://www.imdb.com/title/tt0077131/
Agraharathil Kazhutai | 7.0
https://www.imdb.com/title/tt0233424/
Andru Sinthiya Ratham | 
https://www.imdb.com/title/tt1435458/
Annan Oru Koyil | 8.3
https://www.imdb.com/title/tt0254152/
Avan Oru Sarithiram | 
https://www.imdb.com/title/tt0273446/
Avar Enakke Sontham | 
https://www.imdb.com/title/tt0154181/
Avargal | 6.9
https://www.imdb.com/search/title/?explore=languages&title_type=feature&year=1977,1977&sort=user_rating,desc&languages=ta&ref_=adv_explore_rhs
Balaparichai | 
https://www.imdb.com/title/tt0273465/
Bhuvana Oru Kelvi Kuri | 7.2
https://www.imdb.com/title/tt0233424/
Chakravarthy | 

In [45]:
df

year                        title                    director  \
1   1977               16 Vayathinile             P. Bharathiraja   
2   1977              Aadu Puli Attam            S. P. Muthuraman   
3   1977              Aalukkoru Aasai            S. P. Muthuraman   
4   1977               Aarupushpangal          K. M. Balakrishnan   
5   1977                Aasai Manaivi                   K. Sornam   
6   1977            Aattukara Alamelu              R. Thyagarajan   
7   1977        Agraharathil Kazhutai                John Abraham   
8   1977        Andru Sinthiya Ratham                 R. Sundaram   
9   1977              Annan Oru Koyil                  K. Vijayan   
10  1977          Avan Oru Sarithiram           K. S. Prakash Rao   
11  1977          Avar Enakke Sontham                       Pattu   
12  1977                      Avargal              K. Balachander   
13  1977                 Balaparichai               V. Srinivasan   
14  1977       Bhuvana Oru Kelvi Kuri            S. P. Muthuraman   
15  1977                 Chakravarthy              Krishnan–Panju   
16  1977          Chollu Kanna Chollu  T. N. Thirumalaisamy Nadar   
17  1977           Deviyin Thirumanam                 P. Madhavan   
18  1977                      Dheepam                  K. Vijayan   
19  1977                   Durga Devi                Ra. Sankaran   
20  1977                  Ellam Avale                    Amirtham   
21  1977          Enna Thavam Seithen              Krishnan–Panju   
22  1977       Etharkkum Thuninthavan                   M. Karnan   
23  1977            Gaslight Mangamma                N. S. Maniam   
24  1977                    Gaayathri             R. Pattabiraman   
25  1977            Ilaya Thalaimurai              Krishnan–Panju   
26  1977     Indru Pol Endrum Vaazhga                  K. Shankar   
27  1977               Kalamadi Kalam                        Gopu   
28  1977                   Kavikkuyil               Devaraj–Mohan   
29  1977               Madhurageetham            V. C. Guhanathan   
30  1977               Maamiyar Veedu            N. Krishnamurthy   
..   ...                          ...                         ...   
34  1977               Murugan Adimai              R. Thyagarajan   
35  1977           Naam Pirandha Mann                  A. Vincent   
36  1977       Nallathukku Kalamillai                  T. N. Balu   
37  1977               Nandha En Nila              A. Jagannathan   
38  1977                 Navarathinam             A. P. Nagarajan   
39  1977             Nee Vazha Vendum                A. Bhimsingh   
40  1977          Odi Vilayadu Thatha                  T. N. Balu   
41  1977      Olimayamana Ethirkaalam            K. Krishnamurthy   
42  1977            Oruvanukku Oruthi                Ra. Sankaran   
43  1977                Palabishegham        K. S. Gopalakrishnan   
44  1977             Pattina Pravesam              K. Balachander   
45  1977                   Pen Jenmam        A. C. Tirulokchandar   
46  1977     Pennai Solli Kutramillai            S. P. Muthuraman   
47  1977            Perumaikkuriyaval                Ra. Sankaran   
48  1977           Punidha Anthoniyar                Nanjil Durai   
49  1977           Punniyam Seithaval        K. S. Gopalakrishnan   
50  1977  Raghupathi Raghavan Rajaram                       Durai   
51  1977              Rasi Nalla Rasi                        Gopu   
52  1977               Rowdy Rakkamma        K. S. Gopalakrishnan   
53  1977       Sainthadamma Sainthadu               Devaraj–Mohan   
54  1977        Sonthamadi Nee Enakku            S. P. Muthuraman   
55  1977             Sonnathai Seiven              Krishnan–Panju   
56  1977               Sorgam Naragam              R. Thyagarajan   
57  1977            Sri Krishna Leela             A. P. Nagarajan   
58  1977           Thaliya Salangaiya               T. R. Ramanna   
59  1977              Thanikudithanam                S. A. Kannan   
60

In [46]:
df.to_csv('1977_movies.csv')

# 1976

In [50]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1976'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1976=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [51]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [52]:
for table in tables_1976:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1976'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Akka|Madurai Thirumaran|K. R. Vijaya, Vijayakumar, Vijayabala, Jai Ganesh|null|Mathura Films|/wiki/Akka_(film)
2!Annakili|Devaraj–Mohan|Sivakumar, Sujatha, Srikanth, Fatafat Jayalaxmi, Thengai Srinivasan|null|S. P. T. Films|/wiki/Annakili
3!Asai 60 Naal|Durai|Vijayakumar, Srividya, Y. Vijaya|null|Sri Uma Chitra Productions|/w/index.php?title=Asai_60_Naal&action=edit&redlink=1
4!Athirshtam Azhaikirathu|A. Jagannathan|Srividya, Thengai Srinivasan, Jayasudha, Major Sundarrajan, Pandari Bai|null|J. M. Productions|/w/index.php?title=Athirshtam_Azhaikirathu&action=edit&redlink=1
5!Bhadrakali|A. C. Tirulokchandar|Sivakumar, Rani Chandra, Major Sundarrajan, Bhavani, Thengai Srinivasan, Manorama|null|Cine Bharath Productions|/wiki/Bhadrakali_(film)
6!Chitra Pournami|P. Madhavan|Sivaji Ganesan, Jayalalithaa, R. Muthuraman, C. R. Vijayakumari, Jayakumari, Nagesh|null|Sri Bhuvaneswari Movies|/wiki/Chitra_Pournami_(film)
7!Dasavatharam|K. S. Go

In [53]:
df=df.drop(0)
df

year                             title                director  \
1   1976                              Akka      Madurai Thirumaran   
2   1976                          Annakili           Devaraj–Mohan   
3   1976                      Asai 60 Naal                   Durai   
4   1976           Athirshtam Azhaikirathu          A. Jagannathan   
5   1976                        Bhadrakali    A. C. Tirulokchandar   
6   1976                   Chitra Pournami             P. Madhavan   
7   1976                      Dasavatharam    K. S. Gopalakrishnan   
8   1976                     Gruhapravesam             D. Yoganand   
9   1976                      Idhaya Malar          Gemini Ganesan   
10  1976                 Inspector Manaivi        S. Rajendra Babu   
11  1976             Ithu Ivargalin Kathai       T. P. Arunachalam   
12  1976                   Janaki Sabadham           Avinashi Mani   
13  1976        Kaalangalil Aval Vasantham        S. P. Muthuraman   
14  1976                   Kanavan Manaivi            A. Bhimsingh   
15  1976                     Kula Gouravam          Peketi Sivaram   
16  1976                   Kumaara Vijayam          A. Jagannathan   
17  1976                           Lalitha    Valampuri Somanathan   
18  1976                  Madhana Maaligai              K. Vijayan   
19  1976                   Maharasi Vazhga             Balamurugan   
20  1976             Manamaara Vazhthungal          G. Subramaniam   
21  1976                   Manmadha Leelai          K. Balachander   
22  1976                   Mayor Meenakshi      Madurai Thirumaran   
23  1976                      Mittai Mummy           Avinashi Mani   
24  1976           Mogam Muppadhu Varusham        S. P. Muthuraman   
25  1976                   Moondru Mudichu          K. Balachander   
26  1976              Mutthana Mutthullavo               R. Vittal   
27  1976                    Nalla Pennmani          A. Jagannathan   
28  1976              Nee Indri Naan Illai    A. C. Tirulokchandar   
29  1976                  Nee Oru Maharani               K. Sornam   
30  1976           Needhikku Thalaivanangu          P. Neelakantan   
31  1976             Ninaipadhu Niraiverum        S. T. Dhandapani   
32  1976                          Oh Manju           C. V. Sridhar   
33  1976               Oorukku Uzhaippavan             M. Krishnan   
34  1976                      Ore Thanthai               M. Karnan   
35  1976          Oru Kodiyil Iru Malargal        S. P. Muthuraman   
36  1976   Oru Oodhappu Kan Simittugiradhu        S. P. Muthuraman   
37  1976           Paalootti Valartha Kili           Devaraj–Mohan   
38  1976                    Panakkara Penn            U. Rajendran   
39  1976                           Payanam  Vietnam Veedu Sundaram   
40  1976                    Perum Pugazhum           V. Srinivasan   
41  1976                      Rojavin Raja              K. Vijayan   
42  1976                         Santhathi               Kaushikan   
43  1976                            Satyam            S. A. Kannan   
44  1976                       Seervarisai               K. Sornam   
45  1976  Sila Nerangalil Sila Manithargal            A. Bhimsingh   
46  1976               Thayilla Kuzhandhai          R. Thyagarajan   
47  1976                    Thunive Thunai        S. P. Muthuraman   
48  1976                     Unakkaga Naan         C. V. Rajendran   
49  1976                        Unarchigal            R. C. Sakthi   
50  1976                    Ungalil Oruthi           Devaraj–Mohan   
51  1976            Unmaiye Un Vilai Enna?                     Cho   
52  1976                   Uravadum Nenjam           Devaraj–Mohan   
53  1976                           Uthaman   V. B. Rajendra Prasad   
54  1976                Uzhaikkum Karangal              K. Shankar   
55  1976             Vanga Sambandhi Vanga           P. Bhanumathi   
56  1976                     Varaprasadham            K. Narayanan   
57  1976      

In [54]:
getAllMusic()

In [55]:
getAllrating()

https://www.imdb.com/title/tt0074105/
Akka | 
https://www.imdb.com/title/tt0273429/
Annakili | 7.0
https://www.imdb.com/title/tt10189592/
Asai 60 Naal | 
https://www.imdb.com/title/tt0355227/
Athirshtam Azhaikirathu | 
https://www.imdb.com/title/tt0273450/
Bhadrakali | 6.1
https://www.imdb.com/title/tt1437593/
Chitra Pournami | 6.1
https://www.imdb.com/title/tt0215696/
Dasavatharam | 6.4
https://www.imdb.com/title/tt1440181/
Gruhapravesam | 
https://www.imdb.com/title/tt0214785/
Idhaya Malar | 6.3
https://www.imdb.com/title/tt0156682/
Inspector Manaivi | 6.0
https://www.imdb.com/title/tt0355227/
Ithu Ivargalin Kathai | 
https://www.imdb.com/title/tt3415248/
Janaki Sabadham | 
Kaalangalil Aval Vasantham | 
https://www.imdb.com/title/tt0156682/
Kanavan Manaivi | 6.0
https://www.imdb.com/title/tt0369447/
Kula Gouravam | 7.2
https://www.imdb.com/title/tt0362813/
Kumaara Vijayam | 6.6
https://www.imdb.com/title/tt0371783/
Lalitha | 6.7
https://www.imdb.com/title/tt3415084/
Madhana Maaligai 

In [56]:
df

year                             title                director  \
1   1976                              Akka      Madurai Thirumaran   
2   1976                          Annakili           Devaraj–Mohan   
3   1976                      Asai 60 Naal                   Durai   
4   1976           Athirshtam Azhaikirathu          A. Jagannathan   
5   1976                        Bhadrakali    A. C. Tirulokchandar   
6   1976                   Chitra Pournami             P. Madhavan   
7   1976                      Dasavatharam    K. S. Gopalakrishnan   
8   1976                     Gruhapravesam             D. Yoganand   
9   1976                      Idhaya Malar          Gemini Ganesan   
10  1976                 Inspector Manaivi        S. Rajendra Babu   
11  1976             Ithu Ivargalin Kathai       T. P. Arunachalam   
12  1976                   Janaki Sabadham           Avinashi Mani   
13  1976        Kaalangalil Aval Vasantham        S. P. Muthuraman   
14  1976                   Kanavan Manaivi            A. Bhimsingh   
15  1976                     Kula Gouravam          Peketi Sivaram   
16  1976                   Kumaara Vijayam          A. Jagannathan   
17  1976                           Lalitha    Valampuri Somanathan   
18  1976                  Madhana Maaligai              K. Vijayan   
19  1976                   Maharasi Vazhga             Balamurugan   
20  1976             Manamaara Vazhthungal          G. Subramaniam   
21  1976                   Manmadha Leelai          K. Balachander   
22  1976                   Mayor Meenakshi      Madurai Thirumaran   
23  1976                      Mittai Mummy           Avinashi Mani   
24  1976           Mogam Muppadhu Varusham        S. P. Muthuraman   
25  1976                   Moondru Mudichu          K. Balachander   
26  1976              Mutthana Mutthullavo               R. Vittal   
27  1976                    Nalla Pennmani          A. Jagannathan   
28  1976              Nee Indri Naan Illai    A. C. Tirulokchandar   
29  1976                  Nee Oru Maharani               K. Sornam   
30  1976           Needhikku Thalaivanangu          P. Neelakantan   
31  1976             Ninaipadhu Niraiverum        S. T. Dhandapani   
32  1976                          Oh Manju           C. V. Sridhar   
33  1976               Oorukku Uzhaippavan             M. Krishnan   
34  1976                      Ore Thanthai               M. Karnan   
35  1976          Oru Kodiyil Iru Malargal        S. P. Muthuraman   
36  1976   Oru Oodhappu Kan Simittugiradhu        S. P. Muthuraman   
37  1976           Paalootti Valartha Kili           Devaraj–Mohan   
38  1976                    Panakkara Penn            U. Rajendran   
39  1976                           Payanam  Vietnam Veedu Sundaram   
40  1976                    Perum Pugazhum           V. Srinivasan   
41  1976                      Rojavin Raja              K. Vijayan   
42  1976                         Santhathi               Kaushikan   
43  1976                            Satyam            S. A. Kannan   
44  1976                       Seervarisai               K. Sornam   
45  1976  Sila Nerangalil Sila Manithargal            A. Bhimsingh   
46  1976               Thayilla Kuzhandhai          R. Thyagarajan   
47  1976                    Thunive Thunai        S. P. Muthuraman   
48  1976                     Unakkaga Naan         C. V. Rajendran   
49  1976                        Unarchigal            R. C. Sakthi   
50  1976                    Ungalil Oruthi           Devaraj–Mohan   
51  1976            Unmaiye Un Vilai Enna?                     Cho   
52  1976                   Uravadum Nenjam           Devaraj–Mohan   
53  1976                           Uthaman   V. B. Rajendra Prasad   
54  1976                Uzhaikkum Karangal              K. Shankar   
55  1976             Vanga Sambandhi Vanga           P. Bhanumathi   
56  1976                     Varaprasadham            K. Narayanan   
57  1976      

In [57]:
df.to_csv('1976_movies.csv')

# 1975

In [61]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1975'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1975=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [62]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [63]:
for table in tables_1975:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1975'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Aan Pillai Singam|S. P. Muthuraman|Sivakumar, R. Muthuraman, Sujatha, Sripriya, Vijayakumar, Cho|null|Arun Chandra Movies|/wiki/Aan_Pillai_Singam
2!Aayirathil Oruthi|Avinashi Mani|K. R. Vijaya, K. Balaji, Srikanth, Kamal Haasan, Sujatha, Jayasudha, Thengai Srinivasan, Manorama|null|K. R. G. Pictures|/wiki/Aayirathil_Oruthi
3!Amudha|K. Venkatraman|Ravichandran, R. Muthuraman, C. R. Vijayakumari, Rajasree, Nagesh|null|Sri Poongavanathal Pictures|/w/index.php?title=Amudha_(film)&action=edit&redlink=1
4!Anaiya Vilakku|Krishnan–Panju|M. K. Muthu, Vennira Aadai Nirmala, Lakshmi, Srikanth, Padmapriya|null|Anjugam Pictures|/w/index.php?title=Anaiya_Vilakku&action=edit&redlink=1
5!Anbe Aaruyire|A. C. Tirulokchandar|Sivaji Ganesan, Manjula, Nagesh, K. A. Thangavelu, Manorama, Suruli Rajan|null|Amutham Pictures|/wiki/Anbe_Aaruyire_(1975_film)
6!Anbu Roja|Devaraj–Mohan|R. Muthuraman, Latha, Thengai Srinivasan|null|Sulochana Pictures|/w/index.p

In [64]:
df=df.drop(0)
df

year                          title               director  \
1   1975              Aan Pillai Singam       S. P. Muthuraman   
2   1975              Aayirathil Oruthi          Avinashi Mani   
3   1975                         Amudha         K. Venkatraman   
4   1975                 Anaiya Vilakku         Krishnan–Panju   
5   1975                  Anbe Aaruyire   A. C. Tirulokchandar   
6   1975                      Anbu Roja          Devaraj–Mohan   
7   1975                    Andharangam          V. Srinivasan   
8   1975              Apoorva Raagangal         K. Balachander   
9   1975        Avalukku Aayiram Kangal          T. R. Ramanna   
10  1975              Avandhan Manidhan   A. C. Tirulokchandar   
11  1975               Cinema Paithiyam          V. Srinivasan   
12  1975                       Dr. Siva   A. C. Tirulokchandar   
13  1975             Eduppar Kai Pillai             K. Vijayan   
14  1975              Ellorum Nallavare            S. S. Balan   
15  1975        Enakkoru Magan Pirappan            M. Krishnan   
16  1975              Enga Pattan Sothu              M. Karnan   
17  1975        Engalukkum Kadhal Varum              R. Vittal   
18  1975         Ezhaikkum Kaalam Varum       S. Rajendra Babu   
19  1975                   Hotel Sorgam        G. Ramakrishnan   
20  1975                    Idhayakkani         A. Jagannathan   
21  1975            Ingeyum Manidhargal        A. L. Narayanan   
22  1975             Ippadiyum Oru Penn          P. Bhanumathi   
23  1975  Kadhavai Thattiya Mohini Paey                Rajaram   
24  1975                 Karotti Kannan               R. Pattu   
25  1975               Kasthuri Vijayam            P. Madhavan   
26  1975               Maalai Sooda Vaa        C. V. Rajendran   
27  1975                Mamiyar Vijayam     Madurai Thirumaran   
28  1975       Manidhanum Dheivamagalam            P. Madhavan   
29  1975           Manjal Mugame Varuga       V. C. Guhanathan   
30  1975           Mannavan Vanthaanadi            P. Madhavan   
..   ...                            ...                    ...   
32  1975            Melnaattu Marumagal        A. P. Nagarajan   
33  1975                 Naalai Namadhe    K. S. Sethumadhavan   
34  1975                Nalla Marumagal             A. Vincent   
35  1975         Nambikkai Natchathiram  Thirumalai-Mahalingam   
36  1975        Ninaithadhai Mudippavan         P. Neelakantan   
37  1975         Oru Kudumbathin Kadhai                  Durai   
38  1975                Pallandu Vazhga             K. Shankar   
39  1975             Panam Pathum Seyum         G. Subramaniam   
40  1975                   Pattampoochi         A. S. Prakasam   
41  1975              Pattikkaattu Raja           K. Shanmugam   
42  1975            Paattum Bharathamum            P. Madhavan   
43  1975                    Pinju Manam      A. K. Subramaniam   
44  1975                   Piriya Vidai               Srikanth   
45  1975                   Puthu Vellam             K. Vijayan   
46  1975              Sondhangal Vazhga     Madurai Thirumaran   
47  1975                 Swami Ayyappan         P. Subramaniam   
48  1975           Thai Veetu Seedhanam     Madurai Thirumaran   
49  1975             Thangathile Vairam              K. Sornam   
50  1975          Then Sindhudhe Vaanam           Ra. Sankaran   
51  1975                 Thennangkeetru      K. V. Manisekaran   
52  1975          Thirudanukku Thirudan       T. M. Thirumalai   
53  1975                     Thiruvarul         R. Thyagarajan   
54  1975        Thottadhellam Ponnaagum              R. Vittal   
55  1975           Unga Veettu Kalyanam       K. Krishnamurthy   
56  1975             Uravu Solla Oruvan          Devaraj–Mohan   
57  1975          Uravukku Kai Koduppom   K. S. Gopalakrishnan   
58  1975                   Vaira Nenjam          C. V. Sridhar   
59  1975          Vaazhnthu Kaattugiren         Krishnan–Panju   
60  1975        Yarukku Maappillai Yaro     

In [65]:
getAllMusic()

ex.php?title=Nalla_Marumagal&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Nalla_Marumagal&action=edit...
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Thirudanukku_Thirudan&actio...


Exception
34 | 
ex.php?title=Thirudanukku_Thirudan&action=edit&redlink=1
Exception
52 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [66]:
getAllrating()

https://www.imdb.com/title/tt8903504/
Aan Pillai Singam | 
https://www.imdb.com/title/tt5760166/
Aayirathil Oruthi | 6.3
https://www.imdb.com/title/tt2710258/
Amudha | 
https://www.imdb.com/title/tt0233212/
Anaiya Vilakku | 
https://www.imdb.com/title/tt1435452/
Anbe Aaruyire | 6.0
https://www.imdb.com/title/tt3417484/
Anbu Roja | 
https://www.imdb.com/title/tt0362413/
Andharangam | 5.8
https://www.imdb.com/title/tt0154159/
Apoorva Raagangal | 7.5
https://www.imdb.com/title/tt3451298/
Avalukku Aayiram Kangal | 
https://www.imdb.com/title/tt1435463/
Avandhan Manidhan | 7.2
https://www.imdb.com/title/tt1492769/
Cinema Paithiyam | 6.4
https://www.imdb.com/title/tt0370470/
Dr. Siva | 5.7
https://www.imdb.com/title/tt5760178/
Eduppar Kai Pillai | 
https://www.imdb.com/title/tt3417494/
Ellorum Nallavare | 
https://www.imdb.com/title/tt8947996/
Enakkoru Magan Pirappan | 4.8
https://www.imdb.com/title/tt8948058/
Enga Pattan Sothu | 
https://www.imdb.com/title/tt3451298/
Engalukkum Kadhal Varum

In [67]:
df

year                          title               director  \
1   1975              Aan Pillai Singam       S. P. Muthuraman   
2   1975              Aayirathil Oruthi          Avinashi Mani   
3   1975                         Amudha         K. Venkatraman   
4   1975                 Anaiya Vilakku         Krishnan–Panju   
5   1975                  Anbe Aaruyire   A. C. Tirulokchandar   
6   1975                      Anbu Roja          Devaraj–Mohan   
7   1975                    Andharangam          V. Srinivasan   
8   1975              Apoorva Raagangal         K. Balachander   
9   1975        Avalukku Aayiram Kangal          T. R. Ramanna   
10  1975              Avandhan Manidhan   A. C. Tirulokchandar   
11  1975               Cinema Paithiyam          V. Srinivasan   
12  1975                       Dr. Siva   A. C. Tirulokchandar   
13  1975             Eduppar Kai Pillai             K. Vijayan   
14  1975              Ellorum Nallavare            S. S. Balan   
15  1975        Enakkoru Magan Pirappan            M. Krishnan   
16  1975              Enga Pattan Sothu              M. Karnan   
17  1975        Engalukkum Kadhal Varum              R. Vittal   
18  1975         Ezhaikkum Kaalam Varum       S. Rajendra Babu   
19  1975                   Hotel Sorgam        G. Ramakrishnan   
20  1975                    Idhayakkani         A. Jagannathan   
21  1975            Ingeyum Manidhargal        A. L. Narayanan   
22  1975             Ippadiyum Oru Penn          P. Bhanumathi   
23  1975  Kadhavai Thattiya Mohini Paey                Rajaram   
24  1975                 Karotti Kannan               R. Pattu   
25  1975               Kasthuri Vijayam            P. Madhavan   
26  1975               Maalai Sooda Vaa        C. V. Rajendran   
27  1975                Mamiyar Vijayam     Madurai Thirumaran   
28  1975       Manidhanum Dheivamagalam            P. Madhavan   
29  1975           Manjal Mugame Varuga       V. C. Guhanathan   
30  1975           Mannavan Vanthaanadi            P. Madhavan   
..   ...                            ...                    ...   
32  1975            Melnaattu Marumagal        A. P. Nagarajan   
33  1975                 Naalai Namadhe    K. S. Sethumadhavan   
34  1975                Nalla Marumagal             A. Vincent   
35  1975         Nambikkai Natchathiram  Thirumalai-Mahalingam   
36  1975        Ninaithadhai Mudippavan         P. Neelakantan   
37  1975         Oru Kudumbathin Kadhai                  Durai   
38  1975                Pallandu Vazhga             K. Shankar   
39  1975             Panam Pathum Seyum         G. Subramaniam   
40  1975                   Pattampoochi         A. S. Prakasam   
41  1975              Pattikkaattu Raja           K. Shanmugam   
42  1975            Paattum Bharathamum            P. Madhavan   
43  1975                    Pinju Manam      A. K. Subramaniam   
44  1975                   Piriya Vidai               Srikanth   
45  1975                   Puthu Vellam             K. Vijayan   
46  1975              Sondhangal Vazhga     Madurai Thirumaran   
47  1975                 Swami Ayyappan         P. Subramaniam   
48  1975           Thai Veetu Seedhanam     Madurai Thirumaran   
49  1975             Thangathile Vairam              K. Sornam   
50  1975          Then Sindhudhe Vaanam           Ra. Sankaran   
51  1975                 Thennangkeetru      K. V. Manisekaran   
52  1975          Thirudanukku Thirudan       T. M. Thirumalai   
53  1975                     Thiruvarul         R. Thyagarajan   
54  1975        Thottadhellam Ponnaagum              R. Vittal   
55  1975           Unga Veettu Kalyanam       K. Krishnamurthy   
56  1975             Uravu Solla Oruvan          Devaraj–Mohan   
57  1975          Uravukku Kai Koduppom   K. S. Gopalakrishnan   
58  1975                   Vaira Nenjam          C. V. Sridhar   
59  1975          Vaazhnthu Kaattugiren         Krishnan–Panju   
60  1975        Yarukku Maappillai Yaro     

In [68]:
df.to_csv('1975_movies.csv')

# 1974

In [73]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1974'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1974=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [74]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [75]:
for table in tables_1974:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1974'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Akkarai Pachai|N. Venkatesh|Jaishankar, Ravichandran, Nagesh, Lakshmi, Jayachitra, Savitri|null|Yoga Chithra Productions|/w/index.php?title=Akkarai_Pachai&action=edit&redlink=1
2!Anbai Thedi|V. Srinivasan|Sivaji Ganesan, Jayalalithaa, Major Sundarrajan, C. R. Vijayakumari, Shubha, Cho, Manorama|null|Muktha Films|/wiki/Anbai_Thedi
3!Anbu Thangai|S. P. Muthuraman|R. Muthuraman, Jayalalithaa, Jaya, Srikanth, K. A. Thangavelu|null|Gowri Art Enterprises|/wiki/Anbu_Thangai
4!Appa Amma|Maa. Raa.|Gemini Ganesan, Padmini, Ravichandran, Sheela|null|K. S. SithambaramSimham Productions|/w/index.php?title=Appa_Amma&action=edit&redlink=1
5!Athaiya Mamiya|Gopu|Jaishankar, Ushanandini, Nagesh, Thengai Srinivasan, Manorama, Srikanth, M. N. Rajam, Kumari Padmini, Venniradai Moorthy, Sachu|null|Garuda Films|/wiki/Athaiya_Mamiya
6!Aval Oru Thodar Kathai|K. Balachander|Sujatha, Kamal Haasan, Vijayakumar, Fatafat Jayalaxmi, Jai Ganesh, M. G. Soman, Srip

In [76]:
df=df.drop(0)
df

year                    title                      director  \
1   1974           Akkarai Pachai                  N. Venkatesh   
2   1974              Anbai Thedi                 V. Srinivasan   
3   1974             Anbu Thangai              S. P. Muthuraman   
4   1974                Appa Amma                     Maa. Raa.   
5   1974           Athaiya Mamiya                          Gopu   
6   1974   Aval Oru Thodar Kathai                K. Balachander   
7   1974     Avalukku Nigar Avale            Madurai Thirumanam   
8   1974       Avalum Penn Thaane                         Durai   
9   1974  Devi Sri Karumari Amman        Vietnam Veedu Sundaram   
10  1974                  Dhaagam              Babu Nanthancode   
11  1974        Dheerga Sumangali          A. C. Tirulokchandar   
12  1974               Doctoramma                        Sornam   
13  1974        Dikkatra Parvathi      Singeetham Srinivasa Rao   
14  1974                 En Magan               C. V. Rajendran   
15  1974       Engal Kula Dheivam                    P. R. Somu   
16  1974         Engamma Sapatham              S. P. Muthuraman   
17  1974        Gumasthavin Magal               A. P. Nagarajan   
18  1974       Idhayam Parkiradhu                A. Jagannathan   
19  1974             Kadavul Mama                 K. Singamuthu   
20  1974         Kai Niraya Kaasu                     A. B. Raj   
21  1974          Kaliyuga Kannan                Krishnan–Panju   
22  1974      Kalyanamam Kalyanam              K. Krishnamurthy   
23  1974             Kanmani Raja                 Devaraj–Mohan   
24  1974              Magalukkaga                   M. Krishnan   
25  1974         Maanikka Thottil                   P. Madhavan   
26  1974   Murugan Kaattiya Vazhi                   P. Madhavan   
27  1974           Naan Avanillai                K. Balachander   
28  1974       Netru Indru Naalai                P. Neelakantan   
29  1974    Onne Onnu Kanne Kannu                  Ra. Sankaran   
30  1974               Ore Satchi                    K. Vijayan   
..   ...                      ...                           ...   
33  1974               Pandhattam                Ma. Lakshmanan   
34  1974            Paruva Kaalam            Jos A. N. Fernando   
35  1974      Pathu Madha Bandham                Krishnan–Panju   
36  1974        Penn Ondru Kanden                          Gopu   
37  1974            Pillai Selvam            V. Ramachandra Rao   
38  1974            Prayachittham                   R. Sundaram   
39  1974         Pudhiya Manidhan         Thirumalai-Mahalingam   
40  1974               Raja Nagam                  N. S. Maniam   
41  1974               Roshakkari            Madurai Thirumaran   
42  1974               Samarpanam              Thillai Raghavan   
43  1974             Samayalkaran         Thirumalai-Mahalingam   
44  1974     Sirithu Vazha Vendum                   S. S. Balan   
45  1974                Sisupalan            K. K. Sampathkumar   
46  1974       Sivakamiyin Selvan               C. V. Rajendran   
47  1974     Sorgathil Thirumanam                 T. R. Ramanna   
48  1974       Swathi Nakshathram          K. S. Gopalakrishnan   
49  1974                    Thaai                   D. Yoganand   
50  1974              Thaai Pasam              B. V. Srinivasan   
51  1974           Thai Pirandhal             A. K. Subramaniam   
52  1974         Thanga Pathakkam                   P. Madhavan   
53  1974        Thanga Valayal[3]            M. A. V. Rajendran   
54  1974                  Thirudi            Madurai Thirumaran   
55  1974           Thirumangalyam                    A. Vincent   
56  1974       Tiger Thaathachari                   V. T. Arasu   
57  1974           Ungal Viruppam              K. Krishnamurthy   
58  1974         Unnaithan Thambi                     R. Vittal   
59  1974              Urimaikural                 C. V. Sridhar   
60  1974                   Vairam                 T

In [77]:
getAllMusic()

In [78]:
getAllrating()

https://www.imdb.com/title/tt3418510/
Akkarai Pachai | 
https://www.imdb.com/title/tt1435451/
Anbai Thedi | 6.6
https://www.imdb.com/title/tt3420082/
Anbu Thangai | 7.6
https://www.imdb.com/title/tt3420086/
Appa Amma | 
https://www.imdb.com/title/tt3420100/
Athaiya Mamiya | 5.2
https://www.imdb.com/title/tt2710334/
Aval Oru Thodar Kathai | 7.3
https://www.imdb.com/title/tt0186841/
Avalukku Nigar Avale | 4.3
https://www.imdb.com/title/tt0214508/
Avalum Penn Thaane | 
https://www.imdb.com/title/tt1468307/
Devi Sri Karumari Amman | 4.3
https://www.imdb.com/title/tt0371316/
Dhaagam | 6.3
https://www.imdb.com/title/tt2720488/
Dheerga Sumangali | 
https://www.imdb.com/title/tt3418546/
Doctoramma | 
https://www.imdb.com/title/tt0264537/
Dikkatra Parvathi | 
https://www.imdb.com/title/tt1427040/
En Magan | 6.1
https://www.imdb.com/title/tt2725192/
Engal Kula Dheivam | 
https://www.imdb.com/title/tt3418492/
Engamma Sapatham | 
https://www.imdb.com/title/tt0246664/
Gumasthavin Magal | 6.5
https:

In [79]:
df

year                    title                      director  \
1   1974           Akkarai Pachai                  N. Venkatesh   
2   1974              Anbai Thedi                 V. Srinivasan   
3   1974             Anbu Thangai              S. P. Muthuraman   
4   1974                Appa Amma                     Maa. Raa.   
5   1974           Athaiya Mamiya                          Gopu   
6   1974   Aval Oru Thodar Kathai                K. Balachander   
7   1974     Avalukku Nigar Avale            Madurai Thirumanam   
8   1974       Avalum Penn Thaane                         Durai   
9   1974  Devi Sri Karumari Amman        Vietnam Veedu Sundaram   
10  1974                  Dhaagam              Babu Nanthancode   
11  1974        Dheerga Sumangali          A. C. Tirulokchandar   
12  1974               Doctoramma                        Sornam   
13  1974        Dikkatra Parvathi      Singeetham Srinivasa Rao   
14  1974                 En Magan               C. V. Rajendran   
15  1974       Engal Kula Dheivam                    P. R. Somu   
16  1974         Engamma Sapatham              S. P. Muthuraman   
17  1974        Gumasthavin Magal               A. P. Nagarajan   
18  1974       Idhayam Parkiradhu                A. Jagannathan   
19  1974             Kadavul Mama                 K. Singamuthu   
20  1974         Kai Niraya Kaasu                     A. B. Raj   
21  1974          Kaliyuga Kannan                Krishnan–Panju   
22  1974      Kalyanamam Kalyanam              K. Krishnamurthy   
23  1974             Kanmani Raja                 Devaraj–Mohan   
24  1974              Magalukkaga                   M. Krishnan   
25  1974         Maanikka Thottil                   P. Madhavan   
26  1974   Murugan Kaattiya Vazhi                   P. Madhavan   
27  1974           Naan Avanillai                K. Balachander   
28  1974       Netru Indru Naalai                P. Neelakantan   
29  1974    Onne Onnu Kanne Kannu                  Ra. Sankaran   
30  1974               Ore Satchi                    K. Vijayan   
..   ...                      ...                           ...   
33  1974               Pandhattam                Ma. Lakshmanan   
34  1974            Paruva Kaalam            Jos A. N. Fernando   
35  1974      Pathu Madha Bandham                Krishnan–Panju   
36  1974        Penn Ondru Kanden                          Gopu   
37  1974            Pillai Selvam            V. Ramachandra Rao   
38  1974            Prayachittham                   R. Sundaram   
39  1974         Pudhiya Manidhan         Thirumalai-Mahalingam   
40  1974               Raja Nagam                  N. S. Maniam   
41  1974               Roshakkari            Madurai Thirumaran   
42  1974               Samarpanam              Thillai Raghavan   
43  1974             Samayalkaran         Thirumalai-Mahalingam   
44  1974     Sirithu Vazha Vendum                   S. S. Balan   
45  1974                Sisupalan            K. K. Sampathkumar   
46  1974       Sivakamiyin Selvan               C. V. Rajendran   
47  1974     Sorgathil Thirumanam                 T. R. Ramanna   
48  1974       Swathi Nakshathram          K. S. Gopalakrishnan   
49  1974                    Thaai                   D. Yoganand   
50  1974              Thaai Pasam              B. V. Srinivasan   
51  1974           Thai Pirandhal             A. K. Subramaniam   
52  1974         Thanga Pathakkam                   P. Madhavan   
53  1974        Thanga Valayal[3]            M. A. V. Rajendran   
54  1974                  Thirudi            Madurai Thirumaran   
55  1974           Thirumangalyam                    A. Vincent   
56  1974       Tiger Thaathachari                   V. T. Arasu   
57  1974           Ungal Viruppam              K. Krishnamurthy   
58  1974         Unnaithan Thambi                     R. Vittal   
59  1974              Urimaikural                 C. V. Sridhar   
60  1974                   Vairam                 T

In [80]:
df.to_csv('1974_movies.csv')

# 1973

In [84]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1973'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1973=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [85]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [86]:
for table in tables_1973:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1973'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Alaigal|C. V. Sridhar|Vishnuvardhan, Chandrakala, Cho, Manorama|null|Chithralaya Films|/wiki/Alaigal_(film)
2!Amman Arul|Pattu|Jaishankar, A. V. M. Rajan, Manjula, Thengai Srinivasan, K. A. Thangavelu|null|Ambika Movies|/wiki/Amman_Arul
3!Anbu Sagodharargal|Lakshmi Deepak|Jaishankar, S. V. Ranga Rao, A. V. M. Rajan, Major Sundarrajan, Jamuna, Devika, Vennira Aadai Nirmala, Prameela, Cho, V. K. Ramasamy, Manorama|null|Jayanthi Films|/wiki/Anbu_Sagodharargal
4!Arangetram|K. Balachander|Prameela, Sivakumar, S. V. Subbaiah, M. N. Rajam, Shashikumar, Kamal Haasan, Jayachitra, Jayasudha|null|Kalakendra Films|/wiki/Arangetram_(film)
5!Baghdad Perazhagi|T. R. Ramanna|Jayalalithaa, Ravichandran, Savitri, Major Sundarrajan, Nagesh, Shubha, Jayasudha, Thengai Srinivasan, Sachu|null|Ganesh Creations|/wiki/Baghdad_Perazhagi
6!Bharatha Vilas|A. C. Tirulokchandar|Sivaji Ganesan, K. R. Vijaya, Major Sundarrajan, Devika, V. K. Ramasamy, M. R. R. Va

In [87]:
df=df.drop(0)
df

year                       title                director  \
1   1973                     Alaigal           C. V. Sridhar   
2   1973                  Amman Arul                   Pattu   
3   1973          Anbu Sagodharargal          Lakshmi Deepak   
4   1973                  Arangetram          K. Balachander   
5   1973           Baghdad Perazhagi           T. R. Ramanna   
6   1973              Bharatha Vilas    A. C. Tirulokchandar   
7   1973         Deiva Kuzhandhaigal        S. P. Muthuraman   
8   1973               Dheiva Vamsam              K. Vijayan   
9   1973                 Engal Thaai      Malliyam Rajagopal   
10  1973           Engal Thanga Raja   V. B. Rajendra Prasad   
11  1973                 Ganga Gowri          B. R. Panthulu   
12  1973                    Gauravam  Vietnam Veedu Sundaram   
13  1973         Iraivan Irukkindran              H. S. Venu   
14  1973          Karaikkal Ammaiyar         A. P. Nagarajan   
15  1973               Kasi Yathirai        S. P. Muthuraman   
16  1973            Kattila Thottila      Malliyam Rajagopal   
17  1973        Komatha En Kulamatha        M. A. Thirumugam   
18  1973         Malai Naattu Mangai          P. Subramaniam   
19  1973                Malligai Poo            N. S. Maniam   
20  1973         Manidharil Manikkam         C. V. Rajendran   
21  1973                   Manipayal          A. Jagannathan   
22  1973             Manjal Kungumam                   Pattu   
23  1973                 Maru Piravi           T. R. Ramanna   
24  1973                Nalla Mudivu        C. N. Shanmugham   
25  1973              Nathayil Muthu    K. S. Gopalakrishnan   
26  1973               Nee Ullavarai            C. V. Ramana   
27  1973             Nyayam Ketkirom         C. V. Rajendran   
28  1973                 Pasa Deepam                   Pattu   
29  1973        Pattikaattu Ponnaiya             B. S. Ranga   
30  1973            Pennai Nambungal        B. V. Srinivasan   
31  1973           Petha Manam Pithu        S. P. Muthuraman   
32  1973                   Ponnunjal         C. V. Rajendran   
33  1973      Ponnukku Thanga Manasu           Devaraj–Mohan   
34  1973                    Ponvandu            N. S. Maniam   
35  1973                    Pookkari          Krishnan–Panju   
36  1973                  Prarthanai               Kaushikan   
37  1973                       Radha    A. C. Tirulokchandar   
38  1973             Rajaraja Cholan         A. P. Nagarajan   
39  1973         Rajapart Rangadurai             P. Madhavan   
40  1973               School Master          B. R. Panthulu   
41  1973               Shanmugapriya        K. Krishnamurthy   
42  1973      Sollathaan Ninaikkiren          K. Balachander   
43  1973                     Sondham    A. C. Tirulokchandar   
44  1973                 Suryagandhi           V. Srinivasan   
45  1973             Thalai Prasavam             M. Krishnan   
46  1973        Thedi Vandha Lakshmi            G. R. Nathan   
47  1973           Thirumalai Deivam         A. P. Nagarajan   
48  1973       Ulagam Sutrum Valiban      M. G. Ramachandran   
49  1973                     Vaayadi      Madurai Thirumaran   
50  1973                  Vakkuruthi         Mohan Gandhiram   
51  1973              Valli Deivanai        Thillai Raghavan   
52  1973         Vandhaale Magaraasi    K. S. Gopalakrishnan   
53  1973            Veettu Mappillai       A. K. Subramaniam   
54  1973  Veettukku Vandha Marumagal               R. Vittal   
55  1973                      Vijaya  Vietnam Veedu Sundaram   

                                                 cast genre  \
1           Vishnuvardhan, Chandrakala, Cho, Manorama  null   
2   Jaishankar, A. V. M. Rajan, Manjula, Thengai S...  null   
3   Jaishankar, S. V. Ranga Rao, A. V. M. Rajan, M...  null   
4   Prameela, Sivakumar, S. V. Subbaiah, M. N. Raj...  null   
5   Jayalalithaa, Ravichandran, Savitri, Major Sun...  null   
6   Sivaji Ganesan, K. R. Vijaya, Major S

In [88]:
getAllMusic()

ex.php?title=Nee_Ullavarai&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Nee_Ullavarai&action=edit&r...


Exception
26 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [89]:
getAllrating()

https://www.imdb.com/title/tt1360101/
Alaigal | 6.9
https://www.imdb.com/title/tt3424912/
Amman Arul | 
https://www.imdb.com/title/tt1389435/
Anbu Sagodharargal | 
https://www.imdb.com/title/tt0154162/
Arangetram | 7.7
https://www.imdb.com/title/tt2720232/
Baghdad Perazhagi | 
https://www.imdb.com/title/tt1435469/
Bharatha Vilas | 6.3
https://www.imdb.com/title/tt0360154/
Deiva Kuzhandhaigal | 
https://www.imdb.com/title/tt3424826/
Dheiva Vamsam | 
https://www.imdb.com/title/tt2725190/
Engal Thaai | 
https://www.imdb.com/title/tt1440166/
Engal Thanga Raja | 
https://www.imdb.com/title/tt0250399/
Ganga Gowri | 7.8
https://www.imdb.com/title/tt0369447/
Gauravam | 7.2
https://www.imdb.com/title/tt8718580/
Iraivan Irukkindran | 9.2
https://www.imdb.com/title/tt0246724/
Karaikkal Ammaiyar | 9.6
https://www.imdb.com/title/tt3421628/
Kasi Yathirai | 
https://www.imdb.com/title/tt1468348/
Kattila Thottila | 
https://www.imdb.com/title/tt5760014/
Komatha En Kulamatha | 
https://www.imdb.com/tit

In [90]:
df

year                       title                director  \
1   1973                     Alaigal           C. V. Sridhar   
2   1973                  Amman Arul                   Pattu   
3   1973          Anbu Sagodharargal          Lakshmi Deepak   
4   1973                  Arangetram          K. Balachander   
5   1973           Baghdad Perazhagi           T. R. Ramanna   
6   1973              Bharatha Vilas    A. C. Tirulokchandar   
7   1973         Deiva Kuzhandhaigal        S. P. Muthuraman   
8   1973               Dheiva Vamsam              K. Vijayan   
9   1973                 Engal Thaai      Malliyam Rajagopal   
10  1973           Engal Thanga Raja   V. B. Rajendra Prasad   
11  1973                 Ganga Gowri          B. R. Panthulu   
12  1973                    Gauravam  Vietnam Veedu Sundaram   
13  1973         Iraivan Irukkindran              H. S. Venu   
14  1973          Karaikkal Ammaiyar         A. P. Nagarajan   
15  1973               Kasi Yathirai        S. P. Muthuraman   
16  1973            Kattila Thottila      Malliyam Rajagopal   
17  1973        Komatha En Kulamatha        M. A. Thirumugam   
18  1973         Malai Naattu Mangai          P. Subramaniam   
19  1973                Malligai Poo            N. S. Maniam   
20  1973         Manidharil Manikkam         C. V. Rajendran   
21  1973                   Manipayal          A. Jagannathan   
22  1973             Manjal Kungumam                   Pattu   
23  1973                 Maru Piravi           T. R. Ramanna   
24  1973                Nalla Mudivu        C. N. Shanmugham   
25  1973              Nathayil Muthu    K. S. Gopalakrishnan   
26  1973               Nee Ullavarai            C. V. Ramana   
27  1973             Nyayam Ketkirom         C. V. Rajendran   
28  1973                 Pasa Deepam                   Pattu   
29  1973        Pattikaattu Ponnaiya             B. S. Ranga   
30  1973            Pennai Nambungal        B. V. Srinivasan   
31  1973           Petha Manam Pithu        S. P. Muthuraman   
32  1973                   Ponnunjal         C. V. Rajendran   
33  1973      Ponnukku Thanga Manasu           Devaraj–Mohan   
34  1973                    Ponvandu            N. S. Maniam   
35  1973                    Pookkari          Krishnan–Panju   
36  1973                  Prarthanai               Kaushikan   
37  1973                       Radha    A. C. Tirulokchandar   
38  1973             Rajaraja Cholan         A. P. Nagarajan   
39  1973         Rajapart Rangadurai             P. Madhavan   
40  1973               School Master          B. R. Panthulu   
41  1973               Shanmugapriya        K. Krishnamurthy   
42  1973      Sollathaan Ninaikkiren          K. Balachander   
43  1973                     Sondham    A. C. Tirulokchandar   
44  1973                 Suryagandhi           V. Srinivasan   
45  1973             Thalai Prasavam             M. Krishnan   
46  1973        Thedi Vandha Lakshmi            G. R. Nathan   
47  1973           Thirumalai Deivam         A. P. Nagarajan   
48  1973       Ulagam Sutrum Valiban      M. G. Ramachandran   
49  1973                     Vaayadi      Madurai Thirumaran   
50  1973                  Vakkuruthi         Mohan Gandhiram   
51  1973              Valli Deivanai        Thillai Raghavan   
52  1973         Vandhaale Magaraasi    K. S. Gopalakrishnan   
53  1973            Veettu Mappillai       A. K. Subramaniam   
54  1973  Veettukku Vandha Marumagal               R. Vittal   
55  1973                      Vijaya  Vietnam Veedu Sundaram   

                                                 cast genre  \
1           Vishnuvardhan, Chandrakala, Cho, Manorama  null   
2   Jaishankar, A. V. M. Rajan, Manjula, Thengai S...  null   
3   Jaishankar, S. V. Ranga Rao, A. V. M. Rajan, M...  null   
4   Prameela, Sivakumar, S. V. Subbaiah, M. N. Raj...  null   
5   Jayalalithaa, Ravichandran, Savitri, Major Sun...  null   
6   Sivaji Ganesan, K. R. Vijaya, Major S

In [91]:
df.to_csv('1973_movies.csv')

# 1972

In [92]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1972'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1972=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [93]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [94]:
for table in tables_1972:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1972'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Agathiyar|A. P. Nagarajan|Sirkazhi Govindarajan, T. R. Mahalingam, A. V. M. Rajan, Lakshmi, Kumari Padmini, C. R. Vijayakumari, T. K. Bagavathy, Sivakumar, A. Sakunthala, Suruli Rajan, Manorama, Sridevi|null|Shri Vijayalakshmi Pictures|/wiki/Agathiyar
2!Annai Abirami|G. N. Velumani|K. R. Vijaya, Sivakumar, Jaya, Thengai Srinivasan, Manorama, Suruli Rajan|null|Sri Kamakshi Agencies|/w/index.php?title=Annai_Abirami&action=edit&redlink=1
3!Annamitta Kai|M. Krishnan|M. G. Ramachandran, Jayalalithaa, Bharathi, Nagesh, Manorama|null|Ramachandra Production|/wiki/Annamitta_Kai
4!Appa Tata|Malliyam Rajagopal|Gemini Ganesan, Padmini, Sowcar Janaki, Major Sundarrajan, Nagesh, Manorama|null|Sri Venkateswara Production|/w/index.php?title=Appa_Tata&action=edit&redlink=1
5!Ashirvadham|R. Devarajan|Jaishankar, K. R. Vijaya, Lakshmi, Nagesh|null|Devagi Pictures|/w/index.php?title=Ashirvadham&action=edit&redlink=1
6!Aval|A. C. Tirulokchandar|Vennira

In [95]:
df=df.drop(0)
df

year                      title              director  \
1   1972                  Agathiyar       A. P. Nagarajan   
2   1972              Annai Abirami        G. N. Velumani   
3   1972              Annamitta Kai           M. Krishnan   
4   1972                  Appa Tata    Malliyam Rajagopal   
5   1972                Ashirvadham          R. Devarajan   
6   1972                       Aval  A. C. Tirulokchandar   
7   1972           Avasara Kalyanam     V. T. Thyagarajan   
8   1972          Bathilukku Bathil           Jambulingam   
9   1972            Deiva Sankalpam            P. R. Somu   
10  1972            Delhi To Madras          I. N. Murthy   
11  1972              Dharmam Engey  A. C. Tirulokchandar   
12  1972                    Dheivam      M. A. Thirumugam   
13  1972  Dhikku Theriyadha Kaattil    N. C. Chakravarthy   
14  1972                 Ellai Kodu           S. Raghavan   
15  1972   Enna Muthalali Sowkiyama    Malliyam Rajagopal   
16  1972                      Ganga             M. Karnan   
17  1972                  Gnana Oli           P. Madhavan   
18  1972              Hello Partner      K. Krishnamurthy   
19  1972              Idhaya Veenai        Krishnan–Panju   
20  1972         Itho Enthan Deivam  A. C. Tirulokchandar   
21  1972                   Jakkamma             M. Karnan   
22  1972           Kadhalikka Vanga          I. N. Murthy   
23  1972           Kanimuthu Paappa      S. P. Muthuraman   
24  1972               Kanna Nalama        K. Balachander   
25  1972                   Kannamma         M. Lakshmanan   
26  1972       Karundhel Kannayiram           R. Sundaram   
27  1972        Kasethan Kadavulada       Chitralaya Gopu   
28  1972              Kurathi Magan  K. S. Gopalakrishnan   
29  1972         Mappillai Azhaippu       T. R. Raghunath   
30  1972                Mr. Sampath                   Cho   
31  1972         Naan Yen Pirandhen           M. Krishnan   
32  1972                Nalla Neram      M. A. Thirumugam   
33  1972             Nawab Naarkali       C. V. Rajendran   
34  1972                     Needhi       C. V. Rajendran   
35  1972        Pattikada Pattanama           P. Madhavan   
36  1972            Pillaiyo Pillai        Krishnan–Panju   
37  1972           Ponmagal Vandhal      Vijaya Narayanan   
38  1972             Puguntha Veedu                 Pattu   
39  1972           Rahasiyapenn 117              Anuradha   
40  1972                       Raja       C. V. Rajendran   
41  1972      Raman Thediya Seethai        P. Neelakantan   
42  1972       Rani Yaar Kuzhanthai           D. Yoganand   
43  1972             Sange Muzhangu        P. Neelakantan   
44  1972           Savalukku Savaal            H. S. Venu   
45  1972             Shakthi Leelai         T. R. Ramanna   
46  1972         Thaikku Oru Pillai                 Pattu   
47  1972                Thangadurai         A. Kasilingam   
48  1972            Thavapudhalavan         V. Srinivasan   
49  1972           Thiruneelakandar           Jambulingam   
50  1972            Unakkum Enakkum          N. S. Maniam   
51  1972                  Varaverpu          I. N. Murthy   
52  1972           Vasantha Maligai     K. S. Prakash Rao   
53  1972          Vazhaiyadi Vazhai  K. S. Gopalakrishnan   
54  1972                Velli Vizha        K. Balachander   
55  1972           Yaar Jambulingam      M. S. Gopinathan   

                                                 cast genre  \
1   Sirkazhi Govindarajan, T. R. Mahalingam, A. V....  null   
2   K. R. Vijaya, Sivakumar, Jaya, Thengai Sriniva...  null   
3   M. G. Ramachandran, Jayalalithaa, Bharathi, Na...  null   
4   Gemini Ganesan, Padmini, Sowcar Janaki, Major ...  null   
5           Jaishankar, K. R. Vijaya, Lakshmi, Nagesh  null   
6   Vennira Aadai Nirmala, A. V. M. Rajan, Shashik...  null   
7            Jaishankar, Vanisri, Nagesh, Rama Prabha  null   
8   A. V. M. Rajan, R. Muthuraman, C. R. Vijayakum...  null   
9   A. V. M. Rajan, 

In [96]:
getAllMusic()

In [97]:
getAllrating()

https://www.imdb.com/title/tt0246414/
Agathiyar | 6.6
https://www.imdb.com/title/tt0233222/
Annai Abirami | 6.6
https://www.imdb.com/title/tt0369238/
Annamitta Kai | 6.6
https://www.imdb.com/title/tt1468289/
Appa Tata | 
https://www.imdb.com/title/tt3424908/
Ashirvadham | 
https://www.imdb.com/title/tt0369253/
Aval | 
Avasara Kalyanam | 
https://www.imdb.com/title/tt0316758/
Bathilukku Bathil | 
https://www.imdb.com/title/tt0369351/
Deiva Sankalpam | 
https://www.imdb.com/title/tt1492786/
Delhi To Madras | 
https://www.imdb.com/title/tt2720492/
Dharmam Engey | 
https://www.imdb.com/title/tt5759896/
Dheivam | 
https://www.imdb.com/title/tt3424604/
Dhikku Theriyadha Kaattil | 
https://www.imdb.com/title/tt1466922/
Ellai Kodu | 
https://www.imdb.com/title/tt1466923/
Enna Muthalali Sowkiyama | 
https://www.imdb.com/name/nm0304303/
Ganga | 
https://www.imdb.com/title/tt0315901/
Gnana Oli | 7.1
Hello Partner | 
https://www.imdb.com/title/tt0233913/
Idhaya Veenai | 6.1
https://www.imdb.com/ti

In [98]:
df

year                      title              director  \
1   1972                  Agathiyar       A. P. Nagarajan   
2   1972              Annai Abirami        G. N. Velumani   
3   1972              Annamitta Kai           M. Krishnan   
4   1972                  Appa Tata    Malliyam Rajagopal   
5   1972                Ashirvadham          R. Devarajan   
6   1972                       Aval  A. C. Tirulokchandar   
7   1972           Avasara Kalyanam     V. T. Thyagarajan   
8   1972          Bathilukku Bathil           Jambulingam   
9   1972            Deiva Sankalpam            P. R. Somu   
10  1972            Delhi To Madras          I. N. Murthy   
11  1972              Dharmam Engey  A. C. Tirulokchandar   
12  1972                    Dheivam      M. A. Thirumugam   
13  1972  Dhikku Theriyadha Kaattil    N. C. Chakravarthy   
14  1972                 Ellai Kodu           S. Raghavan   
15  1972   Enna Muthalali Sowkiyama    Malliyam Rajagopal   
16  1972                      Ganga             M. Karnan   
17  1972                  Gnana Oli           P. Madhavan   
18  1972              Hello Partner      K. Krishnamurthy   
19  1972              Idhaya Veenai        Krishnan–Panju   
20  1972         Itho Enthan Deivam  A. C. Tirulokchandar   
21  1972                   Jakkamma             M. Karnan   
22  1972           Kadhalikka Vanga          I. N. Murthy   
23  1972           Kanimuthu Paappa      S. P. Muthuraman   
24  1972               Kanna Nalama        K. Balachander   
25  1972                   Kannamma         M. Lakshmanan   
26  1972       Karundhel Kannayiram           R. Sundaram   
27  1972        Kasethan Kadavulada       Chitralaya Gopu   
28  1972              Kurathi Magan  K. S. Gopalakrishnan   
29  1972         Mappillai Azhaippu       T. R. Raghunath   
30  1972                Mr. Sampath                   Cho   
31  1972         Naan Yen Pirandhen           M. Krishnan   
32  1972                Nalla Neram      M. A. Thirumugam   
33  1972             Nawab Naarkali       C. V. Rajendran   
34  1972                     Needhi       C. V. Rajendran   
35  1972        Pattikada Pattanama           P. Madhavan   
36  1972            Pillaiyo Pillai        Krishnan–Panju   
37  1972           Ponmagal Vandhal      Vijaya Narayanan   
38  1972             Puguntha Veedu                 Pattu   
39  1972           Rahasiyapenn 117              Anuradha   
40  1972                       Raja       C. V. Rajendran   
41  1972      Raman Thediya Seethai        P. Neelakantan   
42  1972       Rani Yaar Kuzhanthai           D. Yoganand   
43  1972             Sange Muzhangu        P. Neelakantan   
44  1972           Savalukku Savaal            H. S. Venu   
45  1972             Shakthi Leelai         T. R. Ramanna   
46  1972         Thaikku Oru Pillai                 Pattu   
47  1972                Thangadurai         A. Kasilingam   
48  1972            Thavapudhalavan         V. Srinivasan   
49  1972           Thiruneelakandar           Jambulingam   
50  1972            Unakkum Enakkum          N. S. Maniam   
51  1972                  Varaverpu          I. N. Murthy   
52  1972           Vasantha Maligai     K. S. Prakash Rao   
53  1972          Vazhaiyadi Vazhai  K. S. Gopalakrishnan   
54  1972                Velli Vizha        K. Balachander   
55  1972           Yaar Jambulingam      M. S. Gopinathan   

                                                 cast genre  \
1   Sirkazhi Govindarajan, T. R. Mahalingam, A. V....  null   
2   K. R. Vijaya, Sivakumar, Jaya, Thengai Sriniva...  null   
3   M. G. Ramachandran, Jayalalithaa, Bharathi, Na...  null   
4   Gemini Ganesan, Padmini, Sowcar Janaki, Major ...  null   
5           Jaishankar, K. R. Vijaya, Lakshmi, Nagesh  null   
6   Vennira Aadai Nirmala, A. V. M. Rajan, Shashik...  null   
7            Jaishankar, Vanisri, Nagesh, Rama Prabha  null   
8   A. V. M. Rajan, R. Muthuraman, C. R. Vijayakum...  null   
9   A. V. M. Rajan, 

In [99]:
df.to_csv('1972_movies.csv')

# 1971

In [101]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1971'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1971=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [102]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [103]:
for table in tables_1971:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1971'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Aathi Parasakthi|K. S. Gopalakrishnan|Gemini Ganesan, Jayalalithaa, Padmini, R. Muthuraman, S. V. Ranga Rao, S. Varalakshmi, Vanisri, Rajasree, Vennira Aadai Nirmala, S. V. Subbaiah, Sridevi|null|Chitra Productions|/wiki/Aathi_Parasakthi
2!Anbukkor Annan|N. S. Maniam|Gemini Ganesan, Jaishankar, Vennira Aadai Nirmala, Srividya|null|Karuna Films|/w/index.php?title=Anbukkor_Annan&action=edit&redlink=1
3!Annai Velankanni|K. Thangappan|Gemini Ganesan, Jayalalithaa, Padmini, K. R. Vijaya, R. Muthuraman, Devika, Nagesh, Thengai Srinivasan, Manorama, Sachu, Sivakumar, Srividya|null|Giri Movies|/wiki/Annai_Velankanni
4!Arunodhayam|V. Srinivasan|Sivaji Ganesan, B. Saroja Devi, R. Muthuraman, Lakshmi, Cho, Manorama|null|Muktha Films|/wiki/Arunodhayam
5!Arutperunjothi|A. T. Krishnaswamy|Master Sridhar, A. P. Nagarajan, K. A. Thangavelu, M. Saroja, Devaki|null|Balu Films|/w/index.php?title=Arutperunjothi&action=edit&redlink=1
6!Avalukendru Oru 

In [104]:
df=df.drop(0)
df

year                             title              director  \
1   1971                  Aathi Parasakthi  K. S. Gopalakrishnan   
2   1971                    Anbukkor Annan          N. S. Maniam   
3   1971                  Annai Velankanni         K. Thangappan   
4   1971                       Arunodhayam         V. Srinivasan   
5   1971                    Arutperunjothi    A. T. Krishnaswamy   
6   1971             Avalukendru Oru Manam         C. V. Sridhar   
7   1971                              Babu  A. C. Tirulokchandar   
8   1971                     Deivam Pesuma        Chinni Sampath   
9   1971                      Iru Thuruvam         S. Ramanathan   
10  1971                     Irulum Oliyum        S. R. Puttanna   
11  1971               Justice Viswanathan          G. R. Nathan   
12  1971                        Kankaatchi       A. P. Nagarajan   
13  1971                        Kettikaran            H. S. Venu   
14  1971                     Kulama Gunama  K. S. Gopalakrishnan   
15  1971                     Kumari Kottam        P. Neelakantan   
16  1971                   Meendum Vazhven            T. N. Balu   
17  1971              Muhammad bin Tughluq                   Cho   
18  1971                Moondru Dheivangal           Dada Mirasi   
19  1971                   Naangu Suvargal        K. Balachander   
20  1971                     Needhi Dhevan          R. Devarajan   
21  1971                   Neerum Neruppum        P. Neelakantan   
22  1971                   Nootrukku Nooru        K. Balachander   
23  1971                  Oru Thaai Makkal        P. Neelakantan   
24  1971                  Pattondru Ketten           S. Raghavan   
25  1971                     Poi Sollathey               L. Balu   
26  1971                          Praptham               Savitri   
27  1971                   Pudhiya Vazhkai       C. V. Rajendran   
28  1971                          Punnagai        K. Balachander   
29  1971                    Ranga Rattinam        Krishnan–Panju   
30  1971                     Rickshawkaran           M. Krishnan   
31  1971                          Sabatham           P. Madhavan   
32  1971                    Savaale Samali    Malliyam Rajagopal   
33  1971              Sudarum Sooravaliyum        S. R. Puttanna   
34  1971                        Soodhattam  'Madurai' Thirumaran   
35  1971                Sumathi En Sundari       C. V. Rajendran   
36  1971                    Thanga Gopuram      M. S. Solaimalai   
37  1971                     Thangaikkaaga           D. Yoganand   
38  1971                       Then Kinnam     K. Krishnamoorthy   
39  1971                     Thenum Paalum           P. Madhavan   
40  1971                         Therottam           V. T. Arasu   
41  1971                        Thirumagal       T. R. Raghunath   
42  1971             Thulli Odum Pullimaan             A. B. Raj   
43  1971            Uttharavindri Ulle Vaa    N. C. Chakravarthy   
44  1971                              Uyir            Somasundar   
45  1971                 Veettukoru Pillai         T. R. Ramanna   
46  1971                       Veguli Penn         S. S. Devadas   
47  1971  Yaanai Valartha Vaanampadi Magan        P. Subramaniam   

                                                 cast genre  \
1   Gemini Ganesan, Jayalalithaa, Padmini, R. Muth...  null   
2   Gemini Ganesan, Jaishankar, Vennira Aadai Nirm...  null   
3   Gemini Ganesan, Jayalalithaa, Padmini, K. R. V...  null   
4   Sivaji Ganesan, B. Saroja Devi, R. Muthuraman,...  null   
5   Master Sridhar, A. P. Nagarajan, K. A. Thangav...  null   
6   Gemini Ganesan, Bharathi, Kanchana, R. Muthuraman  null   
7   Sivaji Ganesan, Sowcar Janaki, K. Balaji, Nage...  null   
8   A. V. M. Rajan, Vennira Aadai Nirmala, Sandhya...  null   
9   Sivaji Ganesan, Padmini, R. Muthuraman, Nagesh...  null   
10  Vanisri, A. V. M. Rajan, R. Muthuraman, Nagesh...  null   
11  Ravichandran, Major Sundarrajan, C. I. D. 

In [105]:
getAllMusic()

In [106]:
getAllrating()

https://www.imdb.com/title/tt0215468/
Aathi Parasakthi | 6.1
https://www.imdb.com/title/tt2710268/
Anbukkor Annan | 
https://www.imdb.com/title/tt0810757/
Annai Velankanni | 7.2
https://www.imdb.com/title/tt5583446/
Arunodhayam | 
https://www.imdb.com/title/tt3426056/
Arutperunjothi | 
https://www.imdb.com/title/tt1431016/
Avalukendru Oru Manam | 6.8
https://www.imdb.com/title/tt1435464/
Babu | 6.1
https://www.imdb.com/title/tt8938864/
Deivam Pesuma | 
https://www.imdb.com/title/tt1440207/
Iru Thuruvam | 5.9
https://www.imdb.com/title/tt3426036/
Irulum Oliyum | 
https://www.imdb.com/title/tt8976534/
Justice Viswanathan | 
https://www.imdb.com/title/tt0246723/
Kankaatchi | 6.6
https://www.imdb.com/title/tt9015484/
Kettikaran | 
https://www.imdb.com/title/tt0215913/
Kulama Gunama | 6.1
https://www.imdb.com/title/tt0248141/
Kumari Kottam | 6.8
https://www.imdb.com/title/tt3426046/
Meendum Vazhven | 
https://www.imdb.com/title/tt0316269/
Muhammad bin Tughluq | 7.7
https://www.imdb.com/titl

In [107]:
df

year                             title              director  \
1   1971                  Aathi Parasakthi  K. S. Gopalakrishnan   
2   1971                    Anbukkor Annan          N. S. Maniam   
3   1971                  Annai Velankanni         K. Thangappan   
4   1971                       Arunodhayam         V. Srinivasan   
5   1971                    Arutperunjothi    A. T. Krishnaswamy   
6   1971             Avalukendru Oru Manam         C. V. Sridhar   
7   1971                              Babu  A. C. Tirulokchandar   
8   1971                     Deivam Pesuma        Chinni Sampath   
9   1971                      Iru Thuruvam         S. Ramanathan   
10  1971                     Irulum Oliyum        S. R. Puttanna   
11  1971               Justice Viswanathan          G. R. Nathan   
12  1971                        Kankaatchi       A. P. Nagarajan   
13  1971                        Kettikaran            H. S. Venu   
14  1971                     Kulama Gunama  K. S. Gopalakrishnan   
15  1971                     Kumari Kottam        P. Neelakantan   
16  1971                   Meendum Vazhven            T. N. Balu   
17  1971              Muhammad bin Tughluq                   Cho   
18  1971                Moondru Dheivangal           Dada Mirasi   
19  1971                   Naangu Suvargal        K. Balachander   
20  1971                     Needhi Dhevan          R. Devarajan   
21  1971                   Neerum Neruppum        P. Neelakantan   
22  1971                   Nootrukku Nooru        K. Balachander   
23  1971                  Oru Thaai Makkal        P. Neelakantan   
24  1971                  Pattondru Ketten           S. Raghavan   
25  1971                     Poi Sollathey               L. Balu   
26  1971                          Praptham               Savitri   
27  1971                   Pudhiya Vazhkai       C. V. Rajendran   
28  1971                          Punnagai        K. Balachander   
29  1971                    Ranga Rattinam        Krishnan–Panju   
30  1971                     Rickshawkaran           M. Krishnan   
31  1971                          Sabatham           P. Madhavan   
32  1971                    Savaale Samali    Malliyam Rajagopal   
33  1971              Sudarum Sooravaliyum        S. R. Puttanna   
34  1971                        Soodhattam  'Madurai' Thirumaran   
35  1971                Sumathi En Sundari       C. V. Rajendran   
36  1971                    Thanga Gopuram      M. S. Solaimalai   
37  1971                     Thangaikkaaga           D. Yoganand   
38  1971                       Then Kinnam     K. Krishnamoorthy   
39  1971                     Thenum Paalum           P. Madhavan   
40  1971                         Therottam           V. T. Arasu   
41  1971                        Thirumagal       T. R. Raghunath   
42  1971             Thulli Odum Pullimaan             A. B. Raj   
43  1971            Uttharavindri Ulle Vaa    N. C. Chakravarthy   
44  1971                              Uyir            Somasundar   
45  1971                 Veettukoru Pillai         T. R. Ramanna   
46  1971                       Veguli Penn         S. S. Devadas   
47  1971  Yaanai Valartha Vaanampadi Magan        P. Subramaniam   

                                                 cast genre  \
1   Gemini Ganesan, Jayalalithaa, Padmini, R. Muth...  null   
2   Gemini Ganesan, Jaishankar, Vennira Aadai Nirm...  null   
3   Gemini Ganesan, Jayalalithaa, Padmini, K. R. V...  null   
4   Sivaji Ganesan, B. Saroja Devi, R. Muthuraman,...  null   
5   Master Sridhar, A. P. Nagarajan, K. A. Thangav...  null   
6   Gemini Ganesan, Bharathi, Kanchana, R. Muthuraman  null   
7   Sivaji Ganesan, Sowcar Janaki, K. Balaji, Nage...  null   
8   A. V. M. Rajan, Vennira Aadai Nirmala, Sandhya...  null   
9   Sivaji Ganesan, Padmini, R. Muthuraman, Nagesh...  null   
10  Vanisri, A. V. M. Rajan, R. Muthuraman, Nagesh...  null   
11  Ravichandran, Major Sundarrajan, C. I. D. 

In [108]:
df.to_csv('1971_movies.csv')

# 1970

In [109]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1970'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1970=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [110]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [111]:
for table in tables_1970:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1970'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Anadhai Anandhan|Krishnan–Panju|A. V. M. Rajan, Jayalalithaa, Master Sekhar, R. Muthuraman, Nagesh, Thengai Srinivasan|null|Muthuvel Movies|/wiki/Anadhai_Anandhan
2!CID Shankar|R. Sundaram|Jaishankar, CID Sakunthala, Thengai Srinivasan|null|Modern Theatres|/wiki/CID_Shankar
3!Dharisanam|V. T. Arasu|A. V. M. Rajan, Pushpalatha, Cho, Manorama, G. Sakunthala|null|Senthoor Films|/wiki/Dharisanam
4!Ethirkalam|M. S. Solaimalai|Gemini Ganesan, Jaishankar, Padmini, Vanisri, Nagesh|null|Thanigaivel Pictures|/wiki/Ethirkalam
5!En Annan|P. Neelakantan|M. G. Ramachandran, Jayalalithaa, R. Muthuraman, Vijaya Nirmala, Cho|null|Venus Pictures|/wiki/En_Annan
6!Enga Mama|A. C. Tirulokchandar|Sivaji Ganesan, Jayalalithaa, Vennira Aadai Nirmala, Cho, Rama Prabha|null|Jayaar Movies|/wiki/Enga_Mama
7!Engal Thangam|Krishnan–Panju|M. G. Ramachandran, Jayalalithaa, A. V. M. Rajan, Pushpalatha, Cho, Thengai Srinivasan, Manorama|null|Mekala Pictures|/wiki/E

In [112]:
df=df.drop(0)
df

year                   title                director  \
1   1970        Anadhai Anandhan          Krishnan–Panju   
2   1970             CID Shankar             R. Sundaram   
3   1970              Dharisanam             V. T. Arasu   
4   1970              Ethirkalam        M. S. Solaimalai   
5   1970                En Annan          P. Neelakantan   
6   1970               Enga Mama    A. C. Tirulokchandar   
7   1970           Engal Thangam          Krishnan–Panju   
8   1970     Engirundho Vandhaal    A. C. Tirulokchandar   
9   1970                Ethiroli          K. Balachander   
10  1970               Jeevanadi       A. K. Subramanyam   
11  1970        Kaaviya Thalaivi          K. Balachander   
12  1970            Kadhal Jothi   Thirumalai-Mahalingam   
13  1970            Kalam Vellum               M. Karnan   
14  1970        Kalyana Oorvalam     K. S. Sethumadhavan   
15  1970                Kanmalar                   Pattu   
16  1970          Kannan Varuvan            I. N. Murthy   
17  1970       Kasthuri Thilakam      Malliyam Rajagopal   
18  1970        Kumara Sambhavam          P. Subramaniam   
19  1970                Maanavan        M. A. Thirumugam   
20  1970        Maattukara Velan          P. Neelakantan   
21  1970                 Malathi    K. S. Gopalakrishnan   
22  1970             Nadu Iravil          S. Balachander   
23  1970     Namma Kuzhanthaigal                Srikanth   
24  1970     Namma Veettu Deivam          G. N. Velumani   
25  1970              Navagraham          K. Balachander   
26  1970       Nilave Nee Satchi             P. Madhavan   
27  1970   Noorandu Kalam Vazhga              K. Sampath   
28  1970            Paadhukaappu            A. Bhimsingh   
29  1970           Patham Pasali          K. Balachander   
30  1970             Penn Deivam        M. A. Thirumugam   
31  1970  Raman Ethanai Ramanadi             P. Madhavan   
32  1970                Sangamam             Dada Mirasi   
33  1970                 Snehidi         G. Ramakrishnan   
34  1970                  Sorgam           T. R. Ramanna   
35  1970     Thabalkaran Thangai    K. S. Gopalakrishnan   
36  1970               Thalaivan  P. A. ThomasSingamuthu   
37  1970  Thedi Vandha Mappillai          B. R. Panthulu   
38  1970   Thirumalai Thenkumari         A. P. Nagarajan   
39  1970               Vairagyam           A. Kasilingam   
40  1970          Veettuku Veedu         C. V. Rajendran   
41  1970           Vietnam Veedu             P. Madhavan   
42  1970      Vilaiyaattu Pillai         A. P. Nagarajan   
43  1970                   Yaen?           T. R. Ramanna   

                                                 cast genre  \
1   A. V. M. Rajan, Jayalalithaa, Master Sekhar, R...  null   
2      Jaishankar, CID Sakunthala, Thengai Srinivasan  null   
3   A. V. M. Rajan, Pushpalatha, Cho, Manorama, G....  null   
4   Gemini Ganesan, Jaishankar, Padmini, Vanisri, ...  null   
5   M. G. Ramachandran, Jayalalithaa, R. Muthurama...  null   
6   Sivaji Ganesan, Jayalalithaa, Vennira Aadai Ni...  null   
7   M. G. Ramachandran, Jayalalithaa, A. V. M. Raj...  null   
8   Sivaji Ganesan, Jayalalithaa, K. Balaji, Devik...  null   
9   Sivaji Ganesan, S. S. Rajendran, K. R. Vijaya,...  null   
10                              Ravichandran, Lakshmi  null   
11        Gemini Ganesan, Sowcar Janaki, Ravichandran  null   
12  Jaishankar, Ravichandran, Kanchana, Bhanumathi...  null   
13  Jaishankar, C. R. Vijayakumari, Nagesh, Vijaya...  null   
14                               Nagesh, K. R. Vijaya  null   
15  Gemini Ganesan, K. R. Vijaya, B. Saroja Devi, ...  null   
16  Jaishankar, R. Muthuraman, Lakshmi, Vennira Aa...  null   
17  Sowcar Janaki, Major Sundarrajan, Nagesh, Siva...  null   
18        Gemini Ganesan, Padmini, Rajasree, Srividya  null   
19  Jaishankar, Sowcar Janaki, Lakshmi, R. Muthura...  null   
20  M. G. Ramachandran, Jayalalithaa, Lakshmi, Cho...  null   
21  Gemini Ganesan, B. Saroja Devi, Ravi

In [113]:
getAllMusic()

In [114]:
getAllrating()

https://www.imdb.com/title/tt0233174/
Anadhai Anandhan | 
https://www.imdb.com/title/tt8929746/
CID Shankar | 5.8
https://www.imdb.com/title/tt3424830/
Dharisanam | 
https://www.imdb.com/title/tt1466930/
Ethirkalam | 
https://www.imdb.com/title/tt0247997/
En Annan | 5.2
https://www.imdb.com/title/tt1440164/
Enga Mama | 6.9
https://www.imdb.com/title/tt0233650/
Engal Thangam | 6.8
https://www.imdb.com/title/tt1440167/
Engirundho Vandhaal | 6.7
https://www.imdb.com/title/tt0154463/
Ethiroli | 7.4
https://www.imdb.com/title/tt3424884/
Jeevanadi | 
https://www.imdb.com/title/tt0154721/
Kaaviya Thalaivi | 6.2
https://www.imdb.com/title/tt3437594/
Kadhal Jothi | 
https://www.imdb.com/title/tt6568614/
Kalam Vellum | 6.0
https://www.imdb.com/title/tt7228740/
Kalyana Oorvalam | 
https://www.imdb.com/title/tt1466957/
Kanmalar | 
https://www.imdb.com/title/tt3424810/
Kannan Varuvan | 
https://www.imdb.com/title/tt3424890/
Kasthuri Thilakam | 
https://www.imdb.com/title/tt0187218/
Kumara Sambhavam

In [115]:
df

year                   title                director  \
1   1970        Anadhai Anandhan          Krishnan–Panju   
2   1970             CID Shankar             R. Sundaram   
3   1970              Dharisanam             V. T. Arasu   
4   1970              Ethirkalam        M. S. Solaimalai   
5   1970                En Annan          P. Neelakantan   
6   1970               Enga Mama    A. C. Tirulokchandar   
7   1970           Engal Thangam          Krishnan–Panju   
8   1970     Engirundho Vandhaal    A. C. Tirulokchandar   
9   1970                Ethiroli          K. Balachander   
10  1970               Jeevanadi       A. K. Subramanyam   
11  1970        Kaaviya Thalaivi          K. Balachander   
12  1970            Kadhal Jothi   Thirumalai-Mahalingam   
13  1970            Kalam Vellum               M. Karnan   
14  1970        Kalyana Oorvalam     K. S. Sethumadhavan   
15  1970                Kanmalar                   Pattu   
16  1970          Kannan Varuvan            I. N. Murthy   
17  1970       Kasthuri Thilakam      Malliyam Rajagopal   
18  1970        Kumara Sambhavam          P. Subramaniam   
19  1970                Maanavan        M. A. Thirumugam   
20  1970        Maattukara Velan          P. Neelakantan   
21  1970                 Malathi    K. S. Gopalakrishnan   
22  1970             Nadu Iravil          S. Balachander   
23  1970     Namma Kuzhanthaigal                Srikanth   
24  1970     Namma Veettu Deivam          G. N. Velumani   
25  1970              Navagraham          K. Balachander   
26  1970       Nilave Nee Satchi             P. Madhavan   
27  1970   Noorandu Kalam Vazhga              K. Sampath   
28  1970            Paadhukaappu            A. Bhimsingh   
29  1970           Patham Pasali          K. Balachander   
30  1970             Penn Deivam        M. A. Thirumugam   
31  1970  Raman Ethanai Ramanadi             P. Madhavan   
32  1970                Sangamam             Dada Mirasi   
33  1970                 Snehidi         G. Ramakrishnan   
34  1970                  Sorgam           T. R. Ramanna   
35  1970     Thabalkaran Thangai    K. S. Gopalakrishnan   
36  1970               Thalaivan  P. A. ThomasSingamuthu   
37  1970  Thedi Vandha Mappillai          B. R. Panthulu   
38  1970   Thirumalai Thenkumari         A. P. Nagarajan   
39  1970               Vairagyam           A. Kasilingam   
40  1970          Veettuku Veedu         C. V. Rajendran   
41  1970           Vietnam Veedu             P. Madhavan   
42  1970      Vilaiyaattu Pillai         A. P. Nagarajan   
43  1970                   Yaen?           T. R. Ramanna   

                                                 cast genre  \
1   A. V. M. Rajan, Jayalalithaa, Master Sekhar, R...  null   
2      Jaishankar, CID Sakunthala, Thengai Srinivasan  null   
3   A. V. M. Rajan, Pushpalatha, Cho, Manorama, G....  null   
4   Gemini Ganesan, Jaishankar, Padmini, Vanisri, ...  null   
5   M. G. Ramachandran, Jayalalithaa, R. Muthurama...  null   
6   Sivaji Ganesan, Jayalalithaa, Vennira Aadai Ni...  null   
7   M. G. Ramachandran, Jayalalithaa, A. V. M. Raj...  null   
8   Sivaji Ganesan, Jayalalithaa, K. Balaji, Devik...  null   
9   Sivaji Ganesan, S. S. Rajendran, K. R. Vijaya,...  null   
10                              Ravichandran, Lakshmi  null   
11        Gemini Ganesan, Sowcar Janaki, Ravichandran  null   
12  Jaishankar, Ravichandran, Kanchana, Bhanumathi...  null   
13  Jaishankar, C. R. Vijayakumari, Nagesh, Vijaya...  null   
14                               Nagesh, K. R. Vijaya  null   
15  Gemini Ganesan, K. R. Vijaya, B. Saroja Devi, ...  null   
16  Jaishankar, R. Muthuraman, Lakshmi, Vennira Aa...  null   
17  Sowcar Janaki, Major Sundarrajan, Nagesh, Siva...  null   
18        Gemini Ganesan, Padmini, Rajasree, Srividya  null   
19  Jaishankar, Sowcar Janaki, Lakshmi, R. Muthura...  null   
20  M. G. Ramachandran, Jayalalithaa, Lakshmi, Cho...  null   
21  Gemini Ganesan, B. Saroja Devi, Ravi

In [116]:
df.to_csv('1970_movies.csv')

# 1969

In [14]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1969'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1969=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [15]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [16]:
for table in tables_1969:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1969'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Adimai Penn|K. Shankar|M. G. Ramachandran, Jayalalithaa, J. P. Chandrababu, Cho, Rajasree, Pandari Bai|null|Em. Gee. Yar. Pictures|/wiki/Adimai_Penn
2!Aindhu Laksham|G. Ramakrishnan|Gemini Ganesan, B. Saroja Devi, Cho, Shylashri, Manorama, Thengai Srinivasan, Sachu|null|Sudha Movies|/wiki/Aindhu_Laksham
3!Akka Thangai|M. A. Thirumugam|Jaishankar, K. R. Vijaya, Sowcar Janaki, Nagesh, Vijaya Lalitha|null|Dhandayuthapani Films|/wiki/Akka_Thangai
4!Anbalippu|A. C. Tirulokchandar|Sivaji Ganesan, Jaishankar, B. Saroja Devi, Vijaya Nirmala, Nagesh, T. R. Ramachandran, Sachu|null|Kamala Movies|/wiki/Anbalippu
5!Anjal Petti 520|T. N. Balu|Sivaji Ganesan, B. Saroja Devi, Nagesh|null|Bharath Movies|/wiki/Anjal_Petti_520
6!Annaiyum Pithavum|Krishnan–Panju|A. V. M. Rajan, Vanisri, Sivakumar, Lakshmi, Cho, Manorama|null|AVM Productions|/wiki/Annaiyum_Pithavum
7!Athai Magal|T. R. Ramanna|Jaishankar, Vanisri, Nagesh|null|Dhanabhagyam Pictures|/w/i

In [17]:
df=df.drop(0)
df

year                  title              director  \
1   1969            Adimai Penn            K. Shankar   
2   1969         Aindhu Laksham       G. Ramakrishnan   
3   1969           Akka Thangai      M. A. Thirumugam   
4   1969              Anbalippu  A. C. Tirulokchandar   
5   1969        Anjal Petti 520            T. N. Balu   
6   1969      Annaiyum Pithavum        Krishnan–Panju   
7   1969            Athai Magal         T. R. Ramanna   
8   1969        Avare En Deivam       C. N. Shanmugam   
9   1969            Aayiram Poi         V. Srinivasan   
10  1969         Captain Ranjan        T. P. Sundaram   
11  1969            Chella Penn      K. Krishnamurthy   
12  1969            Deiva Magan  A. C. Tirulokchandar   
13  1969         Gurudhakshanai       A. P. Nagarajan   
14  1969            Iru Kodugal        K. Balachander   
15  1969            Kanne Pappa           P. Madhavan   
16  1969             Kanni Penn         A. Kasilingam   
17  1969         Kaaval Dheivam            K. Vijayan   
18  1969           Kula Vilakku  K. S. Gopalakrishnan   
19  1969        Kuzhandai Ullam               Savitri   
20  1969      Magane Nee Vazhga           M. Krishnan   
21  1969             Mahizampoo           V. T. Arasu   
22  1969                Manaivi            G. K. Ramu   
23  1969             Manasatchi            T. N. Balu   
24  1969               Mannippu           M. Krishnan   
25  1969      Naangu Killadigal               L. Balu   
26  1969              Nam Naadu     C. P. Jambulingam   
27  1969     Nil Gavani Kadhali       C. V. Rajendran   
28  1969            Nirai Kudam         V. Srinivasan   
29  1969             Odum Nadhi           Dada Mirasi   
30  1969             Paal Kudam                 Pattu   
31  1969  Pennai Vazha Vidungal          R. Devarajan   
32  1969        Ponnu Mappillai         S. Ramanathan   
33  1969         Poova Thalaiya        K. Balachander   
34  1969              Porchilai         A. V. Francis   
35  1969         Shanti Nilayam            G. S. Mani   
36  1969       Singapore Seeman    M. A. V. Rajendran   
37  1969          Sivandha Mann         C. V. Sridhar   
38  1969            Suba Dhinam             Sreekanth   
39  1969               Thalattu              Vipindas   
40  1969        Thanga Surangam         T. R. Ramanna   
41  1969           Thanga Malar       D. S. Rajagopal   
42  1969               Thirudan  A. C. Tirulokchandar   
43  1969            Thulabharam            A. Vincent   
44  1969              Thunaivan      M. A. Thirumugam   
45  1969    Ulagum Ivvalavudhan   Vedantam Raghavaiah   
46  1969           Vaa Raja Vaa       A. P. Nagarajan   

                                                 cast genre  \
1   M. G. Ramachandran, Jayalalithaa, J. P. Chandr...  null   
2   Gemini Ganesan, B. Saroja Devi, Cho, Shylashri...  null   
3   Jaishankar, K. R. Vijaya, Sowcar Janaki, Nages...  null   
4   Sivaji Ganesan, Jaishankar, B. Saroja Devi, Vi...  null   
5              Sivaji Ganesan, B. Saroja Devi, Nagesh  null   
6   A. V. M. Rajan, Vanisri, Sivakumar, Lakshmi, C...  null   
7                         Jaishankar, Vanisri, Nagesh  null   
8   Gemini Ganesan, K. R. Vijaya, C. R. Vijayakuma...  null   
9                  Jaishankar, Vanisri, Cho, Manorama  null   
10                                     Ranjan, Vanaja  null   
11                     Ravichandran, Kanchana, Nagesh  null   
12  Sivaji Ganesan, Jayalalithaa, Pandari Bai, Vij...  null   
13              Sivaji Ganesan, Padmini, Jayalalithaa  null   
14    Gemini Ganesan, Sowcar Janaki, Jayanthi, Nagesh  null   
15  K. R. Vijaya, R. Muthuraman, 'Baby' Rani, C. R...  null   
16  Jaishankar, Vanisri, Sivakumar, Lakshmi, Venni...  null   
17  Sivaji Ganesan, Sowcar Janaki, Sivakumar, Lakshmi  null   
18  Gemini Ganesan, B. Saroja Devi, Nagesh, S. V. ...  null   
19  Gemini Ganesan, Sowcar Janaki, Vanisri, Thenga...  null   
20           Jaishankar, Lakshmi, Anjali Devi, Nagesh  null   
2

In [18]:
getAllMusic()

In [19]:
getAllrating()

https://www.imdb.com/title/tt0317056/
Adimai Penn | 7.4
https://www.imdb.com/title/tt1464166/
Aindhu Laksham | 
https://www.imdb.com/title/tt8908240/
Akka Thangai | 
https://www.imdb.com/title/tt0290415/
Anbalippu | 6.2
https://www.imdb.com/title/tt1440120/
Anjal Petti 520 | 
https://www.imdb.com/title/tt0233223/
Annaiyum Pithavum | 6.2
https://www.imdb.com/title/tt0246666/
Athai Magal | 
https://www.imdb.com/title/tt1464170/
Avare En Deivam | 
https://www.imdb.com/title/tt5235106/
Aayiram Poi | 7.0
https://www.imdb.com/name/nm0710206/
Captain Ranjan | 
https://www.imdb.com/title/tt0317859/
Chella Penn | 6.0
https://www.imdb.com/title/tt1437598/
Deiva Magan | 7.3
https://www.imdb.com/title/tt0246666/
Gurudhakshanai | 
https://www.imdb.com/title/tt0154651/
Iru Kodugal | 7.1
https://www.imdb.com/title/tt2729284/
Kanne Pappa | 
https://www.imdb.com/title/tt2729288/
Kanni Penn | 
https://www.imdb.com/title/tt0317818/
Kaaval Dheivam | 
https://www.imdb.com/title/tt0215914/
Kula Vilakku | 5.

In [20]:
df

year                  title              director  \
1   1969            Adimai Penn            K. Shankar   
2   1969         Aindhu Laksham       G. Ramakrishnan   
3   1969           Akka Thangai      M. A. Thirumugam   
4   1969              Anbalippu  A. C. Tirulokchandar   
5   1969        Anjal Petti 520            T. N. Balu   
6   1969      Annaiyum Pithavum        Krishnan–Panju   
7   1969            Athai Magal         T. R. Ramanna   
8   1969        Avare En Deivam       C. N. Shanmugam   
9   1969            Aayiram Poi         V. Srinivasan   
10  1969         Captain Ranjan        T. P. Sundaram   
11  1969            Chella Penn      K. Krishnamurthy   
12  1969            Deiva Magan  A. C. Tirulokchandar   
13  1969         Gurudhakshanai       A. P. Nagarajan   
14  1969            Iru Kodugal        K. Balachander   
15  1969            Kanne Pappa           P. Madhavan   
16  1969             Kanni Penn         A. Kasilingam   
17  1969         Kaaval Dheivam            K. Vijayan   
18  1969           Kula Vilakku  K. S. Gopalakrishnan   
19  1969        Kuzhandai Ullam               Savitri   
20  1969      Magane Nee Vazhga           M. Krishnan   
21  1969             Mahizampoo           V. T. Arasu   
22  1969                Manaivi            G. K. Ramu   
23  1969             Manasatchi            T. N. Balu   
24  1969               Mannippu           M. Krishnan   
25  1969      Naangu Killadigal               L. Balu   
26  1969              Nam Naadu     C. P. Jambulingam   
27  1969     Nil Gavani Kadhali       C. V. Rajendran   
28  1969            Nirai Kudam         V. Srinivasan   
29  1969             Odum Nadhi           Dada Mirasi   
30  1969             Paal Kudam                 Pattu   
31  1969  Pennai Vazha Vidungal          R. Devarajan   
32  1969        Ponnu Mappillai         S. Ramanathan   
33  1969         Poova Thalaiya        K. Balachander   
34  1969              Porchilai         A. V. Francis   
35  1969         Shanti Nilayam            G. S. Mani   
36  1969       Singapore Seeman    M. A. V. Rajendran   
37  1969          Sivandha Mann         C. V. Sridhar   
38  1969            Suba Dhinam             Sreekanth   
39  1969               Thalattu              Vipindas   
40  1969        Thanga Surangam         T. R. Ramanna   
41  1969           Thanga Malar       D. S. Rajagopal   
42  1969               Thirudan  A. C. Tirulokchandar   
43  1969            Thulabharam            A. Vincent   
44  1969              Thunaivan      M. A. Thirumugam   
45  1969    Ulagum Ivvalavudhan   Vedantam Raghavaiah   
46  1969           Vaa Raja Vaa       A. P. Nagarajan   

                                                 cast genre  \
1   M. G. Ramachandran, Jayalalithaa, J. P. Chandr...  null   
2   Gemini Ganesan, B. Saroja Devi, Cho, Shylashri...  null   
3   Jaishankar, K. R. Vijaya, Sowcar Janaki, Nages...  null   
4   Sivaji Ganesan, Jaishankar, B. Saroja Devi, Vi...  null   
5              Sivaji Ganesan, B. Saroja Devi, Nagesh  null   
6   A. V. M. Rajan, Vanisri, Sivakumar, Lakshmi, C...  null   
7                         Jaishankar, Vanisri, Nagesh  null   
8   Gemini Ganesan, K. R. Vijaya, C. R. Vijayakuma...  null   
9                  Jaishankar, Vanisri, Cho, Manorama  null   
10                                     Ranjan, Vanaja  null   
11                     Ravichandran, Kanchana, Nagesh  null   
12  Sivaji Ganesan, Jayalalithaa, Pandari Bai, Vij...  null   
13              Sivaji Ganesan, Padmini, Jayalalithaa  null   
14    Gemini Ganesan, Sowcar Janaki, Jayanthi, Nagesh  null   
15  K. R. Vijaya, R. Muthuraman, 'Baby' Rani, C. R...  null   
16  Jaishankar, Vanisri, Sivakumar, Lakshmi, Venni...  null   
17  Sivaji Ganesan, Sowcar Janaki, Sivakumar, Lakshmi  null   
18  Gemini Ganesan, B. Saroja Devi, Nagesh, S. V. ...  null   
19  Gemini Ganesan, Sowcar Janaki, Vanisri, Thenga...  null   
20           Jaishankar, Lakshmi, Anjali Devi, Nagesh  null   
2

In [21]:
df.to_csv('1969_movies.csv')

# 1968

In [22]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1968'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1968=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [23]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [24]:
for table in tables_1968:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1968'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Anbu Vazhi|M. Natesan|Jaishankar, L. Vijayalakshmi, Nagesh|null|Pavithra Pictures|/w/index.php?title=Anbu_Vazhi&action=edit&redlink=1
2!Andru Kanda Mugam|G. Ramakrishnan|Ravichandran, Jayalalithaa, Nagesh|null|New India Productions|/wiki/Andru_Kanda_Mugam
3!Bommalattam|V. Srinivasan|Jaishankar, Jayalalithaa, Nagesh, Cho|null|Muktha Films|/wiki/Bommalattam_(1968_film)
4!Chakkaram|A. Kasilingam|Gemini Ganesan, A. V. M. Rajan, Vennira Aadai Nirmala|null|Annai Films|/wiki/Chakkaram
5!Deivega Uravu|Sathyam|Jaishankar, Devika, Nagesh|null|Thirumurugan Pictures|/w/index.php?title=Deivega_Uravu&action=edit&redlink=1
6!Delhi Mapillai|Devan|Ravichandran, Rajasree, Cho, Sachu|null|Poomagal Productions|/wiki/Delhi_Mapillai
7!DeviRemake of Malayalam film|A. K. Velan|R. Muthuraman, Devika|null|Arunachalam Pictures|/wiki/Kavyamela
8!Ethir Neechal|K. Balachander|Nagesh, R. Muthuraman, Sowcar Janaki, Jayanthi|null|Kalakendra Productions|/wiki/Ethir

In [25]:
df=df.drop(0)
df

year                                 title               director  \
1   1968                            Anbu Vazhi             M. Natesan   
2   1968                     Andru Kanda Mugam        G. Ramakrishnan   
3   1968                           Bommalattam          V. Srinivasan   
4   1968                             Chakkaram          A. Kasilingam   
5   1968                         Deivega Uravu                Sathyam   
6   1968                        Delhi Mapillai                  Devan   
7   1968          DeviRemake of Malayalam film            A. K. Velan   
8   1968                         Ethir Neechal         K. Balachander   
9   1968                             En Thambi   A. C. Tirulokchandar   
10  1968                         Enga Oor Raja            P. Madhavan   
11  1968                      Galatta Kalyanam        C. V. Rajendran   
12  1968                           Harichandra      K. S. Prakash Rao   
13  1968                          Jeevanaamsam     Malliyam Rajagopal   
14  1968                      Kallum Kaniyagum             K. Shankar   
15  1968                               Kanavan         P. Neelakantan   
16  1968                    Kannan En Kadhalan         P. Neelakantan   
17  1968                       Kadhal Vaaganam       M. A. Thirumugam   
18  1968  Kollaikaran MaganDubbed from Kannada   A. V. Sheshagiri Rao   
19  1968                    Kudiyirundha Koyil             K. Shankar   
20  1968                       Kuzhanthaikkaga            P. Madhavan   
21  1968                      Lakshmi Kalyanam          G. Or. Nathan   
22  1968                         Moondrezhuthu          T. R. Ramanna   
23  1968                          Muthu Chippi            M. Krishnan   
24  1968                   Naalum Therindhavan      C. P. Jambulingam   
25  1968                     Neelagiri Express  Thirumalai-Mahalingam   
26  1968                         Neeyum Naanum          T. R. Ramanna   
27  1968                              Nervazhi         M. G. Balu Rao   
28  1968                         Nimirndhu Nil                  Devan   
29  1968                           Oli Vilakku          Tapi Chanakya   
30  1968                            Paal Manam    K. S. Sethumadhavan   
31  1968                      Panakkara Pillai      C. P. Jambulingam   
32  1968                         Panama Pasama   K. S. Gopalakrishnan   
33  1968                         Poovum Pottum            Dada Mirasi   
34  1968                        Pudhiya Bhoomi          Tapi Chanakya   
35  1968                          Puthisaligal                   Arun   
36  1968                   Ragasiya Police 115         B. R. Panthulu   
37  1968                  Sathiyam Thavaradhey         Pondy Selvaraj   
38  1968                       Selviyin Selvan        Mohan Gandhiram   
39  1968                         Siritha Mugam            V. V. Raman   
40  1968                Soaappu Seeppu Kannadi  Thirumalai-Mahalingam   
41  1968                           Teacheramma         S. R. Puttanna   
42  1968                       Thamarai Nenjam         K. Balachander   
43  1968                      Thaer Thiruvizha       M. A. Thirumugam   
44  1968                   Thillana Mohanambal        A. P. Nagarajan   
45  1968                      Thirumal Perumai        A. P. Nagarajan   
46  1968                     Uyarndha Manithan         Krishnan–Panju   
47  1968                         Uyira Maanama   K. S. Gopalakrishnan   

                                                 cast genre  \
1                Jaishankar, L. Vijayalakshmi, Nagesh  null   
2                  Ravichandran, Jayalalithaa, Nagesh  null   
3               Jaishankar, Jayalalithaa, Nagesh, Cho  null   
4   Gemini Ganesan, A. V. M. Rajan, Vennira Aadai ...  null   
5                          Jaishankar, Devika, Nagesh  null   
6                  Ravichandran, Rajasree, Cho, Sachu  null   
7                               R. Muthuraman, Devika  nul

In [26]:
getAllMusic()

In [27]:
getAllrating()

https://www.imdb.com/title/tt0456285/
Anbu Vazhi | 
https://www.imdb.com/title/tt0290419/
Andru Kanda Mugam | 6.4
https://www.imdb.com/title/tt0290478/
Bommalattam | 6.4
https://www.imdb.com/title/tt0290513/
Chakkaram | 
https://www.imdb.com/title/tt8938916/
Deivega Uravu | 
https://www.imdb.com/title/tt3442644/
Delhi Mapillai | 
https://www.imdb.com/title/tt3417422/
DeviRemake of Malayalam film | 8.3
https://www.imdb.com/title/tt0154462/
Ethir Neechal | 8.1
https://www.imdb.com/title/tt0290590/
En Thambi | 6.2
https://www.imdb.com/title/tt1440165/
Enga Oor Raja | 5.8
https://www.imdb.com/title/tt0290622/
Galatta Kalyanam | 7.5
https://www.imdb.com/title/tt0254375/
Harichandra | 8.8
https://www.imdb.com/title/tt0290680/
Jeevanaamsam | 
https://www.imdb.com/title/tt0290691/
Kallum Kaniyagum | 
https://www.imdb.com/title/tt0248128/
Kanavan | 7.2
https://www.imdb.com/title/tt0248129/
Kannan En Kadhalan | 6.5
https://www.imdb.com/title/tt0290693/
Kadhal Vaaganam | 6.7
https://www.imdb.com/

In [28]:
df

year                                 title               director  \
1   1968                            Anbu Vazhi             M. Natesan   
2   1968                     Andru Kanda Mugam        G. Ramakrishnan   
3   1968                           Bommalattam          V. Srinivasan   
4   1968                             Chakkaram          A. Kasilingam   
5   1968                         Deivega Uravu                Sathyam   
6   1968                        Delhi Mapillai                  Devan   
7   1968          DeviRemake of Malayalam film            A. K. Velan   
8   1968                         Ethir Neechal         K. Balachander   
9   1968                             En Thambi   A. C. Tirulokchandar   
10  1968                         Enga Oor Raja            P. Madhavan   
11  1968                      Galatta Kalyanam        C. V. Rajendran   
12  1968                           Harichandra      K. S. Prakash Rao   
13  1968                          Jeevanaamsam     Malliyam Rajagopal   
14  1968                      Kallum Kaniyagum             K. Shankar   
15  1968                               Kanavan         P. Neelakantan   
16  1968                    Kannan En Kadhalan         P. Neelakantan   
17  1968                       Kadhal Vaaganam       M. A. Thirumugam   
18  1968  Kollaikaran MaganDubbed from Kannada   A. V. Sheshagiri Rao   
19  1968                    Kudiyirundha Koyil             K. Shankar   
20  1968                       Kuzhanthaikkaga            P. Madhavan   
21  1968                      Lakshmi Kalyanam          G. Or. Nathan   
22  1968                         Moondrezhuthu          T. R. Ramanna   
23  1968                          Muthu Chippi            M. Krishnan   
24  1968                   Naalum Therindhavan      C. P. Jambulingam   
25  1968                     Neelagiri Express  Thirumalai-Mahalingam   
26  1968                         Neeyum Naanum          T. R. Ramanna   
27  1968                              Nervazhi         M. G. Balu Rao   
28  1968                         Nimirndhu Nil                  Devan   
29  1968                           Oli Vilakku          Tapi Chanakya   
30  1968                            Paal Manam    K. S. Sethumadhavan   
31  1968                      Panakkara Pillai      C. P. Jambulingam   
32  1968                         Panama Pasama   K. S. Gopalakrishnan   
33  1968                         Poovum Pottum            Dada Mirasi   
34  1968                        Pudhiya Bhoomi          Tapi Chanakya   
35  1968                          Puthisaligal                   Arun   
36  1968                   Ragasiya Police 115         B. R. Panthulu   
37  1968                  Sathiyam Thavaradhey         Pondy Selvaraj   
38  1968                       Selviyin Selvan        Mohan Gandhiram   
39  1968                         Siritha Mugam            V. V. Raman   
40  1968                Soaappu Seeppu Kannadi  Thirumalai-Mahalingam   
41  1968                           Teacheramma         S. R. Puttanna   
42  1968                       Thamarai Nenjam         K. Balachander   
43  1968                      Thaer Thiruvizha       M. A. Thirumugam   
44  1968                   Thillana Mohanambal        A. P. Nagarajan   
45  1968                      Thirumal Perumai        A. P. Nagarajan   
46  1968                     Uyarndha Manithan         Krishnan–Panju   
47  1968                         Uyira Maanama   K. S. Gopalakrishnan   

                                                 cast genre  \
1                Jaishankar, L. Vijayalakshmi, Nagesh  null   
2                  Ravichandran, Jayalalithaa, Nagesh  null   
3               Jaishankar, Jayalalithaa, Nagesh, Cho  null   
4   Gemini Ganesan, A. V. M. Rajan, Vennira Aadai ...  null   
5                          Jaishankar, Devika, Nagesh  null   
6                  Ravichandran, Rajasree, Cho, Sachu  null   
7                               R. Muthuraman, Devika  nul

In [29]:
df.to_csv('1968_movies.csv')

# 1967

In [33]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1967'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1967=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [34]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [35]:
for table in tables_1967:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1967'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Aalayam|Thirumalai Mahalingam|Major Sundarrajan, Srikanth, Nagesh|null|Sunbeam Productions|/wiki/Aalayam_(film)
2!Anubavam Pudhumai|C. V. Rajendran|R. Muthuraman, Rajasree, T. R. Ramachandran|null|Thirumagal Combines|/wiki/Anubavam_Pudhumai
3!Anubavi Raja Anubavi|K. Balachander|Nagesh, R. Muthuraman, Rajasree, Jayabharathi|null|Ayya Films|/wiki/Anubavi_Raja_Anubavi
4!Apoorva Piravigal[3]|G. Viswanatham|T. L. Kantha Rao, Vanisree|null|Lakshmi Saraswathi Movies|/w/index.php?title=Apoorva_Piravigal&action=edit&redlink=1
5!Arasa Kattalai|M. G. Chakrapani|M. G. Ramachandran, B. Saroja Devi, Jayalalithaa|null|Sathyaraja Pictures|/wiki/Arasa_Kattalai
6!Adhey Kangal|A. C. Tirulokchandar|Ravichandran, Kanchana, Nagesh|null|AVM Productions|/wiki/Adhey_Kangal
7!Bhakta Prahlada|CH. Narayana Moorthy|S. V. Ranga Rao, Anjali Devi, Roja Ramani|null|AVM Productions|/wiki/Bhakta_Prahlada_(1967_film)
8!Bama Vijayam|K. Balachander|Sowcar Janaki, R. Muth

In [36]:
df=df.drop(0)
df

year                    title               director  \
1   1967                  Aalayam  Thirumalai Mahalingam   
2   1967        Anubavam Pudhumai        C. V. Rajendran   
3   1967     Anubavi Raja Anubavi         K. Balachander   
4   1967     Apoorva Piravigal[3]         G. Viswanatham   
5   1967           Arasa Kattalai       M. G. Chakrapani   
6   1967             Adhey Kangal   A. C. Tirulokchandar   
7   1967          Bhakta Prahlada   CH. Narayana Moorthy   
8   1967             Bama Vijayam         K. Balachander   
9   1967                  Bhavani          T. R. Ramanna   
10  1967             Deiva Cheyal             M. G. Balu   
11  1967    Ethirigal Jakkirathai            R. Sundaram   
12  1967  Engalukkum Kaalam Varum             A. Vincent   
13  1967             Iru Malargal   A. C. Tirulokchandar   
14  1967      Kadhalithal Podhuma         K. V. Srinivas   
15  1967         Kan Kanda Deivam   K. S. Gopalakrishnan   
16  1967          Kandhan Karunai        A. P. Nagarajan   
17  1967                Karpooram       C. N. Shanmugham   
18  1967             Kaavalkaaran         P. Neelakantan   
19  1967    Maadi Veettu Mappilai         S. K. A. Chari   
20  1967                Magaraasi       M. A. Thirumugam   
21  1967        Manam Oru Kurangu     A. T. Krishnaswamy   
22  1967            Muhurtha Naal            P. Madhavan   
23  1967                     Naan          T. R. Ramanna   
24  1967      Naan Yaar Theriyuma          V. V. Ramanan   
25  1967        Nenjirukkum Varai          C. V. Sridhar   
26  1967       Ninaivil Nindraval          V. Srinivasan   
27  1967         Ooty Varai Uravu          C. V. Sridhar   
28  1967                 Paaladai           A. Bhimsingh   
29  1967                 Pandhyam          A. Kasilingam   
30  1967     Pattanathil Bhootham            M. V. Raman   
31  1967            Pattathu Rani          S. Ramanathan   
32  1967           Pen Endral Pen              Aroor Das   
33  1967         Penne Nee Vazhga            P. Madhavan   
34  1967            Pesum Dheivam   K. S. Gopalakrishnan   
35  1967           Ponnana Vazhvu           R. Devarajan   
36  1967        Raja Veetu Pillai            Dada Mirasi   
37  1967                  Rajathi          M. Lakshmanan   
38  1967                   Seetha        A. P. Nagarajan   
39  1967              Selva Magal       K. V. Srinivasan   
40  1967            Sabash Thambi     C. P. Jambulingham   
41  1967      Sri Krishnavatharam      K. Kameshwara Rao   
42  1967    Sundaramurthi Nayanar                K. Somu   
43  1967      Thaikku Thalaimagan       M. A. Thirumugam   
44  1967            Thanga Thambi         S. K. A. Chari   
45  1967                  Thangai   A. C. Tirulokchandar   
46  1967        Thiruvarutchelvar        A. P. Nagarajan   
47  1967           Uyir Mel Aasai     C. P. Jambulingham   
48  1967          Valiba Virundhu         Murasoli Maran   
49  1967                Vivasayee       M. A. Thirumugam   

                                                 cast genre  \
1                 Major Sundarrajan, Srikanth, Nagesh  null   
2         R. Muthuraman, Rajasree, T. R. Ramachandran  null   
3       Nagesh, R. Muthuraman, Rajasree, Jayabharathi  null   
4                          T. L. Kantha Rao, Vanisree  null   
5    M. G. Ramachandran, B. Saroja Devi, Jayalalithaa  null   
6                      Ravichandran, Kanchana, Nagesh  null   
7           S. V. Ranga Rao, Anjali Devi, Roja Ramani  null   
8   Sowcar Janaki, R. Muthuraman, Nagesh, Rajasree...  null   
9   Jaishankar, S. A. Ashokan, C. R. Vijayakumari,...  null   
10         Major Sundarrajan, R. Muthuraman, Bharathi  null   
11      Ravichandran, R. S. Manohar, L. Vijayalakshmi  null   
12             Gemini Ganesan, Padmini, Madhu, Nagesh  null   
13              Sivaji Ganesan, Padmini, K. R. Vijaya  null   
14         Jaishankar, Vanisri, Vennira Aadai Nirmala  null   
15                   Padmini, S. V. Ranga Rao, Nagesh  nul

In [37]:
getAllMusic()

ex.php?title=Apoorva_Piravigal&action=edit&redlink=1


en.wikipedia.org (parse) ex.php?title=Apoorva_Piravigal&action=ed...


Exception
4 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [38]:
getAllrating()

https://www.imdb.com/title/tt0443835/
Aalayam | 
https://www.imdb.com/title/tt1426990/
Anubavam Pudhumai | 
https://www.imdb.com/title/tt0154157/
Anubavi Raja Anubavi | 7.1
https://www.imdb.com/title/tt0457767/
Apoorva Piravigal[3] | 5.8
https://www.imdb.com/title/tt0157324/
Arasa Kattalai | 6.4
https://www.imdb.com/title/tt0457781/
Adhey Kangal | 7.5
https://www.imdb.com/title/tt0247204/
Bhakta Prahlada | 6.8
https://www.imdb.com/title/tt0154197/
Bama Vijayam | 7.4
https://www.imdb.com/title/tt0465917/
Bhavani | 
https://www.imdb.com/title/tt5954586/
Deiva Cheyal | 
https://www.imdb.com/title/tt0497369/
Ethirigal Jakkirathai | 
https://www.imdb.com/title/tt0465917/
Engalukkum Kaalam Varum | 
https://www.imdb.com/title/tt0497391/
Iru Malargal | 6.9
https://www.imdb.com/title/tt8983308/
Kadhalithal Podhuma | 
https://www.imdb.com/title/tt0214833/
Kan Kanda Deivam | 
https://www.imdb.com/title/tt0192198/
Kandhan Karunai | 7.1
https://www.imdb.com/title/tt0248131/
Karpooram | 7.1
https://

In [39]:
df

year                    title               director  \
1   1967                  Aalayam  Thirumalai Mahalingam   
2   1967        Anubavam Pudhumai        C. V. Rajendran   
3   1967     Anubavi Raja Anubavi         K. Balachander   
4   1967     Apoorva Piravigal[3]         G. Viswanatham   
5   1967           Arasa Kattalai       M. G. Chakrapani   
6   1967             Adhey Kangal   A. C. Tirulokchandar   
7   1967          Bhakta Prahlada   CH. Narayana Moorthy   
8   1967             Bama Vijayam         K. Balachander   
9   1967                  Bhavani          T. R. Ramanna   
10  1967             Deiva Cheyal             M. G. Balu   
11  1967    Ethirigal Jakkirathai            R. Sundaram   
12  1967  Engalukkum Kaalam Varum             A. Vincent   
13  1967             Iru Malargal   A. C. Tirulokchandar   
14  1967      Kadhalithal Podhuma         K. V. Srinivas   
15  1967         Kan Kanda Deivam   K. S. Gopalakrishnan   
16  1967          Kandhan Karunai        A. P. Nagarajan   
17  1967                Karpooram       C. N. Shanmugham   
18  1967             Kaavalkaaran         P. Neelakantan   
19  1967    Maadi Veettu Mappilai         S. K. A. Chari   
20  1967                Magaraasi       M. A. Thirumugam   
21  1967        Manam Oru Kurangu     A. T. Krishnaswamy   
22  1967            Muhurtha Naal            P. Madhavan   
23  1967                     Naan          T. R. Ramanna   
24  1967      Naan Yaar Theriyuma          V. V. Ramanan   
25  1967        Nenjirukkum Varai          C. V. Sridhar   
26  1967       Ninaivil Nindraval          V. Srinivasan   
27  1967         Ooty Varai Uravu          C. V. Sridhar   
28  1967                 Paaladai           A. Bhimsingh   
29  1967                 Pandhyam          A. Kasilingam   
30  1967     Pattanathil Bhootham            M. V. Raman   
31  1967            Pattathu Rani          S. Ramanathan   
32  1967           Pen Endral Pen              Aroor Das   
33  1967         Penne Nee Vazhga            P. Madhavan   
34  1967            Pesum Dheivam   K. S. Gopalakrishnan   
35  1967           Ponnana Vazhvu           R. Devarajan   
36  1967        Raja Veetu Pillai            Dada Mirasi   
37  1967                  Rajathi          M. Lakshmanan   
38  1967                   Seetha        A. P. Nagarajan   
39  1967              Selva Magal       K. V. Srinivasan   
40  1967            Sabash Thambi     C. P. Jambulingham   
41  1967      Sri Krishnavatharam      K. Kameshwara Rao   
42  1967    Sundaramurthi Nayanar                K. Somu   
43  1967      Thaikku Thalaimagan       M. A. Thirumugam   
44  1967            Thanga Thambi         S. K. A. Chari   
45  1967                  Thangai   A. C. Tirulokchandar   
46  1967        Thiruvarutchelvar        A. P. Nagarajan   
47  1967           Uyir Mel Aasai     C. P. Jambulingham   
48  1967          Valiba Virundhu         Murasoli Maran   
49  1967                Vivasayee       M. A. Thirumugam   

                                                 cast genre  \
1                 Major Sundarrajan, Srikanth, Nagesh  null   
2         R. Muthuraman, Rajasree, T. R. Ramachandran  null   
3       Nagesh, R. Muthuraman, Rajasree, Jayabharathi  null   
4                          T. L. Kantha Rao, Vanisree  null   
5    M. G. Ramachandran, B. Saroja Devi, Jayalalithaa  null   
6                      Ravichandran, Kanchana, Nagesh  null   
7           S. V. Ranga Rao, Anjali Devi, Roja Ramani  null   
8   Sowcar Janaki, R. Muthuraman, Nagesh, Rajasree...  null   
9   Jaishankar, S. A. Ashokan, C. R. Vijayakumari,...  null   
10         Major Sundarrajan, R. Muthuraman, Bharathi  null   
11      Ravichandran, R. S. Manohar, L. Vijayalakshmi  null   
12             Gemini Ganesan, Padmini, Madhu, Nagesh  null   
13              Sivaji Ganesan, Padmini, K. R. Vijaya  null   
14         Jaishankar, Vanisri, Vennira Aadai Nirmala  null   
15                   Padmini, S. V. Ranga Rao, Nagesh  nul

In [40]:
df.to_csv('1967_movies.csv')

# 1966

In [41]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1966'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1966=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [42]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [43]:
for table in tables_1966:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1966'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Producer', 'Music', 'Cast']
1!Anbe Vaa|A. C. Tirulokchandar|M. G. Ramachandran, B. Sarojadevi, Nagesh|null|A.V.M. Productions|/wiki/Anbe_Vaa
2!Annavin Aasai|Dada Mirasi|Gemini Ganesan, Savitri, K. Balaji, K. R. Vijaya|null|Sujatha Cine Arts|/wiki/Annavin_Aasai
3!Avan Pithana?|P. Neelakantan|S. S. Rajendran, C. R. Vijayakumari, S. V. Sahasranamam|null|Umayal Productions|/wiki/Avan_Pithana%3F
4!Chandhrodhayam|K. Shankar|M.G. Ramachandran, Jayalalitha, M. R. Radha|null|Saravana Films|/wiki/Chandhrodhayam
5!Chinnanchiru Ulagam|K. S. Gopalakrishnan|Gemini Ganesan, K. R. Vijaya, Nagesh, Magic Radhika|null|Chitra Productions|/wiki/Chinnanchiru_Ulagam
6!Chitthi|K. S. Gopalakrishnan|Gemini Ganesan, Padmini, M. R. Radha, R. Muthuraman, Vijayasree, Vijaya Nirmala, Baby Rani|null|Chitra Productions|/wiki/Chitthi
7!Enga Pappa|B. R. Panthulu|Ravichandran, Kumari Bharathi, Nagesh, K. A. Thangavelu, M. V. Rajamma, Manorama|null|Padmini Pictures|/wiki/Enga_Pappa
8!Gowri Kalyanam|

In [44]:
df=df.drop(0)
df

year                               title                        director  \
1   1966                            Anbe Vaa            A. C. Tirulokchandar   
2   1966                       Annavin Aasai                     Dada Mirasi   
3   1966                       Avan Pithana?                  P. Neelakantan   
4   1966                      Chandhrodhayam                      K. Shankar   
5   1966                 Chinnanchiru Ulagam            K. S. Gopalakrishnan   
6   1966                             Chitthi            K. S. Gopalakrishnan   
7   1966                          Enga Pappa                  B. R. Panthulu   
8   1966                      Gowri Kalyanam                      K. Shankar   
9   1966                     Iru Vallavargal                K. V. Srinivasan   
10  1966                Kathal Paduthum Padu             Joseph Thaliath Jr.   
11  1966                      Kattu Malligai                  P. Subramanyam   
12  1966                           Kodimalar                         Sridhar   
13  1966                         Kumari Penn                   T. R. Ramanna   
14  1966               Madras to Pondicherry           Thirumalai Mahalingam   
15  1966                    Mahakavi Kalidas                  R. R. Chandran   
16  1966                  Major Chandrakanth                  K. Balachander   
17  1966                        Mani Magudam                 S. S. Rajendran   
18  1966                  Marakka Mudiyumaa?                  Murasoli Maran   
19  1966               Motor Sundaram Pillai                            Balu   
20  1966                           Mugaraasi                M. A. Thirumugam   
21  1966                         Naam Moovar                     Jambulingam   
22  1966                    Naan Aanaiyittal                        Chanakya   
23  1966                              Nadodi                   B.R. Panthulu   
24  1966                Namma Veettu Lakshmi                  B. R. Panthulu   
25  1966  Panchali SabathamDubbed from Hindi                 Babubhai Mistri   
26  1966                     Parakkum Paavai                   T. R. Ramanna   
27  1966                     Periya Manidhan            K. C. Krishnamoorthy   
28  1966                 Petralthan Pillaiya                  Krishnan-Panju   
29  1966                                Ramu            A. C. Tirulokchandar   
30  1966                      Sadhu Mirandal                   A. Bhim Singh   
31  1966                  Saraswati Sabatham                 A. P. Nagarajan   
32  1966                              Selvam            K. S. Gopalakrishnan   
33  1966                      Thaali Bhagyam             K. P. Nagabhooshnam   
34  1966                         Thanipiravi                M. A. Thirumugam   
35  1966           Thattungal Thirakkappadum               J. P. Chandrababu   
36  1966                     Thaaye Unakkaga                     B. Pullaiah   
37  1966                    Thayin Mel Aanai                    G. R. Nathan   
38  1966             Thedi Vandha Thirumagal  Sathyam[disambiguation needed]   
39  1966                          Thenmazhai                   V. Srinivasan   
40  1966                     Vallavan Oruvan                     R. Sundaram   
41  1966                           Yaar Nee?                          Satyam   
42  1966                Yaarukkaga Azhudhaan                     Jayakanthan   

                                                 cast genre  \
1           M. G. Ramachandran, B. Sarojadevi, Nagesh  null   
2    Gemini Ganesan, Savitri, K. Balaji, K. R. Vijaya  null   
3   S. S. Rajendran, C. R. Vijayakumari, S. V. Sah...  null   
4         M.G. Ramachandran, Jayalalitha, M. R. Radha  null   
5   Gemini Ganesan, K. R. Vijaya, Nagesh, Magic Ra...  null   
6   Gemini Ganesan, Padmini, M. R. Radha, R. Muthu...  null   
7   Ravichandran, Kumari Bharathi, Nagesh, K. A. T...  null   
8               Jaishankar, Ravichandran, Jayalalitha  null

In [45]:
getAllMusic()

In [46]:
getAllrating()

https://www.imdb.com/title/tt0370279/
Anbe Vaa | 7.0
https://www.imdb.com/title/tt3454552/
Annavin Aasai | 
https://www.imdb.com/title/tt0247940/
Avan Pithana? | 
https://www.imdb.com/title/tt1476972/
Chandhrodhayam | 7.1
https://www.imdb.com/title/tt0214583/
Chinnanchiru Ulagam | 
https://www.imdb.com/title/tt0214586/
Chitthi | 6.9
https://www.imdb.com/title/tt0250360/
Enga Pappa | 
https://www.imdb.com/title/tt2043872/
Gowri Kalyanam | 
https://www.imdb.com/title/tt8971612/
Iru Vallavargal | 
https://www.imdb.com/title/tt5759438/
Kathal Paduthum Padu | 
https://www.imdb.com/title/tt0254461/
Kattu Malligai | 
https://www.imdb.com/title/tt1431093/
Kodimalar | 
https://www.imdb.com/title/tt0317859/
Kumari Penn | 6.0
https://www.imdb.com/title/tt2548220/
Madras to Pondicherry | 6.4
https://www.imdb.com/title/tt1441324/
Mahakavi Kalidas | 7.1
https://www.imdb.com/title/tt0154822/
Major Chandrakanth | 7.9
https://www.imdb.com/title/tt2043915/
Mani Magudam | 
https://www.imdb.com/title/tt14

In [47]:
df

year                               title                        director  \
1   1966                            Anbe Vaa            A. C. Tirulokchandar   
2   1966                       Annavin Aasai                     Dada Mirasi   
3   1966                       Avan Pithana?                  P. Neelakantan   
4   1966                      Chandhrodhayam                      K. Shankar   
5   1966                 Chinnanchiru Ulagam            K. S. Gopalakrishnan   
6   1966                             Chitthi            K. S. Gopalakrishnan   
7   1966                          Enga Pappa                  B. R. Panthulu   
8   1966                      Gowri Kalyanam                      K. Shankar   
9   1966                     Iru Vallavargal                K. V. Srinivasan   
10  1966                Kathal Paduthum Padu             Joseph Thaliath Jr.   
11  1966                      Kattu Malligai                  P. Subramanyam   
12  1966                           Kodimalar                         Sridhar   
13  1966                         Kumari Penn                   T. R. Ramanna   
14  1966               Madras to Pondicherry           Thirumalai Mahalingam   
15  1966                    Mahakavi Kalidas                  R. R. Chandran   
16  1966                  Major Chandrakanth                  K. Balachander   
17  1966                        Mani Magudam                 S. S. Rajendran   
18  1966                  Marakka Mudiyumaa?                  Murasoli Maran   
19  1966               Motor Sundaram Pillai                            Balu   
20  1966                           Mugaraasi                M. A. Thirumugam   
21  1966                         Naam Moovar                     Jambulingam   
22  1966                    Naan Aanaiyittal                        Chanakya   
23  1966                              Nadodi                   B.R. Panthulu   
24  1966                Namma Veettu Lakshmi                  B. R. Panthulu   
25  1966  Panchali SabathamDubbed from Hindi                 Babubhai Mistri   
26  1966                     Parakkum Paavai                   T. R. Ramanna   
27  1966                     Periya Manidhan            K. C. Krishnamoorthy   
28  1966                 Petralthan Pillaiya                  Krishnan-Panju   
29  1966                                Ramu            A. C. Tirulokchandar   
30  1966                      Sadhu Mirandal                   A. Bhim Singh   
31  1966                  Saraswati Sabatham                 A. P. Nagarajan   
32  1966                              Selvam            K. S. Gopalakrishnan   
33  1966                      Thaali Bhagyam             K. P. Nagabhooshnam   
34  1966                         Thanipiravi                M. A. Thirumugam   
35  1966           Thattungal Thirakkappadum               J. P. Chandrababu   
36  1966                     Thaaye Unakkaga                     B. Pullaiah   
37  1966                    Thayin Mel Aanai                    G. R. Nathan   
38  1966             Thedi Vandha Thirumagal  Sathyam[disambiguation needed]   
39  1966                          Thenmazhai                   V. Srinivasan   
40  1966                     Vallavan Oruvan                     R. Sundaram   
41  1966                           Yaar Nee?                          Satyam   
42  1966                Yaarukkaga Azhudhaan                     Jayakanthan   

                                                 cast genre  \
1           M. G. Ramachandran, B. Sarojadevi, Nagesh  null   
2    Gemini Ganesan, Savitri, K. Balaji, K. R. Vijaya  null   
3   S. S. Rajendran, C. R. Vijayakumari, S. V. Sah...  null   
4         M.G. Ramachandran, Jayalalitha, M. R. Radha  null   
5   Gemini Ganesan, K. R. Vijaya, Nagesh, Magic Ra...  null   
6   Gemini Ganesan, Padmini, M. R. Radha, R. Muthu...  null   
7   Ravichandran, Kumari Bharathi, Nagesh, K. A. T...  null   
8               Jaishankar, Ravichandran, Jayalalitha  null

In [48]:
df.to_csv('1966_movies.csv')

# 1965

In [52]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1965'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1965=soup.find_all('table')[0:1]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test',
          'music':'test'}
df=df.append(data_2019,ignore_index=True)

In [53]:
df

year title director  cast genre producer  link music
0  2017  test     test  test  test     test  test  test

In [54]:
for table in tables_1965:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    music_index=5
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    music_member=False
    producer_member=False
    cast_member=False
    yr='1965'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')
      producer_member=True
    if 'Music' in column_names:
      music_index=column_names.index('Music')
      music_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+1].text.strip() if music_member else ''))]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index+2].text.strip() if music_member else ''))]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''),re.sub(r'\d+', '', (all_data[music_index].text.strip() if music_member else 'null'))]


['Title', 'Director', 'Production', 'Music', 'Cast']
1!Aasai Mugam|P. Pullaiah|M. G. Ramachandran, B. Sarojadevi, M. N. Nambiar|null|Mohan Productions|/wiki/Aasai_Mugam
2!Aavathellaam Pennaale[3]Dubbed from Telugu Dhampathyam 1957 film|E. Appa Rao|Dialogues: Chandar[4]|null|Lingampatti Muthaiah (Nadar Films)|/w/index.php?title=Aavathellaam_Pennaale&action=edit&redlink=1
3!Ayirathil Oruvan|B. R. Panthulu|M. G. Ramachandran, Jayalalitha, M. N. Nambiar|null|Padmini Pictures|/wiki/Aayirathil_Oruvan_(1965_film)
4!Anandhi|P. Neelakantan|S. S. Rajendran, C. R. Vijayakumari, M. R. Radha|null|A.L.S. Productions|/wiki/Anandhi_(film)
5!Anbu Karangal|K. Shankar|Sivaji Ganesan, Manimala Devika, Nagesh|null|Santhi Films|/wiki/Anbu_Karangal
6!Enga Veetu Penn|Tapi Chanakya|A. V. M. Rajan, Jaishankar, Vijaya Nirmala, Vasantha|null|Vijaya Combines Productions|/wiki/Enga_Veetu_Penn
7!Enga Veettu Pillai|Tapi Chanakya|M. G. Ramachandran, B. Sarojadevi, S. V. Ranga Rao|null|Vijaya Combines Productions|/wiki

In [55]:
df=df.drop(0)
df

year                                              title  \
1   1965                                        Aasai Mugam   
2   1965  Aavathellaam Pennaale[3]Dubbed from Telugu Dha...   
3   1965                                   Ayirathil Oruvan   
4   1965                                            Anandhi   
5   1965                                      Anbu Karangal   
6   1965                                    Enga Veetu Penn   
7   1965                                 Enga Veettu Pillai   
8   1965                                    Ennathan Mudivu   
9   1965                              Hello Mister Zamindar   
10  1965                                     Idhaya Kamalam   
11  1965                                     Iravum Pagalum   
12  1965                                    Kaakum Karangal   
13  1965                                        Kaattu Rani   
14  1965                                Kalangarai Vilakkam   
15  1965                                   Kalyana Mandapam   
16  1965                                        Kanni Thaai   
17  1965                                   Karthigai Deepam   
18  1965                              Kuzhandaiyum Deivamum   
19  1965                                        Maganey Kel   
20  1965                                  Maha Bharatham[5]   
21  1965                                             Naanal   
22  1965                                               Nee!   
23  1965                                       Neela Vaanam   
24  1965                                        Neerkumizhi   
25  1965                                          Oru Viral   
26  1965                                    Paditha Manaivi   
27  1965                                  Panam Padaithavan   
28  1965                                Panam Tharum Parisu   
29  1965                                   Panchavarna Kili   
30  1965                                  Pandava Vanavasam   
31  1965                                            Pazhani   
32  1965                             Poojaikku Vandha Malar   
33  1965                                           Poomalai   
34  1965                                        Sarasa B.A.   
35  1965                                             Santhi   
36  1965                                     Thayin Karunai   
37  1965                                    Thaayum Magalum   
38  1965                                         Thazhampoo   
39  1965                                    Thiruvilaiyadal   
40  1965                                    Unnaipol Oruvan   
41  1965                              Vallavanukku Vallavan   
42  1965                                        Vazhikaatti   
43  1965                                    Vaazhkai Padagu   
44  1965                                    Veera Abhimanyu   
45  1965                                      Vennira Aadai   
46  1965                                    Vilakketriyaval   

                director                                               cast  \
1            P. Pullaiah   M. G. Ramachandran, B. Sarojadevi, M. N. Nambiar   
2            E. Appa Rao                              Dialogues: Chandar[4]   
3         B. R. Panthulu     M. G. Ramachandran, Jayalalitha, M. N. Nambiar   
4         P. Neelakantan   S. S. Rajendran, C. R. Vijayakumari, M. R. Radha   
5             K. Shankar            Sivaji Ganesan, Manimala Devika, Nagesh   
6          Tapi Chanakya  A. V. M. Rajan, Jaishankar, Vijaya Nirmala, Va...   
7          Tapi Chanakya  M. G. Ramachandran, B. Sarojadevi, S. V. Ranga...   
8   K. S. Gopalakrishnan  A. V. M. Rajan, Anjali Devi, Vasanthi, Vennira...   
9        K. J. Mahadevan               Gemini Ganesan, Savitri, M. R. Radha   
10          L. V. Prasad                 Ravichandran, K. R. Vijaya, Nagesh   
11   Joseph Thaliath Jr.                       Jaishankar, Vasantha, Nagesh   
12  A. C. Tirulokchandar  S. S. Rajendran, S. V. Ranga Rao, C. R. Vijaya...   
13      M. A. T

In [56]:
getAllMusic()

In [57]:
getAllrating()

https://www.imdb.com/title/tt0255809/
Aasai Mugam | 6.6
https://www.imdb.com/search/title/?genres=comedy&languages=te&sort=moviemeter
Aavathellaam Pennaale[3]Dubbed from Telugu Dhampathyam 1957 film | 
https://www.imdb.com/title/tt0250227/
Ayirathil Oruvan | 7.1
https://www.imdb.com/title/tt0247925/
Anandhi | 
https://www.imdb.com/title/tt1435454/
Anbu Karangal | 
https://www.imdb.com/title/tt0319358/
Enga Veetu Penn | 
https://www.imdb.com/title/tt0157581/
Enga Veettu Pillai | 7.1
https://www.imdb.com/title/tt0215752/
Ennathan Mudivu | 
https://www.imdb.com/title/tt1463174/
Hello Mister Zamindar | 6.1
https://www.imdb.com/title/tt5296952/
Idhaya Kamalam | 
https://www.imdb.com/title/tt11700268/
Iravum Pagalum | 
https://www.imdb.com/title/tt8993888/
Kaakum Karangal | 
https://www.imdb.com/title/tt6166100/
Kaattu Rani | 
https://www.imdb.com/title/tt1476997/
Kalangarai Vilakkam | 6.4
Kalyana Mandapam | 
https://www.imdb.com/title/tt1475372/
Kanni Thaai | 6.7
https://www.imdb.com/title/

In [58]:
df

year                                              title  \
1   1965                                        Aasai Mugam   
2   1965  Aavathellaam Pennaale[3]Dubbed from Telugu Dha...   
3   1965                                   Ayirathil Oruvan   
4   1965                                            Anandhi   
5   1965                                      Anbu Karangal   
6   1965                                    Enga Veetu Penn   
7   1965                                 Enga Veettu Pillai   
8   1965                                    Ennathan Mudivu   
9   1965                              Hello Mister Zamindar   
10  1965                                     Idhaya Kamalam   
11  1965                                     Iravum Pagalum   
12  1965                                    Kaakum Karangal   
13  1965                                        Kaattu Rani   
14  1965                                Kalangarai Vilakkam   
15  1965                                   Kalyana Mandapam   
16  1965                                        Kanni Thaai   
17  1965                                   Karthigai Deepam   
18  1965                              Kuzhandaiyum Deivamum   
19  1965                                        Maganey Kel   
20  1965                                  Maha Bharatham[5]   
21  1965                                             Naanal   
22  1965                                               Nee!   
23  1965                                       Neela Vaanam   
24  1965                                        Neerkumizhi   
25  1965                                          Oru Viral   
26  1965                                    Paditha Manaivi   
27  1965                                  Panam Padaithavan   
28  1965                                Panam Tharum Parisu   
29  1965                                   Panchavarna Kili   
30  1965                                  Pandava Vanavasam   
31  1965                                            Pazhani   
32  1965                             Poojaikku Vandha Malar   
33  1965                                           Poomalai   
34  1965                                        Sarasa B.A.   
35  1965                                             Santhi   
36  1965                                     Thayin Karunai   
37  1965                                    Thaayum Magalum   
38  1965                                         Thazhampoo   
39  1965                                    Thiruvilaiyadal   
40  1965                                    Unnaipol Oruvan   
41  1965                              Vallavanukku Vallavan   
42  1965                                        Vazhikaatti   
43  1965                                    Vaazhkai Padagu   
44  1965                                    Veera Abhimanyu   
45  1965                                      Vennira Aadai   
46  1965                                    Vilakketriyaval   

                director                                               cast  \
1            P. Pullaiah   M. G. Ramachandran, B. Sarojadevi, M. N. Nambiar   
2            E. Appa Rao                              Dialogues: Chandar[4]   
3         B. R. Panthulu     M. G. Ramachandran, Jayalalitha, M. N. Nambiar   
4         P. Neelakantan   S. S. Rajendran, C. R. Vijayakumari, M. R. Radha   
5             K. Shankar            Sivaji Ganesan, Manimala Devika, Nagesh   
6          Tapi Chanakya  A. V. M. Rajan, Jaishankar, Vijaya Nirmala, Va...   
7          Tapi Chanakya  M. G. Ramachandran, B. Sarojadevi, S. V. Ranga...   
8   K. S. Gopalakrishnan  A. V. M. Rajan, Anjali Devi, Vasanthi, Vennira...   
9        K. J. Mahadevan               Gemini Ganesan, Savitri, M. R. Radha   
10          L. V. Prasad                 Ravichandran, K. R. Vijaya, Nagesh   
11   Joseph Thaliath Jr.                       Jaishankar, Vasantha, Nagesh   
12  A. C. Tirulokchandar  S. S. Rajendran, S. V. Ranga Rao, C. R. Vijaya...   
13      M. A. T

In [59]:
df.to_csv('1965_movies.csv')